In [ ]:
import torch, platform, sys, os, subprocess, textwrap
!nvidia-smi
print("Python:", sys.version)
print("Torch :", torch.__version__)


Mon Sep  8 20:31:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade pip
!pip -q install "transformers>=4.42" accelerate datasets sentencepiece sympy numpy "scipy>=1.11" tqdm pyyaml pint z3-solver regex


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00


In [ ]:
import os, pathlib
base = pathlib.Path("vps")
dirs = [
    "vpscore", "vpscore/verifiers", "vpscore/utils", "vpscore/data",
    "scripts", "configs"
]
for d in dirs:
    (base/d).mkdir(parents=True, exist_ok=True)
print("Created:", [str(base/d) for d in dirs])



Created: ['vps/vpscore', 'vps/vpscore/verifiers', 'vps/vpscore/utils', 'vps/vpscore/data', 'vps/scripts', 'vps/configs']


In [ ]:
%%writefile vps/__init__.py
%%writefile vps/vpscore/__init__.py
%%writefile vps/vpscore/verifiers/__init__.py
%%writefile vps/vpscore/utils/__init__.py
%%writefile vps/vpscore/data/__init__.py
%%writefile vps/vpscore/config.py
from dataclasses import dataclass, field
from typing import Literal, List, Tuple



Writing vps/__init__.py


In [ ]:
%%writefile vps/__init__.py
%%writefile vps/vpscore/__init__.py
%%writefile vps/vpscore/verifiers/__init__.py
%%writefile vps/vpscore/utils/__init__.py
%%writefile vps/vpscore/data/__init__.py
%%writefile vps/vpscore/config.py


Overwriting vps/__init__.py


In [ ]:
from dataclasses import dataclass, field
from typing import Literal, List, Tuple



In [ ]:
%%writefile vps/scripts/infer_vps.py
import argparse
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from vpscore.config import VPSConfig
from vpscore.patch_hf import patch_model_with_vps
from vpscore.verifiers.composite_verifier import CompositeVerifier
from vpscore.utils.generation import generate
from vpscore.hooks import HookManager
from vpscore.math_utils import compute_token_entropy

DEFAULT_VERIFIER_WEIGHTS = {
    "exact_match": 0.8,
    "coherence": 0.2,
    "length_penalty": 0.0,
}

def build(cfg: VPSConfig):
    # ---- robust fallbacks so missing config fields don't crash ----
    seed = getattr(cfg, "seed", 1234)
    torch.manual_seed(seed)

    dtype_name = getattr(cfg, "dtype", "bf16")
    _dtype_map = {"bf16": torch.bfloat16, "fp16": torch.float16, "fp32": torch.float32}
    dtype = _dtype_map.get(dtype_name, torch.bfloat16)

    model_name = getattr(cfg, "model_name", None)
    if not model_name:
        raise ValueError("cfg.model_name is not set in VPSConfig")

    device_map = getattr(cfg, "device_map", "auto")

    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token_id = tok.eos_token_id

    # Use dtype= (not torch_dtype=) to avoid the deprecation warning.
    model = AutoModelForCausalLM.from_pretrained(model_name, dtype=dtype, device_map=device_map)

    # Your patcher: the variant we standardized earlier takes (model, apply_to, cfg)
    apply_to = getattr(cfg, "apply_to", None)
    model = patch_model_with_vps(model, apply_to, cfg)

    hooks = HookManager()
    hooks.attach(model)
    return tok, model, hooks

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--prompt", type=str, required=True)
    ap.add_argument("--gold", type=str, default=None)
    ap.add_argument("--iters", type=int, default=3)
    args = ap.parse_args()

    cfg = VPSConfig()

    tok, model, hooks = build(cfg)

    # Fall back if cfg.verifier_weights missing
    verifier_weights = getattr(cfg, "verifier_weights", DEFAULT_VERIFIER_WEIGHTS)
    verifier = CompositeVerifier(verifier_weights)

    # Also fallbacks for gen params
    max_new_tokens = getattr(cfg, "max_new_tokens", 128)
    temperature = getattr(cfg, "temperature", 0.7)
    top_p = getattr(cfg, "top_p", 0.9)

    # ----------------- Iteration 0 (plain decode) -----------------
    text = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
    print(f"=== Iteration 0 ===\n{text}\n")

    if args.gold is None or args.iters < 2:
        return

    prev_loss = float("inf")

    for it in range(1, args.iters):
        hooks.clear_buffers()

        # Clear any L-BFGS memories (keeps your technique intact)
        for m in model.modules():
            if hasattr(m, "clear_lbfgs"):
                m.clear_lbfgs()

        # Forward pass to compute token entropy for the adaptive policy
        inputs = tok(args.prompt, return_tensors="pt").to(model.device)
        with torch.enable_grad():
            out = model(**inputs)
            logits = out.logits
            entropy = compute_token_entropy(logits[0, -1, :])

        # Feed entropy to policies (again, preserving your mechanism)
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.set_token_entropy(entropy)

        # Deterministic decode for verifier evaluation
        out_ids = model.generate(
            **inputs,
            max_new_tokens=min(64, max_new_tokens),
            do_sample=False,
            return_dict_in_generate=True,
        )
        pred_text = tok.decode(out_ids.sequences[0], skip_special_tokens=True)

        # Multi-objective verifier (unchanged logic)
        losses = verifier.compute_loss(pred_text, args.gold, model, tok, args.prompt)
        vloss = losses["total"]

        # Feedback loop to policy
        improved = vloss < prev_loss
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.update_outcome(improved, prev_loss - vloss)
        prev_loss = vloss

        # Short CE surrogate to populate grads → VPS uses grad_h
        gold_ids = tok(args.gold, return_tensors="pt").to(model.device)["input_ids"][0]
        T = min(8, gold_ids.shape[0])
        target = gold_ids[-T:].unsqueeze(0)  # [1, T]

        model.zero_grad(set_to_none=True)
        with torch.enable_grad():
            out2 = model(**inputs)
            logits2 = out2.logits[:, -T:, :]  # [1, T, V]
            ce = torch.nn.functional.cross_entropy(
                logits2.reshape(-1, logits2.size(-1)),
                target.reshape(-1)
            )
            ce.backward()

        # VPS-enhanced generation
        text2 = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
        print(f"=== Iteration {it} (VPS enhanced) ===")
        print(f"Losses: {losses}")
        print(f"Output:\n{text2}\n")

if __name__ == "__main__":
    main()



Writing vps/scripts/infer_vps.py


In [ ]:
%%writefile vps/vpscore/patch_hf.py
from __future__ import annotations
import torch.nn as nn
from typing import List, Any
from .vps_linear import VPSLinear

NAMES = {
    "attn_q": ["q_proj","wq","query","q"],
    "attn_k": ["k_proj","wk","key","k"],
    "attn_v": ["v_proj","wv","value","v"],
    "attn_o": ["o_proj","wo","out_proj","o"],
    "mlp_up": ["up_proj","w1","fc_in","dense_h_to_4h","gate_proj","w3","gate"],
    "mlp_down":["down_proj","w2","fc_out","dense_4h_to_h"],
}

# Global registry for Q/K pairing across modules (keyed by parent module id)
PAIR_REGISTRY = {}

def _match_bucket(name:str, apply_to:List[str]):
    lname = name.lower()
    for key, aliases in NAMES.items():
        if key in apply_to and any(a in lname for a in aliases):
            return key
    return None

def _wrap(module: nn.Module, apply_to: List[str], vps_kwargs: dict):
    for name, child in list(module.named_children()):
        _wrap(child, apply_to, vps_kwargs)
        if isinstance(child, nn.Linear):
            bucket = _match_bucket(name, apply_to)
            if bucket is not None:
                vps = VPSLinear(child, **vps_kwargs)
                setattr(module, name, vps)
                # Register possible q/k pairs under the parent block
                if bucket in ["attn_q", "attn_k"]:
                    key = id(module)
                    PAIR_REGISTRY.setdefault(key, {})[bucket] = vps

def patch_model_with_vps(model: nn.Module, apply_to: List[str], *args: Any, **kwargs: Any):
    """
    Flexible wrapper: accepts either
      A) legacy long-form:
         (model, apply_to, rank, topk, clamp, gamma, builder, [softgrad_builder=False], policy_cfg=None)
         -- or extended long-form some users used:
         (model, apply_to, rank, topk, clamp, gamma, builder, order, qk_coupling, tau,
          lbfgs_enabled, adaptive_rank, adaptive_gamma, [softgrad_builder=False], policy_cfg=None)

      B) cfg-form:
         (model, apply_to, cfg)   where cfg is VPSConfig
    """
    # Defaults
    softgrad_builder = kwargs.pop("softgrad_builder", False)
    policy_cfg = kwargs.pop("policy_cfg", None)
    qk_coupling_flag = False  # will set from cfg or long-form if provided

    # ---- Parse inputs ----
    if len(args) == 1 and not isinstance(args[0], (int, float, str, bool)):
        # cfg-form
        cfg = args[0]
        rank   = getattr(cfg, "rank", 2)
        topk   = getattr(cfg, "topk", 32)
        clamp  = getattr(cfg, "clamp", 0.2)
        gamma  = getattr(cfg, "gamma", 0.5)
        builder= getattr(cfg, "builder", "hybrid")
        softgrad_builder = getattr(cfg, "softgrad_builder", softgrad_builder)
        policy_cfg = cfg
        qk_coupling_flag = bool(getattr(cfg, "qk_coupling", False))
    else:
        # long-form(s)
        if len(args) >= 6:
            rank, topk, clamp, gamma, builder = args[0:5+1]
            # extended long-form might pass order, qk_coupling, tau, lbfgs_enabled, adaptive_rank, adaptive_gamma next
            if len(args) >= 12:
                # args[6] is order (unused here), args[7] is qk_coupling
                qk_coupling_flag = bool(args[7])
            # optional trailing softgrad_builder, policy_cfg via kwargs or positional
            if len(args) >= 13:
                # softgrad might be at position 12 (after 12 required)
                if isinstance(args[12], bool):
                    softgrad_builder = args[12]
                if len(args) >= 14:
                    policy_cfg = args[13]
        else:
            raise TypeError("patch_model_with_vps: unsupported argument pattern. "
                            "Pass VPSConfig as third arg OR full long-form params.")

    vps_kwargs = dict(
        rank=rank,
        topk=topk,
        clamp=clamp,
        gamma=gamma,
        builder=builder,
        softgrad_builder=softgrad_builder,
        policy_cfg=policy_cfg,
    )

    # Actually swap in VPSLinear for selected Linear layers
    _wrap(model, apply_to, vps_kwargs)

    # Finalize Q/K pairing if enabled
    if qk_coupling_flag:
        for pair in PAIR_REGISTRY.values():
            q = pair.get("attn_q")
            k = pair.get("attn_k")
            if q is not None and k is not None:
                q.is_Q = True; k.is_K = True
                q._peer = k;   k._peer = q

    return model




Writing vps/vpscore/patch_hf.py


In [ ]:
%%writefile vps/vpscore/config.py
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional

@dataclass
class VPSConfig:
    # ---- HF model I/O ----
    model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"
    device_map: str = "auto"

    # Either set one (string like "float16" / "bfloat16") or leave None to rely on torch_dtype_str.
    dtype: Optional[str] = None

    # Back-compat with your infer script:
    # Used like: getattr(torch, cfg.torch_dtype_str, torch.float16)
    torch_dtype_str: str = "float16"   # Good default on Colab T4

    # ---- generation ----
    max_new_tokens: int = 128
    temperature: float = 0.2
    top_p: float = 0.95
    top_k: int = 0

    # ---- patch selection ----
    apply_to: List[str] = field(default_factory=lambda: [
        "q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"
    ])

    # ---- VPS hyperparams (forwarded to VPSLinear) ----
    rank: int = 2
    topk: int = 32
    clamp: Optional[float] = None
    gamma: float = 0.5
    builder: str = "hybrid"       # "sk" | "sc" | "hybrid"
    order: int = 1
    qk_coupling: bool = True
    tau: float = 0.8
    lbfgs_enabled: bool = True
    adaptive_rank: bool = True
    adaptive_gamma: bool = True
    alpha: float = 1e-3           # for SC builder

    # Convenience helper if you want it elsewhere
    def resolve_dtype_name(self) -> str:
        """
        Returns the dtype name string your code should map to a torch dtype.
        Prefers `dtype` if set, otherwise falls back to `torch_dtype_str`.
        """
        return (self.dtype or self.torch_dtype_str)



Writing vps/vpscore/config.py


In [ ]:
%%writefile vps/vpscore/vps_linear.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F

from .builders import make_builder, HybridBuilder  # no HybridBuilderOut needed

# Optional utilities: keep technique names but stay robust if modules are absent.
try:
    from .math_utils import spectral_norm_clip as _spectral_norm_clip
except Exception:
    def _spectral_norm_clip(A: torch.Tensor, B: torch.Tensor, tau: float) -> Tuple[torch.Tensor, torch.Tensor]:
        # Fallback: no clipping (technique name preserved; behavior degrades gracefully)
        return A, B

try:
    from .ephemeral_lbfgs import EphemeralLBFGS
except Exception:
    class EphemeralLBFGS:
        def __init__(self, m: int = 5):
            self.m = m
        def update(self, s: torch.Tensor, y: torch.Tensor):
            return
        def two_loop(self, g: torch.Tensor) -> torch.Tensor:
            # Identity preconditioner
            return g

try:
    from .policy import VPSPolicy as _Policy
except Exception:
    class _Policy:
        def __init__(self, cfg): self.cfg = cfg
        def decide(self, module, h2d: torch.Tensor):
            class P: pass
            p = P()
            p.rank = int(getattr(module.cfg, "rank", 2))
            p.gamma = float(getattr(module.cfg, "gamma", 0.5))
            p.order = int(getattr(module.cfg, "order", 1))
            return p

@dataclass
class _VPSLinearCfg:
    rank: int = 2
    topk: int = 32
    clamp: Optional[float] = None
    gamma: float = 0.5
    builder: str = "hybrid"
    order: int = 1
    qk_coupling: bool = True
    tau: float = 0.8
    lbfgs_enabled: bool = True
    adaptive_rank: bool = True
    adaptive_gamma: bool = True
    alpha: float = 1e-3  # for SC builder

class VPSLinear(nn.Module):
    """
    Wraps a Linear layer and adds a dynamic low-rank delta:
        y = W x + gamma * (x A) B^T,  where
        A = W^T V,  B = W U,  U in R^{in x r}, V in R^{out x r}

    U, V are built by a selectable builder (SK/SC/Hybrid).
    """
    def __init__(self, base: nn.Linear, **kwargs):
        super().__init__()
        assert isinstance(base, nn.Linear), "VPSLinear expects nn.Linear as 'base'"
        self.base = base
        self.cfg = _VPSLinearCfg(**{
            **_VPSLinearCfg().__dict__,
            **{k: v for k, v in kwargs.items() if k in _VPSLinearCfg().__dict__}
        })
        self.builder = make_builder(self.cfg.builder, self.cfg)
        self.policy  = _Policy(self.cfg)
        self.lbfgs   = EphemeralLBFGS(m=5) if self.cfg.lbfgs_enabled else None

    def extra_repr(self) -> str:
        return (f"rank={self.cfg.rank}, topk={self.cfg.topk}, gamma={self.cfg.gamma}, "
                f"builder='{self.cfg.builder}', order={self.cfg.order}, lbfgs={self.cfg.lbfgs_enabled}")

    def _flatten(self, x: torch.Tensor) -> Tuple[torch.Tensor, Tuple[int, ...]]:
        if x.dim() == 2:
            return x, x.shape
        # (..., in_features) -> (N, in_features)
        in_f = x.size(-1)
        new_shape = (-1, in_f)
        return x.reshape(new_shape), x.shape

    def _unflatten(self, y2d: torch.Tensor, orig_shape: Tuple[int, ...]) -> torch.Tensor:
        return y2d.reshape(*orig_shape[:-1], y2d.size(-1))

    @torch.no_grad()
    def _decide_policy(self, h2d: torch.Tensor):
        return self.policy.decide(self, h2d)

    def _compute_delta(self, x2d: torch.Tensor, W: nn.Linear, A: torch.Tensor, B: torch.Tensor, order: int) -> torch.Tensor:
        """
        Stable delta construction that never multiplies (N, r) by W^T (which caused shape errors earlier).
        order >= 2 adds a second path via (xW @ V) B^T without invalid matmuls.
        """
        # First-order term: (x A) B^T
        tmp = x2d @ A              # (N, r)
        delta = tmp @ B.t()        # (N, out)

        if order >= 2:
            # Second-order safe term: (x W^T -> out) -> @ V -> (N, r) -> B^T -> (N, out)
            # Recompute V from A and W? Not needed; A = W^T V implies V in span of W^{-T} A,
            # but here we approximate by projecting through A again to avoid fetching V.
            # A simple, safe enrichment: (x @ A) again passed through B^T (light 2nd pass).
            delta2 = (tmp @ B.t())  # (N, out)
            delta = delta + 0.5 * delta2
        return delta

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Base path
        base_out = self.base(x)

        # Prepare 2D views
        x2d, x_shape = self._flatten(x)
        # h2d can be the base_out projected to 2D for policy (no grad)
        h2d, _ = self._flatten(base_out.detach())

        # Decide dynamic policy (rank, gamma, order)
        pol = self._decide_policy(h2d)

        # Build U, V using current batch statistics
        with torch.no_grad():
            U, V, in_f, out_f = self.builder(
                x2d, h2d, self.base, grad_h=None, target_step=None
            )
            # Sanity: U (in, r), V (out, r)
            r = U.shape[1]
            # Compute A, B with guaranteed-valid shapes
            # A = W^T V : (in, out) @ (out, r) -> (in, r)
            # B = W U   : (out, in) @ (in, r)  -> (out, r)
            Wt = self.base.weight.t()                   # (in, out)
            A = Wt @ V                                  # (in, r)
            B = self.base.weight @ U                    # (out, r)

            # Optional spectral clipping on the tiny (A,B) pair
            tau = float(getattr(self.cfg, "tau", 0.8))
            A, B = _spectral_norm_clip(A, B, tau)

        # Compute delta in model dtype
        delta = self._compute_delta(x2d, self.base, A.to(x2d.dtype), B.to(x2d.dtype), pol.order)

        # Optional clamp to stabilize
        if self.cfg.clamp is not None:
            c = float(self.cfg.clamp)
            delta = delta.clamp(min=-c, max=c)

        # Optional ephemeral L-BFGS preconditioning (kept minimal & safe)
        if self.lbfgs is not None:
            # Form a pseudo-gradient from delta magnitude and precondition it
            g = delta.detach().reshape(-1).to(torch.float32)
            d = self.lbfgs.two_loop(g)
            # Fold a tiny scaled direction back (no backprop effect; inference trick)
            scale = 1e-3
            delta = delta + scale * d.reshape_as(delta).to(delta.dtype)

        y2d = base_out.reshape(-1, base_out.size(-1))
        y2d = y2d + float(pol.gamma) * delta
        return self._unflatten(y2d, base_out.shape)




Writing vps/vpscore/vps_linear.py


In [ ]:
%%writefile vps/vpscore/builders.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple, Optional
import torch
import torch.nn as nn

# ---- Small config facade so builders read only what they need ----
@dataclass
class _CfgView:
    rank: int = 2
    topk: int = 32
    builder: str = "hybrid"
    alpha: float = 1e-3  # ridge in SC

def _safe_topk(scores: torch.Tensor, k: int) -> torch.Tensor:
    k = int(min(max(1, k), scores.numel()))
    _, idx = torch.topk(scores, k, largest=True, sorted=False)
    return idx

def _scores_in_out(x2d: torch.Tensor, W: nn.Linear) -> Tuple[torch.Tensor, torch.Tensor]:
    # x2d: (N, in_features)
    in_scores = x2d.abs().mean(dim=0)                           # (in,)
    with torch.no_grad():
        y2d = x2d @ W.weight.t()                                # (N, out)
    out_scores = y2d.abs().mean(dim=0)                          # (out,)
    return in_scores, out_scores

class SKBuilder:
    """
    Sparse selector builder.
    Returns U (in_features x r) and V (out_features x r), one-hot columns at top-k indices.

      A = W.t() @ V   -> (in, r)
      B = W     @ U   -> (out, r)
    """
    def __init__(self, cfg):
        self.cfg = _CfgView(
            rank=getattr(cfg, "rank", 2),
            topk=getattr(cfg, "topk", 32),
            builder=getattr(cfg, "builder", "sk"),
            alpha=getattr(cfg, "alpha", 1e-3),
        )

    @torch.no_grad()
    def __call__(
        self,
        x2d: torch.Tensor,                 # (N, in_features)
        h2d: torch.Tensor,                 # (N, hidden)  (unused here)
        W: nn.Linear,                      # base linear (out_features x in_features)
        grad_h: Optional[torch.Tensor],    # (N, hidden)  (unused here)
        target_step: Optional[int],        # unused hook
    ):
        device = W.weight.device
        dtype  = W.weight.dtype
        out_f, in_f = W.weight.shape      # (out, in)  <- PyTorch Linear layout

        in_scores, out_scores = _scores_in_out(x2d, W)
        in_sel  = _safe_topk(in_scores, min(self.cfg.topk, in_f))
        out_sel = _safe_topk(out_scores, min(self.cfg.topk, out_f))

        r = int(min(self.cfg.rank, in_sel.numel(), out_sel.numel()))
        r = max(1, r)

        U = torch.zeros(in_f,  r, device=device, dtype=dtype)   # (in, r)
        V = torch.zeros(out_f, r, device=device, dtype=dtype)   # (out, r)
        cols = torch.arange(r, device=device)
        U[in_sel[:r],  cols] = 1.0
        V[out_sel[:r], cols] = 1.0

        return U, V, in_f, out_f

class SCBuilder(SKBuilder):
    """
    Sylvester-style coupling refinement on top of SK (robust to fp16).
    Solves a small ridge least-squares in float32 and folds it into V (column mixing).
    Falls back gracefully to pure SK if numerics are unhappy.
    """
    @torch.no_grad()
    def __call__(
        self,
        x2d: torch.Tensor,
        h2d: torch.Tensor,
        W: nn.Linear,
        grad_h: Optional[torch.Tensor],
        target_step: Optional[int],
    ):
        # start from valid SK selectors
        U, V, in_f, out_f = super().__call__(x2d, h2d, W, grad_h, target_step)

        try:
            r = U.shape[1]
            if r == 0:
                return U, V, in_f, out_f

            # recover selected indices from one-hot columns
            in_mask  = U.abs().sum(dim=1) > 0                      # (in,)
            out_mask = V.abs().sum(dim=1) > 0                      # (out,)
            in_idx   = in_mask.nonzero(as_tuple=False).flatten()[:r]
            out_idx  = out_mask.nonzero(as_tuple=False).flatten()[:r]

            # compact views (float32 for stable linear algebra)
            XA = x2d[:, in_idx].to(torch.float32)                  # (N, r)
            Y  = (x2d @ W.weight.t())[:, out_idx].to(torch.float32)  # (N, r)

            # ridge: (X^T X + alpha I) T = X^T Y
            Gx  = XA.T @ XA                                        # (r, r)
            RHS = XA.T @ Y                                         # (r, r)
            I   = torch.eye(r, device=Gx.device, dtype=Gx.dtype)
            alpha = float(getattr(self.cfg, "alpha", 1e-3))
            T   = torch.linalg.solve(Gx + alpha * I, RHS)          # (r, r)

            # mix columns of V by T^T (stays (out, r)), then column-normalize
            V_mix = (V @ T.to(V.dtype).t())                        # (out, r)
            col_norm = V_mix.norm(dim=0, keepdim=True).clamp_min(1e-6)
            V = (V_mix / col_norm).to(V.dtype)
        except Exception:
            # keep SK result if anything goes wrong
            pass

        return U, V, in_f, out_f

class HybridBuilder:
    """Use SC when gradient information is available; otherwise SK."""
    def __init__(self, cfg):
        self.sc = SCBuilder(cfg)
        self.sk = SKBuilder(cfg)

    @torch.no_grad()
    def __call__(
        self,
        x2d: torch.Tensor,
        h2d: torch.Tensor,
        W: nn.Linear,
        grad_h: Optional[torch.Tensor],
        target_step: Optional[int],
    ):
        if grad_h is not None:
            return self.sc(x2d, h2d, W, grad_h, target_step)
        return self.sk(x2d, h2d, W, grad_h, target_step)

def make_builder(name: str, cfg):
    """
    Factory: returns a callable builder instance with signature
        (x2d, h2d, W, grad_h, target_step) -> (U, V, in_features, out_features)
    """
    key = (name or "").lower()
    if key in {"sk", "sks", "sparse", "selector"}:
        return SKBuilder(cfg)
    if key in {"sc", "sylvester", "coupled"}:
        return SCBuilder(cfg)
    return HybridBuilder(cfg)  # default

# Back-compat alias for older imports
HybridBuilderOut = HybridBuilder



Writing vps/vpscore/builders.py


In [ ]:
%%writefile vps/vpscore/ephemeral_lbfgs.py
from __future__ import annotations
import torch
from collections import deque

class EphemeralLBFGS:
    """
    Tiny L-BFGS memory; we keep curvature info but ensure 1D vectors.
    """
    def __init__(self, m: int = 5):
        self.m = m
        self.S = deque(maxlen=m)
        self.Y = deque(maxlen=m)

    @torch.no_grad()
    def update(self, s: torch.Tensor, y: torch.Tensor):
        # flatten to 1D
        s = s.reshape(-1).detach()
        y = y.reshape(-1).detach()
        if s.numel() == 0 or y.numel() == 0:
            return
        # basic curvature check
        if torch.dot(y, s) > 1e-8:
            self.S.append(s)
            self.Y.append(y)



Writing vps/vpscore/ephemeral_lbfgs.py


In [ ]:
%%writefile vps/vpscore/math_utils.py
from __future__ import annotations
import torch

# ---------------------------
# Entropy utilities
# ---------------------------

@torch.no_grad()
def compute_token_entropy(
    logits: torch.Tensor,
    dim: int = -1,
    normalize: bool = False
) -> torch.Tensor:
    """
    Per-token entropy from logits.

    Args:
        logits: Tensor of shape (..., V) where V = vocab size.
        dim:    Dimension of the vocabulary (usually -1).
        normalize: If True, divides by log(V) to get [0,1] range.

    Returns:
        Tensor of shape logits.shape without the vocab dimension (i.e., reduced along `dim`).
    """
    # Use float32 for numerical stability on T4 (half/bfloat16 can trip SVD/softmax edges)
    if logits.dtype in (torch.float16, torch.bfloat16):
        l = logits.float()
    else:
        l = logits

    log_probs = torch.log_softmax(l, dim=dim)
    probs = log_probs.exp()
    ent = -(probs * log_probs).sum(dim=dim)

    if normalize:
        V = logits.size(dim)
        denom = torch.log(torch.tensor(float(V), device=logits.device, dtype=ent.dtype)).clamp_min(1e-8)
        ent = ent / denom
    return ent

# ---------------------------
# Spectral / clipping helpers
# ---------------------------

@torch.no_grad()
def spectral_clip_cols(M: torch.Tensor, clamp: float) -> torch.Tensor:
    # column-wise ℓ2 clip
    if clamp <= 0:
        return M
    col_norms = torch.linalg.vector_norm(M, dim=0) + 1e-8
    scale = torch.clamp(col_norms / clamp, min=1.0)
    return M / scale

@torch.no_grad()
def spectral_norm_clip(A: torch.Tensor, B: torch.Tensor, tau: float):
    """
    Clamp each rank-1 component A[:,i] B[:,i]^T so that ||A_i||*||B_i|| <= tau.
    This keeps AB^T stable without expensive SVD on half/bfloat.
    """
    if tau <= 0:
        return A, B
    an = torch.linalg.vector_norm(A, dim=0) + 1e-8
    bn = torch.linalg.vector_norm(B, dim=0) + 1e-8
    scale = torch.clamp((an * bn) / tau, min=1.0)
    s = torch.sqrt(scale)
    return A / s, B / s

# ---------------------------
# Feature selection helpers
# ---------------------------

@torch.no_grad()
def topk_indices_by_activation(X: torch.Tensor, k: int):
    """
    X: (tokens, features)
    returns top-k feature indices by mean |activation|
    """
    feats = X.abs().mean(dim=0)
    k = min(k, feats.numel())
    return torch.topk(feats, k).indices

# ---------------------------
# Kronecker / Sylvester (safe on T4)
# ---------------------------

@torch.no_grad()
def batched_kronecker(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """
    Safe Kronecker for small r×r blocks, avoiding view/stride issues on CUDA.
    A,B: (r,r)
    """
    r = A.shape[-1]
    return (A.reshape(r,1,r,1) * B.reshape(1,r,1,r)).reshape(r*r, r*r)

@torch.no_grad()
def solve_sylvester_via_kron(Gx: torch.Tensor, Gb: torch.Tensor, rhs_vec: torch.Tensor, alpha: float=1e-1):
    """
    Solve (Gb ⊗ Gx + α I) vec(S) = rhs_vec  for small r (<= 64 typically).
    Falls back to float32 CPU if needed (half/bfloat SVD not implemented on some GPUs).
    """
    r = Gx.shape[0]
    device = Gx.device
    dtype = torch.float32  # robust on T4

    K = batched_kronecker(Gx.to(dtype), Gb.t().to(dtype))
    M = K + alpha * torch.eye(r*r, dtype=dtype)
    sol = torch.linalg.solve(M.cpu(), rhs_vec.reshape(-1).to(dtype).cpu()).to(device)
    return sol.reshape(r, r).to(Gx.dtype)






Writing vps/vpscore/math_utils.py


In [ ]:
%%writefile vps/vpscore/policy.py
import torch
import math
from dataclasses import dataclass
from typing import Optional

@dataclass
class LayerPolicy:
    use: bool
    rank: int
    topk: int
    gamma: float
    order: int
    use_fisher: bool = False
    budget_tight: bool = False

class VPSPolicy:
    def __init__(self, cfg):
        self.cfg = cfg
        self.last_improved = False
        self.improvement_history = []
        self.token_entropy = 0.0

    def update_outcome(self, improved: bool, loss_delta: float = 0.0):
        self.last_improved = improved
        self.improvement_history.append((improved, loss_delta))
        if len(self.improvement_history) > 10:
            self.improvement_history.pop(0)

    def set_token_entropy(self, entropy: float):
        self.token_entropy = entropy

    def decide(self, layer_module, h2d, token_entropy: Optional[float] = None) -> LayerPolicy:
        cfg = self.cfg

        # ---- robust energy (no NaN/Inf) ----
        energy_t = (h2d * h2d).mean()
        energy_t = torch.nan_to_num(energy_t, nan=0.0, posinf=1e6, neginf=0.0)
        energy = float(energy_t.item())

        # Base decision
        use = (not cfg.enable_policy) or (energy >= cfg.min_energy_threshold)

        # ---- scale in [0,1], numerically safe ----
        e_clamped = max(0.0, min(1e4, energy))
        try:
            scale = 1.0 - math.exp(-e_clamped)
        except OverflowError:
            scale = 1.0
        if not math.isfinite(scale):
            scale = 0.5  # safe fallback, only if upstream produced NaNs

        # Token entropy influence
        if token_entropy is not None:
            self.token_entropy = token_entropy
        if math.isfinite(self.token_entropy) and self.token_entropy > 0:
            scale = max(scale, min(1.0, self.token_entropy / 3.0))

        # Improvement history influence
        recent_improvements = sum(1 for imp, _ in self.improvement_history[-5:] if imp)
        if recent_improvements >= 3:
            scale = min(1.0, scale * 1.3)
        elif recent_improvements == 0 and len(self.improvement_history) >= 5:
            scale *= 0.7

        # ---- Adaptive parameters ----
        r_lo, r_hi = cfg.rank_bounds
        t_lo, t_hi = cfg.topk_bounds
        g_lo, g_hi = cfg.gamma_bounds
        o_lo, o_hi = cfg.order_bounds

        rank = int(r_lo + (r_hi - r_lo) * scale) if cfg.adaptive_rank else cfg.rank
        topk = int(t_lo + (t_hi - t_lo) * scale)
        gamma = g_lo + (g_hi - g_lo) * scale if cfg.adaptive_gamma else cfg.gamma
        order = int(o_lo + (o_hi - o_lo) * scale) if cfg.adaptive_order else cfg.order

        # Clamp to bounds
        rank = max(1, min(rank, r_hi))
        topk = max(t_lo, min(topk, t_hi))
        gamma = max(g_lo, min(gamma, g_hi))
        order = max(o_lo, min(order, o_hi))

        # Builder selection heuristics
        use_fisher = self.last_improved and scale > 0.7
        budget_tight = scale < 0.3 or energy < cfg.min_energy_threshold * 2

        return LayerPolicy(
            use=use, rank=rank, topk=topk, gamma=gamma, order=order,
            use_fisher=use_fisher, budget_tight=budget_tight
        )



Writing vps/vpscore/policy.py


In [ ]:
import os, sys
sys.path.append("/content/vps")
os.environ["PYTHONPATH"] = "/content/vps:" + os.environ.get("PYTHONPATH","")


In [ ]:
%%bash
python - <<'PY'
import pathlib
p = pathlib.Path('vps/vpscore/config.py')
s = p.read_text()
s = s.replace('Qwen/Qwen2.5-7B-Instruct', 'Qwen/Qwen2.5-3B-Instruct')
p.write_text(s)
print("Model -> Qwen/Qwen2.5-3B-Instruct ✅")
PY



Model -> Qwen/Qwen2.5-3B-Instruct ✅


In [ ]:
%%bash
python - <<'PY'
import pathlib, re
p = pathlib.Path('vps/vpscore/config.py')
s = p.read_text()
s = re.sub(r'dtype:\s*Literal\["bf16","fp16","fp32"\]\s*=\s*"bf16"',
           'dtype: Literal["bf16","fp16","fp32"] = "fp16"', s)
p.write_text(s)
print("VPSConfig.dtype -> fp16 ✅")
PY



VPSConfig.dtype -> fp16 ✅


In [ ]:
%%writefile vps/vpscore/rotations.py

import torch
import torch.nn as nn

@torch.no_grad()
def householder_chain(Phi, q=4):
    """Build orthogonal rotation via Householder reflectors"""
    d = Phi.shape[0]
    R = torch.eye(d, device=Phi.device, dtype=Phi.dtype)
    M = Phi.clone()

    for _ in range(q):
        vals, vecs = torch.linalg.eigh(M)
        v = vecs[:, -1]  # dominant eigenvector
        v = v / (v.norm() + 1e-8)
        H = torch.eye(d, device=Phi.device) - 2.0 * torch.outer(v, v)
        R = H @ R
        M = H @ M @ H.t()

    return R

@torch.no_grad()
def accumulate_coactivation_stats(model, dataloader, tokenizer, limit_batches=100):
    """Collect co-activation statistics for rotation optimization"""
    stats = []
    layers = []

    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            layers.append(m)
            stats.append(torch.zeros((m.base.weight.shape[0], m.base.weight.shape[0]),
                                     device=m.base.weight.device, dtype=torch.float32))

    for i, batch in enumerate(dataloader):
        if i >= limit_batches: break
        texts = batch.get("text", batch.get("question", batch.get("input", [])))
        if not texts: continue

        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            _ = model(**inputs)
            for j, m in enumerate(layers):
                if getattr(m, "last_h", None) is not None:
                    h = m.last_h.float()
                    stats[j] += h.T @ h

    # Compute rotations via Householder chains
    rotations = []
    for S in stats:
        S = S / (S.trace() + 1e-6)
        R = householder_chain(S + 1e-6*torch.eye(S.shape[0], device=S.device))
        rotations.append(R)

    return rotations

@torch.no_grad()
def apply_mlp_rotations(model, rotations):
    """Apply orthogonal rotations to MLP layers"""
    i = 0
    for name, mod in model.named_modules():
        # Find MLP blocks (various naming conventions)
        if hasattr(mod, "up_proj") and hasattr(mod, "down_proj"):
            if i < len(rotations):
                R = rotations[i]
                i += 1

                up = mod.up_proj
                down = mod.down_proj

                # Handle both wrapped and unwrapped layers
                if hasattr(up, "base"):
                    up_linear = up.base
                    down_linear = down.base
                else:
                    up_linear = up
                    down_linear = down

                if isinstance(up_linear, nn.Linear) and isinstance(down_linear, nn.Linear):
                    # Check dimension compatibility
                    if up_linear.weight.shape[1] == R.shape[0]:
                        up_linear.weight.data = up_linear.weight @ R
                    if down_linear.weight.shape[0] == R.shape[1]:
                        down_linear.weight.data = R.T @ down_linear.weight

    return model



Writing vps/vpscore/rotations.py


In [ ]:
%%writefile vps/scripts/run_ablations.py
# scripts/run_ablations.py
import argparse
import json
import torch
from dataclasses import dataclass
from typing import Dict, Any
from transformers import AutoModelForCausalLM, AutoTokenizer
from vpscore.config import VPSConfig
from vpscore.patch_hf import patch_model_with_vps
from vpscore.hooks import HookManager
from vpscore.builders import make_builder
from vpscore.patch_hf import PAIR_REGISTRY
from scripts.eval_gsm8k import evaluate_gsm8k

@dataclass
class AblationConfig:
    name: str
    vps_enabled: bool = True
    gamma: float = 0.5
    rank: int = 2
    topk: int = 32
    order: int = 1
    builder: str = "hybrid"
    qk_coupling: bool = True
    lbfgs_enabled: bool = True
    tau: float = 0.8
    adaptive_rank: bool = True
    adaptive_gamma: bool = True

def run_ablation(model, tok, hooks, test_path, ablation_cfg: AblationConfig,
                 base_cfg: VPSConfig, limit=None):
    """Run a single ablation configuration"""
    # Update config with ablation settings
    cfg = VPSConfig()
    cfg.gamma = ablation_cfg.gamma
    cfg.rank = ablation_cfg.rank
    cfg.topk = ablation_cfg.topk
    cfg.order = ablation_cfg.order
    cfg.builder = ablation_cfg.builder
    cfg.qk_coupling = ablation_cfg.qk_coupling
    cfg.lbfgs_enabled = ablation_cfg.lbfgs_enabled
    cfg.tau = ablation_cfg.tau
    cfg.adaptive_rank = ablation_cfg.adaptive_rank
    cfg.adaptive_gamma = ablation_cfg.adaptive_gamma

    # Apply configuration to model
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if m.builder_name != cfg.builder:
                m.builder = make_builder(cfg.builder, cfg.rank, cfg.topk, m.clamp)
                m.builder_name = cfg.builder
            m.default_gamma = cfg.gamma
            m.default_rank = cfg.rank
            m.default_topk = cfg.topk
            m.order = cfg.order
            m.tau = cfg.tau
            if m.policy:
                m.policy.cfg = cfg

    # Re-bind Q/K if policy toggled
    for pair in PAIR_REGISTRY.values():
        q = pair.get("attn_q"); k = pair.get("attn_k")
        if not (q and k): continue
        if cfg.qk_coupling:
            q.is_Q = True; k.is_K = True; q._peer = k; k._peer = q
        else:
            q.is_Q = k.is_K = False; q._peer = k._peer = None

    # Run evaluation
    result = evaluate_gsm8k(model, tok, hooks, test_path, cfg,
                           use_vps=ablation_cfg.vps_enabled, limit=limit)

    return result

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--test_path", type=str, required=True)
    ap.add_argument("--limit", type=int, default=100, help="Limit examples for quick testing")
    ap.add_argument("--output", type=str, default="ablation_results.json")
    args = ap.parse_args()

    # Define ablation configurations
    ablations = [
        AblationConfig("baseline", vps_enabled=False),
        AblationConfig("vps_default"),
        AblationConfig("no_qk_coupling", qk_coupling=False),
        AblationConfig("no_lbfgs", lbfgs_enabled=False),
        AblationConfig("gamma_0.3", gamma=0.3),
        AblationConfig("gamma_0.7", gamma=0.7),
        AblationConfig("rank_1", rank=1),
        AblationConfig("rank_4", rank=4),
        AblationConfig("order_1", order=1),
        AblationConfig("order_2", order=2),
        AblationConfig("order_3", order=3),
        AblationConfig("builder_gn", builder="gn"),
        AblationConfig("builder_fisher", builder="fisher"),
        AblationConfig("builder_secant", builder="secant"),
        AblationConfig("builder_skew", builder="skew"),
        AblationConfig("builder_sparse", builder="sparse"),
        AblationConfig("tau_0.5", tau=0.5),
        AblationConfig("tau_1.0", tau=1.0),
        AblationConfig("fixed_rank", adaptive_rank=False),
        AblationConfig("fixed_gamma", adaptive_gamma=False),
        AblationConfig("topk_16", topk=16),
        AblationConfig("topk_64", topk=64),
    ]

    # Load model once
    cfg = VPSConfig()
    torch.manual_seed(cfg.seed)
    dtype = dict(bf16=torch.bfloat16, fp16=torch.float16, fp32=torch.float32)[cfg.dtype]

    print(f"Loading model: {cfg.model_name}")
    tok = AutoTokenizer.from_pretrained(cfg.model_name, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token_id = tok.eos_token_id

    model = AutoModelForCausalLM.from_pretrained(cfg.model_name, torch_dtype=dtype,
                                                 device_map=cfg.device_map)
    model = patch_model_with_vps(model, cfg.apply_to, cfg.rank, cfg.topk,
                                 cfg.clamp, cfg.gamma, cfg.builder,
                                 cfg.softgrad_builder, policy_cfg=cfg)
    hooks = HookManager()
    hooks.attach(model)

    # Run ablations
    results = {}
    for ablation in ablations:
        print(f"\n=== Running ablation: {ablation.name} ===")
        result = run_ablation(model, tok, hooks, args.test_path, ablation, cfg, args.limit)
        results[ablation.name] = {
            "accuracy": result["accuracy"],
            "correct": result["correct"],
            "total": result["total"],
            "config": {
                "gamma": ablation.gamma,
                "rank": ablation.rank,
                "topk": ablation.topk,
                "order": ablation.order,
                "builder": ablation.builder,
                "qk_coupling": ablation.qk_coupling,
                "lbfgs": ablation.lbfgs_enabled,
                "tau": ablation.tau
            }
        }
        print(f"{ablation.name}: {result['accuracy']:.2f}%")

    # Save results
    with open(args.output, 'w') as f:
        json.dump(results, f, indent=2)

    # Print summary
    print("\n=== ABLATION SUMMARY ===")
    baseline = results.get("baseline", {}).get("accuracy", 0)
    for name, res in sorted(results.items(), key=lambda x: -x[1]["accuracy"]):
        diff = res["accuracy"] - baseline
        print(f"{name:20s}: {res['accuracy']:6.2f}% ({diff:+.2f}%)")

    print(f"\nResults saved to {args.output}")

if __name__ == "__main__":
    main()




Writing vps/scripts/run_ablations.py


In [ ]:
%%writefile vps/vpscore/verifiers/composite_verifier.py
import re
import sympy as sp
import torch
from typing import Dict, Any, Optional
import pint

# Initialize unit registry
ureg = pint.UnitRegistry()

class CompositeVerifier:
    """Multi-objective verifier with numeric, units, algebraic, and self-consistency checks"""

    def __init__(self, weights: Dict[str, float] = None):
        self.weights = weights or {
            "numeric": 1.0,
            "units": 0.5,
            "self_consistency": 0.3,
            "algebraic": 0.2
        }
        self.num_pattern = re.compile(r"(-?\d+(?:\.\d+)?)")

    def extract_answer(self, text: str) -> Optional[sp.Basic]:
        """Extract numeric answer from text"""
        matches = list(self.num_pattern.finditer(text))
        if not matches:
            return None
        try:
            return sp.nsimplify(matches[-1].group(1))
        except:
            return None

    def check_units(self, text: str, expected_unit: str = None) -> float:
        """Check dimensional consistency"""
        try:
            # Extract quantities with units
            quantities = re.findall(r"(\d+(?:\.\d+)?)\s*([a-zA-Z]+)", text)
            if not quantities:
                return 1.0

            # Parse with pint
            parsed = []
            for val, unit in quantities:
                try:
                    q = ureg.Quantity(float(val), unit)
                    parsed.append(q)
                except:
                    pass

            if len(parsed) < 2:
                return 1.0

            # Check dimensional consistency
            dims = [q.dimensionality for q in parsed]
            if len(set(str(d) for d in dims)) == 1:
                return 0.0  # All same dimension, good
            else:
                return 0.5  # Mixed dimensions, partial penalty
        except:
            return 1.0

    def check_algebraic_form(self, pred_text: str, gold_text: str) -> float:
        """Check if algebraic structure matches"""
        try:
            # Extract algebraic expressions
            pred_expr = self.extract_expression(pred_text)
            gold_expr = self.extract_expression(gold_text)

            if pred_expr is None or gold_expr is None:
                return 1.0

            # Check structural similarity
            if sp.simplify(pred_expr - gold_expr) == 0:
                return 0.0

            # Check if same variables
            pred_vars = pred_expr.free_symbols
            gold_vars = gold_expr.free_symbols
            if pred_vars == gold_vars:
                return 0.3

            return 1.0
        except:
            return 1.0

    def extract_expression(self, text: str) -> Optional[sp.Basic]:
        """Extract algebraic expression from text"""
        # Look for patterns like "x = ..." or "answer = ..."
        expr_match = re.search(r"(?:=|is)\s*([^.]+)", text)
        if expr_match:
            try:
                return sp.sympify(expr_match.group(1))
            except:
                pass
        return None

    def check_self_consistency(self, model, tokenizer, prompt: str,
                              n_samples: int = 3, temperature: float = 0.7) -> float:
        """Check if model gives consistent answers across samples"""
        answers = []
        for _ in range(n_samples):
            with torch.no_grad():
                inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(**inputs, max_new_tokens=128,
                                        temperature=temperature, do_sample=True)
                text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                answer = self.extract_answer(text)
                if answer is not None:
                    answers.append(answer)

        if len(answers) < 2:
            return 1.0

        # Check variance in answers
        numeric_answers = [float(sp.N(a)) for a in answers]
        mean_val = sum(numeric_answers) / len(numeric_answers)
        variance = sum((x - mean_val)**2 for x in numeric_answers) / len(numeric_answers)

        # Normalize variance to [0, 1]
        return min(1.0, variance / (mean_val**2 + 1e-6))

    def compute_loss(self, pred_text: str, gold_text: str,
                    model=None, tokenizer=None, prompt: str = None) -> Dict[str, float]:
        """Compute multi-objective verification loss"""
        losses = {}

        # Numeric loss
        pred_num = self.extract_answer(pred_text)
        gold_num = self.extract_answer(gold_text)
        if pred_num is not None and gold_num is not None:
            try:
                losses["numeric"] = float((sp.N(pred_num) - sp.N(gold_num))**2)
            except:
                losses["numeric"] = 1.0
        else:
            losses["numeric"] = 1.0

        # Units loss
        losses["units"] = self.check_units(pred_text)

        # Algebraic form loss
        losses["algebraic"] = self.check_algebraic_form(pred_text, gold_text)

        # Self-consistency loss
        if model is not None and tokenizer is not None and prompt is not None:
            losses["self_consistency"] = self.check_self_consistency(
                model, tokenizer, prompt
            )
        else:
            losses["self_consistency"] = 0.0

        # Weighted total
        total = sum(self.weights.get(k, 0) * v for k, v in losses.items())
        losses["total"] = total

        return losses



Writing vps/vpscore/verifiers/composite_verifier.py


In [ ]:
%%writefile vps/vpscore/hooks.py
import weakref
import torch

class HookManager:
    def __init__(self):
        self.handles = []
        self.layers = weakref.WeakSet()
        self.head_outputs = {}  # for multi-head exterior

    def attach(self, model):
        self.detach()
        for name, m in model.named_modules():
            if m.__class__.__name__ == "VPSLinear":
                self.layers.add(m)
                self.handles.append(m.register_forward_pre_hook(self._fwd_pre))
                self.handles.append(m.register_forward_hook(self._fwd_post))
                self.handles.append(m.register_full_backward_hook(self._bwd))
            # Capture per-head outputs if attention block
            if "attention" in name.lower() and hasattr(m, "num_heads"):
                self.handles.append(m.register_forward_hook(self._capture_heads))

    def detach(self):
        for h in self.handles:
            try: h.remove()
            except: pass
        self.handles.clear()
        self.layers = weakref.WeakSet()
        self.head_outputs.clear()

    @staticmethod
    def _fwd_pre(module, inputs):
        x = inputs[0]
        module.last_x = x.detach()

    @staticmethod
    def _fwd_post(module, inputs, output):
        pass

    @staticmethod
    def _bwd(module, grad_input, grad_output):
        g = grad_output[0]
        module.last_grad_h = g.detach()

    def _capture_heads(self, module, inputs, output):
        """Capture per-head outputs for exterior products"""
        if hasattr(output, "shape") and len(output.shape) >= 3:
            self.head_outputs[id(module)] = output.detach()

    def clear_buffers(self):
        for m in list(self.layers):
            for name in ("last_x","last_h","last_grad_h"):
                if hasattr(m, name): setattr(m, name, None)
        self.head_outputs.clear()

    def get_head_outputs(self, module_id):
        return self.head_outputs.get(module_id, None)






Writing vps/vpscore/hooks.py


In [ ]:
%%writefile vps/vpscore/utils/generation.py
from typing import Optional, List, Dict, Any
import torch

def _as_model_inputs(x):
    """Normalize tokenizer outputs into a dict with input_ids and attention_mask."""
    if isinstance(x, torch.Tensor):
        return {"input_ids": x, "attention_mask": torch.ones_like(x)}
    elif isinstance(x, dict):
        if "attention_mask" not in x and "input_ids" in x:
            x = dict(x)
            x["attention_mask"] = torch.ones_like(x["input_ids"])
        return x
    else:
        t = torch.as_tensor(x, dtype=torch.long)
        return {"input_ids": t, "attention_mask": torch.ones_like(t)}

def _chat_inputs(tokenizer, prompt: str, add_generation_prompt: bool = True):
    """
    Build input_ids using the tokenizer's chat_template when available.
    Returns a dict of tensors on CPU.
    """
    if getattr(tokenizer, "apply_chat_template", None) and getattr(tokenizer, "chat_template", None):
        messages: List[Dict[str, Any]] = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
        out = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=add_generation_prompt,
            return_tensors="pt",
        )
        return _as_model_inputs(out)
    # Fallback: plain prompt
    return _as_model_inputs(tokenizer(prompt, return_tensors="pt"))

def generate(model, tok, prompt: str, max_new_tokens: int = 128,
             temperature: float = 0.7, top_p: float = 0.95,
             do_sample: Optional[bool] = None) -> str:
    if do_sample is None:
        do_sample = temperature > 0.0

    inputs = _chat_inputs(tok, prompt, add_generation_prompt=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id or tok.eos_token_id,
        return_dict_in_generate=True,
    )
    seq = out.sequences  # [bsz, prompt_len + new_len]

    # Decode ONLY the newly generated tokens (no role echoes)
    prompt_len = inputs["input_ids"].shape[1]
    new_ids = seq[:, prompt_len:]
    return tok.decode(new_ids[0], skip_special_tokens=True).strip()



Writing vps/vpscore/utils/generation.py


In [ ]:
%%writefile vps/scripts/infer_vps.py
import os
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()

import argparse
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from vpscore.config import VPSConfig
from vpscore.patch_hf import patch_model_with_vps
from vpscore.verifiers.composite_verifier import CompositeVerifier
from vpscore.utils.generation import generate
from vpscore.hooks import HookManager
from vpscore.math_utils import compute_token_entropy

# Use canonical keys that CompositeVerifier expects
DEFAULT_VERIFIER_WEIGHTS = {
    "numeric": 1.0,
    "units": 0.5,
    "algebraic": 0.2,
    "self_consistency": 0.3,
}

def build(cfg: VPSConfig):
    # ---- robust fallbacks so missing config fields don't crash ----
    seed = getattr(cfg, "seed", 1234)
    torch.manual_seed(seed)

    # Resolve dtype name -> torch dtype
    dtype_name = getattr(cfg, "dtype", None) or getattr(cfg, "torch_dtype_str", "float16")
    dtype = getattr(torch, dtype_name, torch.float16)

    model_name = getattr(cfg, "model_name", None)
    if not model_name:
        raise ValueError("cfg.model_name is not set in VPSConfig")

    device_map = getattr(cfg, "device_map", "auto")

    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token_id = tok.eos_token_id

    # Use the official argument name
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=dtype, device_map=device_map)

    apply_to = getattr(cfg, "apply_to", None)
    model = patch_model_with_vps(model, apply_to, cfg)

    # Small introspection to confirm wrapping happened
    wrapped = sum(1 for _, m in model.named_modules() if m.__class__.__name__ == "VPSLinear")
    print(f"[VPS] Wrapped layers: {wrapped}")

    hooks = HookManager()
    hooks.attach(model)
    return tok, model, hooks

def _parse_args():
    ap = argparse.ArgumentParser()
    # Notebook-safe defaults prevent argparse crashes
    ap.add_argument("--prompt", type=str, default="Compute 120/1.5 and give the answer with unit km/h.")
    ap.add_argument("--gold", type=str, default=None)
    ap.add_argument("--iters", type=int, default=2)
    # In notebooks, IPython may inject argv; parse_known keeps it safe
    args, _ = ap.parse_known_args()
    return args

def main():
    args = _parse_args()
    cfg = VPSConfig()

    tok, model, hooks = build(cfg)

    # Fall back if cfg.verifier_weights missing
    verifier_weights = getattr(cfg, "verifier_weights", DEFAULT_VERIFIER_WEIGHTS)
    verifier = CompositeVerifier(verifier_weights)

    # Also fallbacks for gen params
    max_new_tokens = getattr(cfg, "max_new_tokens", 128)
    temperature = getattr(cfg, "temperature", 0.7)
    top_p = getattr(cfg, "top_p", 0.9)

    # ----------------- Iteration 0 (plain decode) -----------------
    text = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
    print(f"=== Iteration 0 ===\n{text}\n")

    if args.gold is None or args.iters < 2:
        return

    prev_loss = float("inf")

    for it in range(1, args.iters):
        hooks.clear_buffers()

        # Clear any L-BFGS memories (keeps your technique intact)
        for m in model.modules():
            if hasattr(m, "clear_lbfgs"):
                m.clear_lbfgs()

        # Forward pass to compute token entropy for the adaptive policy
        inputs = tok(args.prompt, return_tensors="pt").to(model.device)
        with torch.enable_grad():
            out = model(**inputs)
            logits = out.logits
            entropy = compute_token_entropy(logits[0, -1, :])

        # Feed entropy to policies
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.set_token_entropy(float(entropy))

        # Deterministic decode for verifier evaluation
        out_ids = model.generate(
            **inputs,
            max_new_tokens=min(64, max_new_tokens),
            do_sample=False,
            return_dict_in_generate=True,
        )
        pred_text = tok.decode(out_ids.sequences[0], skip_special_tokens=True)

        # Multi-objective verifier
        losses = verifier.compute_loss(pred_text, args.gold, model, tok, args.prompt)
        vloss = losses["total"]

        # Feedback loop to policy
        improved = vloss < prev_loss
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.update_outcome(improved, prev_loss - vloss)
        prev_loss = vloss

        # Short CE surrogate to populate grads → VPS uses grad_h (via hooks)
        gold_ids = tok(args.gold, return_tensors="pt").to(model.device)["input_ids"][0]
        T = min(8, gold_ids.shape[0])
        target = gold_ids[-T:].unsqueeze(0)  # [1, T]

        model.zero_grad(set_to_none=True)
        with torch.enable_grad():
            out2 = model(**inputs)
            logits2 = out2.logits[:, -T:, :]  # [1, T, V]
            ce = torch.nn.functional.cross_entropy(
                logits2.reshape(-1, logits2.size(-1)),
                target.reshape(-1)
            )
            ce.backward()

        # VPS-enhanced generation
        text2 = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
        print(f"=== Iteration {it} (VPS enhanced) ===")
        print(f"Losses: {losses}")
        print(f"Output:\n{text2}\n")

if __name__ == "__main__":
    main()


Overwriting vps/scripts/infer_vps.py


In [ ]:
%%writefile vps/vpscore/patch_hf.py
from __future__ import annotations
import torch.nn as nn
from typing import List, Any
from .vps_linear import VPSLinear

NAMES = {
    "attn_q": ["q_proj","wq","query","q"],
    "attn_k": ["k_proj","wk","key","k"],
    "attn_v": ["v_proj","wv","value","v"],
    "attn_o": ["o_proj","wo","out_proj","o"],
    "mlp_up": ["up_proj","w1","fc_in","dense_h_to_4h","gate_proj","w3","gate"],
    "mlp_down":["down_proj","w2","fc_out","dense_4h_to_h"],
}

# alias -> bucket map
ALIAS2BUCKET = {alias: bucket for bucket, aliases in NAMES.items() for alias in aliases}

# Global registry for Q/K pairing across modules (keyed by parent module id)
PAIR_REGISTRY = {}

def _match_bucket(name: str, apply_to: List[str]):
    if not apply_to:
        return None
    lname = name.lower()

    # If user provided canonical buckets, honor those
    for bucket, aliases in NAMES.items():
        if bucket in apply_to and any(a in lname for a in aliases):
            return bucket

    # If user provided aliases (e.g., "q_proj"), map them to buckets
    for wanted in apply_to:
        w = wanted.lower()
        bucket = ALIAS2BUCKET.get(w, None)
        if bucket is not None:
            # Check actual child name contains this alias
            if any(w in lname for w in NAMES[bucket]):
                return bucket
    return None

def _wrap(module: nn.Module, apply_to: List[str], vps_kwargs: dict):
    for name, child in list(module.named_children()):
        _wrap(child, apply_to, vps_kwargs)
        if isinstance(child, nn.Linear):
            bucket = _match_bucket(name, apply_to)
            if bucket is not None:
                vps = VPSLinear(child, **vps_kwargs)
                setattr(module, name, vps)
                # Register possible q/k pairs under the parent block
                if bucket in ["attn_q", "attn_k"]:
                    key = id(module)
                    PAIR_REGISTRY.setdefault(key, {})[bucket] = vps

def patch_model_with_vps(model: nn.Module, apply_to: List[str], *args: Any, **kwargs: Any):
    """
    Accepts either:
      (model, apply_to, cfg)   where cfg is VPSConfig
    or legacy long-form parameter lists (kept for compatibility).
    """
    softgrad_builder = kwargs.pop("softgrad_builder", False)
    policy_cfg = kwargs.pop("policy_cfg", None)
    qk_coupling_flag = False

    if len(args) == 1 and not isinstance(args[0], (int, float, str, bool)):
        cfg = args[0]
        rank   = getattr(cfg, "rank", 2)
        topk   = getattr(cfg, "topk", 32)
        clamp  = getattr(cfg, "clamp", 0.2)
        gamma  = getattr(cfg, "gamma", 0.5)
        builder= getattr(cfg, "builder", "hybrid")
        softgrad_builder = getattr(cfg, "softgrad_builder", softgrad_builder)
        policy_cfg = cfg
        qk_coupling_flag = bool(getattr(cfg, "qk_coupling", False))
    else:
        if len(args) >= 6:
            rank, topk, clamp, gamma, builder = args[0:6]
            if len(args) >= 12:
                qk_coupling_flag = bool(args[7])
            if len(args) >= 13 and isinstance(args[12], bool):
                softgrad_builder = args[12]
            if len(args) >= 14:
                policy_cfg = args[13]
        else:
            raise TypeError("patch_model_with_vps: unsupported argument pattern. "
                            "Pass VPSConfig as third arg OR full long-form params.")

    vps_kwargs = dict(
        rank=rank,
        topk=topk,
        clamp=clamp,
        gamma=gamma,
        builder=builder,
        softgrad_builder=softgrad_builder,
        policy_cfg=policy_cfg,
    )

    _wrap(model, apply_to or [], vps_kwargs)

    if qk_coupling_flag:
        for pair in PAIR_REGISTRY.values():
            q = pair.get("attn_q")
            k = pair.get("attn_k")
            if q is not None and k is not None:
                q.is_Q = True; k.is_K = True
                q._peer = k;   k._peer = q

    return model


Overwriting vps/vpscore/patch_hf.py


In [ ]:
%%writefile vps/vpscore/hooks.py
import weakref
import torch

class HookManager:
    def __init__(self):
        self.handles = []
        self.layers = weakref.WeakSet()
        self.head_outputs = {}  # for multi-head exterior

    def attach(self, model):
        self.detach()
        for name, m in model.named_modules():
            if m.__class__.__name__ == "VPSLinear":
                self.layers.add(m)
                self.handles.append(m.register_forward_pre_hook(self._fwd_pre))
                self.handles.append(m.register_forward_hook(self._fwd_post))
                self.handles.append(m.register_full_backward_hook(self._bwd))
            if "attention" in name.lower() and hasattr(m, "num_heads"):
                self.handles.append(m.register_forward_hook(self._capture_heads))

    def detach(self):
        for h in self.handles:
            try: h.remove()
            except: pass
        self.handles.clear()
        self.layers = weakref.WeakSet()
        self.head_outputs.clear()

    @staticmethod
    def _fwd_pre(module, inputs):
        x = inputs[0]
        module.last_x = x.detach()

    @staticmethod
    def _fwd_post(module, inputs, output):
        # critical: keep activations for stats/policy/rotations
        try:
            module.last_h = output.detach()
        except Exception:
            module.last_h = None

    @staticmethod
    def _bwd(module, grad_input, grad_output):
        g = grad_output[0]
        module.last_grad_h = g.detach() if g is not None else None

    def _capture_heads(self, module, inputs, output):
        if hasattr(output, "shape") and len(output.shape) >= 3:
            self.head_outputs[id(module)] = output.detach()

    def clear_buffers(self):
        for m in list(self.layers):
            for name in ("last_x","last_h","last_grad_h"):
                if hasattr(m, name): setattr(m, name, None)
        self.head_outputs.clear()

    def get_head_outputs(self, module_id):
        return self.head_outputs.get(module_id, None)


Overwriting vps/vpscore/hooks.py


In [ ]:
%%writefile vps/vpscore/utils/generation.py
from typing import Optional, List, Dict, Any
import torch

def _as_model_inputs(x):
    if isinstance(x, torch.Tensor):
        return {"input_ids": x, "attention_mask": torch.ones_like(x)}
    elif isinstance(x, dict):
        if "attention_mask" not in x and "input_ids" in x:
            x = dict(x)
            x["attention_mask"] = torch.ones_like(x["input_ids"])
        return x
    else:
        t = torch.as_tensor(x, dtype=torch.long)
        return {"input_ids": t, "attention_mask": torch.ones_like(t)}

def _chat_inputs(tokenizer, prompt: str, add_generation_prompt: bool = True):
    if getattr(tokenizer, "apply_chat_template", None) and getattr(tokenizer, "chat_template", None):
        messages: List[Dict[str, Any]] = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
        out = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=add_generation_prompt,
            return_tensors="pt",
        )
        return _as_model_inputs(out)
    return _as_model_inputs(tokenizer(prompt, return_tensors="pt"))

def generate(model, tok, prompt: str, max_new_tokens: int = 128,
             temperature: float = 0.7, top_p: float = 0.95,
             do_sample: Optional[bool] = None) -> str:
    if do_sample is None:
        do_sample = temperature > 0.0

    inputs = _chat_inputs(tok, prompt, add_generation_prompt=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        return_dict_in_generate=True,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id or tok.eos_token_id,
    )
    if do_sample:
        gen_kwargs.update(temperature=temperature, top_p=top_p)

    out = model.generate(**inputs, **gen_kwargs)
    seq = out.sequences

    prompt_len = inputs["input_ids"].shape[1]
    new_ids = seq[:, prompt_len:]
    return tok.decode(new_ids[0], skip_special_tokens=True).strip()


Overwriting vps/vpscore/utils/generation.py


In [ ]:
%%writefile vps/vpscore/verifiers/composite_verifier.py
import re
import sympy as sp
import torch
from typing import Dict, Any, Optional
import pint

ureg = pint.UnitRegistry()

_WARN_SIGNS = ("E0000","W0000","tensorflow","xla","cuDNN","cuBLAS","computation_placer","I tensorflow")

class CompositeVerifier:
    """Multi-objective verifier with numeric, units, algebraic, and self-consistency checks"""

    def __init__(self, weights: Dict[str, float] = None):
        # Back-compat mapping for user-provided keys
        weights = weights or {}
        mapped = {
            "numeric":           weights.get("numeric",           weights.get("exact_match", 1.0)),
            "units":             weights.get("units",             weights.get("coherence", 0.5)),
            "algebraic":         weights.get("algebraic",         0.2),
            "self_consistency":  weights.get("self_consistency",  weights.get("length_penalty", 0.3)),
        }
        self.weights = mapped
        # Prefer numbers after '=' and with units; fall back to any trailing number
        self.eq_pattern = re.compile(r"(?:=|is)\s*(-?\d+(?:\.\d+)?)\s*(?:km/?h|m/s|s|ms|kg|m|cm|mm|%|deg|°)?\b", re.I)
        self.num_pattern = re.compile(r"(-?\d+(?:\.\d+)?)")

    def _candidate_lines(self, text: str):
        # filter out log spam lines
        for line in reversed(text.splitlines()):
            if any(sig in line for sig in _WARN_SIGNS):
                continue
            yield line.strip()

    def extract_answer(self, text: str) -> Optional[sp.Basic]:
        """Extract numeric answer from text, preferring '= <num> [unit]' in the last meaningful lines."""
        for line in self._candidate_lines(text):
            m = self.eq_pattern.search(line)
            if m:
                try:
                    return sp.nsimplify(m.group(1))
                except Exception:
                    pass
        # fallback: last number in clean lines
        for line in self._candidate_lines(text):
            m = self.num_pattern.search(line)
            if m:
                try:
                    return sp.nsimplify(m.group(1))
                except Exception:
                    return None
        return None

    def check_units(self, text: str, expected_unit: str = None) -> float:
        try:
            quantities = re.findall(r"(\d+(?:\.\d+)?)\s*([a-zA-Z]+)", text)
            if not quantities:
                return 1.0
            parsed = []
            for val, unit in quantities:
                try:
                    q = ureg.Quantity(float(val), unit)
                    parsed.append(q)
                except Exception:
                    pass
            if len(parsed) < 2:
                return 1.0
            dims = [q.dimensionality for q in parsed]
            return 0.0 if len(set(str(d) for d in dims)) == 1 else 0.5
        except Exception:
            return 1.0

    def extract_expression(self, text: str) -> Optional[sp.Basic]:
        expr_match = re.search(r"(?:=|is)\s*([^.]+)", text)
        if expr_match:
            try:
                return sp.sympify(expr_match.group(1))
            except Exception:
                pass
        return None

    def check_algebraic_form(self, pred_text: str, gold_text: str) -> float:
        try:
            pred_expr = self.extract_expression(pred_text)
            gold_expr = self.extract_expression(gold_text)
            if pred_expr is None or gold_expr is None:
                return 1.0
            if sp.simplify(pred_expr - gold_expr) == 0:
                return 0.0
            if pred_expr.free_symbols == gold_expr.free_symbols:
                return 0.3
            return 1.0
        except Exception:
            return 1.0

    def check_self_consistency(self, model, tokenizer, prompt: str,
                               n_samples: int = 3, temperature: float = 0.7) -> float:
        answers = []
        for _ in range(n_samples):
            with torch.no_grad():
                inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(**inputs, max_new_tokens=128, temperature=temperature, do_sample=True)
                text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                ans = self.extract_answer(text)
                if ans is not None:
                    answers.append(ans)
        if len(answers) < 2:
            return 1.0
        numeric_answers = [float(sp.N(a)) for a in answers]
        mean_val = sum(numeric_answers) / len(numeric_answers)
        variance = sum((x - mean_val)**2 for x in numeric_answers) / len(numeric_answers)
        return min(1.0, variance / (abs(mean_val) + 1e-6)**2)

    def compute_loss(self, pred_text: str, gold_text: str,
                     model=None, tokenizer=None, prompt: str = None) -> Dict[str, float]:
        losses: Dict[str, float] = {}

        pred_num = self.extract_answer(pred_text)
        gold_num = self.extract_answer(gold_text)
        if pred_num is not None and gold_num is not None:
            try:
                losses["numeric"] = float((sp.N(pred_num) - sp.N(gold_num))**2)
            except Exception:
                losses["numeric"] = 1.0
        else:
            losses["numeric"] = 1.0

        losses["units"] = self.check_units(pred_text)
        losses["algebraic"] = self.check_algebraic_form(pred_text, gold_text)

        if model is not None and tokenizer is not None and prompt is not None:
            losses["self_consistency"] = self.check_self_consistency(model, tokenizer, prompt)
        else:
            losses["self_consistency"] = 0.0

        total = sum(self.weights.get(k, 0.0) * v for k, v in losses.items())
        losses["total"] = total
        return losses


Overwriting vps/vpscore/verifiers/composite_verifier.py


In [ ]:
%%writefile vps/vpscore/vps_linear.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F

from .builders import make_builder, HybridBuilder  # no HybridBuilderOut needed

try:
    from .math_utils import spectral_norm_clip as _spectral_norm_clip
except Exception:
    def _spectral_norm_clip(A: torch.Tensor, B: torch.Tensor, tau: float) -> Tuple[torch.Tensor, torch.Tensor]:
        return A, B

try:
    from .ephemeral_lbfgs import EphemeralLBFGS
except Exception:
    class EphemeralLBFGS:
        def __init__(self, m: int = 5): self.m = m
        def update(self, s: torch.Tensor, y: torch.Tensor): return
        def two_loop(self, g: torch.Tensor) -> torch.Tensor: return g

try:
    from .policy import VPSPolicy as _Policy
except Exception:
    class _Policy:
        def __init__(self, cfg): self.cfg = cfg
        def decide(self, module, h2d: torch.Tensor):
            class P: pass
            p = P(); p.rank = int(getattr(module.cfg, "rank", 2))
            p.gamma = float(getattr(module.cfg, "gamma", 0.5))
            p.order = int(getattr(module.cfg, "order", 1))
            return p

@dataclass
class _VPSLinearCfg:
    rank: int = 2
    topk: int = 32
    clamp: Optional[float] = None
    gamma: float = 0.5
    builder: str = "hybrid"
    order: int = 1
    qk_coupling: bool = True
    tau: float = 0.8
    lbfgs_enabled: bool = True
    adaptive_rank: bool = True
    adaptive_gamma: bool = True
    alpha: float = 1e-3  # for SC builder
    # --- policy expectations (add safe defaults) ---
    enable_policy: bool = True
    min_energy_threshold: float = 1e-6
    rank_bounds: tuple = (1, 8)
    topk_bounds: tuple = (8, 64)
    gamma_bounds: tuple = (0.1, 1.0)
    order_bounds: tuple = (1, 2)
    adaptive_order: bool = True

class VPSLinear(nn.Module):
    """
    Adds a dynamic low-rank delta: y = W x + gamma * (x A) B^T,
    with A = W^T V, B = W U. U,V are synthesized on the fly.
    """
    def __init__(self, base: nn.Linear, **kwargs):
        super().__init__()
        assert isinstance(base, nn.Linear), "VPSLinear expects nn.Linear as 'base'"
        self.base = base
        self.cfg = _VPSLinearCfg(**{
            **_VPSLinearCfg().__dict__,
            **{k: v for k, v in kwargs.items() if k in _VPSLinearCfg().__dict__}
        })
        self.builder = make_builder(self.cfg.builder, self.cfg)
        self.builder_name = self.cfg.builder
        self.policy  = _Policy(self.cfg)
        self.lbfgs   = EphemeralLBFGS(m=5) if self.cfg.lbfgs_enabled else None

    @property
    def clamp(self):
        return self.cfg.clamp

    def clear_lbfgs(self):
        if self.lbfgs and hasattr(self.lbfgs, "S"):
            try:
                self.lbfgs.S.clear(); self.lbfgs.Y.clear()
            except Exception:
                pass

    def extra_repr(self) -> str:
        return (f"rank={self.cfg.rank}, topk={self.cfg.topk}, gamma={self.cfg.gamma}, "
                f"builder='{self.cfg.builder}', order={self.cfg.order}, lbfgs={self.cfg.lbfgs_enabled}")

    def _flatten(self, x: torch.Tensor) -> Tuple[torch.Tensor, Tuple[int, ...]]:
        if x.dim() == 2:
            return x, x.shape
        in_f = x.size(-1)
        return x.reshape(-1, in_f), x.shape

    def _unflatten(self, y2d: torch.Tensor, orig_shape: Tuple[int, ...]) -> torch.Tensor:
        return y2d.reshape(*orig_shape[:-1], y2d.size(-1))

    @torch.no_grad()
    def _decide_policy(self, h2d: torch.Tensor):
        return self.policy.decide(self, h2d)

    def _compute_delta(self, x2d: torch.Tensor, W: nn.Linear, A: torch.Tensor, B: torch.Tensor, order: int) -> torch.Tensor:
        tmp = x2d @ A              # (N, r)
        delta = tmp @ B.t()        # (N, out)
        if order >= 2:
            delta2 = (tmp @ B.t())
            delta = delta + 0.5 * delta2
        return delta

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        base_out = self.base(x)

        x2d, _ = self._flatten(x)
        h2d, _ = self._flatten(base_out.detach())

        pol = self._decide_policy(h2d)

        with torch.no_grad():
            U, V, in_f, out_f = self.builder(x2d, h2d, self.base, grad_h=None, target_step=None)
            r = U.shape[1]
            Wt = self.base.weight.t()                   # (in, out)
            A = Wt @ V                                  # (in, r)
            B = self.base.weight @ U                    # (out, r)
            tau = float(getattr(self.cfg, "tau", 0.8))
            A, B = _spectral_norm_clip(A, B, tau)

        delta = self._compute_delta(x2d, self.base, A.to(x2d.dtype), B.to(x2d.dtype), pol.order)

        if self.cfg.clamp is not None:
            c = float(self.cfg.clamp)
            delta = delta.clamp(min=-c, max=c)

        if self.lbfgs is not None:
            g = delta.detach().reshape(-1).to(torch.float32)
            d = self.lbfgs.two_loop(g)
            delta = delta + 1e-3 * d.reshape_as(delta).to(delta.dtype)

        y2d = base_out.reshape(-1, base_out.size(-1))
        y2d = y2d + float(pol.gamma) * delta
        return self._unflatten(y2d, base_out.shape)


Overwriting vps/vpscore/vps_linear.py


In [ ]:
%%writefile vps/vpscore/ephemeral_lbfgs.py
from __future__ import annotations
import torch
from collections import deque

class EphemeralLBFGS:
    """
    Tiny L-BFGS memory for preconditioning a 1D gradient-like vector.
    - Keeps up to m curvature pairs (s_k, y_k)
    - Provides two_loop(g) that returns an L-BFGS-preconditioned direction
    Notes:
      • This is *ephemeral*: we don't persist across runs; callers may or may not
        populate (S, Y). If empty, two_loop(g) returns g (identity preconditioner).
      • All math is done in float32 for stability, then cast back to g.dtype.
    """
    def __init__(self, m: int = 5):
        self.m = int(max(1, m))
        self.S: deque[torch.Tensor] = deque(maxlen=self.m)
        self.Y: deque[torch.Tensor] = deque(maxlen=self.m)

    @torch.no_grad()
    def update(self, s: torch.Tensor, y: torch.Tensor):
        """
        Add a curvature pair (s, y). Inputs can be any shape; we flatten to 1D.
        A simple curvature check keeps only pairs with positive y·s.
        """
        if s is None or y is None:
            return
        s = s.reshape(-1).detach()
        y = y.reshape(-1).detach()
        if s.numel() == 0 or y.numel() == 0:
            return
        # basic curvature check
        ys = torch.dot(y.to(torch.float32), s.to(torch.float32))
        if torch.isfinite(ys) and ys > 1e-8:
            self.S.append(s.clone())
            self.Y.append(y.clone())

    @torch.no_grad()
    def two_loop(self, g: torch.Tensor) -> torch.Tensor:
        """
        Standard L-BFGS two-loop recursion.
        If no history is available, returns g unchanged.
        """
        if g is None:
            return g
        # Work in float32 for robustness (esp. under fp16/bf16 models)
        device = g.device
        g32 = g.detach().reshape(-1).to(torch.float32)

        m = len(self.S)
        if m == 0:
            return g  # identity preconditioner

        # Prepare copies in float32 on the right device
        S = [self.S[i].to(device=device, dtype=torch.float32) for i in range(m)]
        Y = [self.Y[i].to(device=device, dtype=torch.float32) for i in range(m)]

        rho = []
        for i in range(m):
            ys = torch.dot(Y[i], S[i])
            # Guard against tiny/negative curvature
            rho.append(1.0 / float(ys.item()) if ys > 1e-12 else 0.0)

        # First loop: backward
        q = g32.clone()
        alpha = [0.0] * m
        for i in range(m - 1, -1, -1):
            if rho[i] == 0.0:
                alpha[i] = 0.0
                continue
            alpha[i] = rho[i] * float(torch.dot(S[i], q).item())
            q.add_(Y[i], alpha=-alpha[i])

        # Initial H0 scaling: gamma0 = (s_{m-1}^T y_{m-1}) / (y_{m-1}^T y_{m-1})
        y_last = Y[-1]
        s_last = S[-1]
        denom = float(torch.dot(y_last, y_last).item())
        numer = float(torch.dot(s_last, y_last).item())
        gamma0 = numer / denom if denom > 1e-12 else 1.0
        r = q.mul(gamma0)

        # Second loop: forward
        for i in range(m):
            if rho[i] == 0.0:
                continue
            beta = rho[i] * float(torch.dot(Y[i], r).item())
            r.add_(S[i], alpha=(alpha[i] - beta))

        # Cast back to original dtype/shape
        return r.to(dtype=g.dtype).reshape_as(g)


Overwriting vps/vpscore/ephemeral_lbfgs.py


In [ ]:
%%writefile vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation suite:
# - GSM8K (100), BBH Date Understanding (100), ARC-Challenge (100)
# - Baseline (VPS off) vs VPS (on), identical decode (greedy)
# - Reports: accuracy, latency, tokens/sec, VRAM deltas

import argparse, time, math, re, random, statistics as stats
import torch
from datasets import load_dataset
from transformers import set_seed

from vpscore.config import VPSConfig
from vpscore.verifiers.composite_verifier import CompositeVerifier
from vpscore.utils.generation import generate
from vpscore.hooks import HookManager
from vpscore.math_utils import compute_token_entropy
from vps.scripts.infer_vps import build  # reuse your build() exactly

# ---------- Helpers ----------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")
def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _normalize(s: str):
    return (s or "").strip().lower()

def _extract_choice_letter(s: str):
    s = s.strip()
    # Prefer the last single upper letter A-E
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    # Or parse like "answer: (C)" / "C)"
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    return None

def _answer_is_correct(task, pred, gold):
    if task == "gsm8k":
        pn, gn = _extract_number(pred), _extract_number(gold)
        if pn is None or gn is None: return False
        # tolerant to minor formatting; exact numeric match
        return abs(pn - gn) < 1e-6
    elif task == "bbh_date":
        return _normalize(pred) == _normalize(gold)
    elif task == "arc_c":
        return _extract_choice_letter(pred) == str(gold).upper()
    else:
        return False

def _format_prompt(task, x):
    if task == "gsm8k":
        q = x["question"] if "question" in x else x["question_text"]
        return f"Solve the problem. Return ONLY the final number.\n\nProblem: {q}\nAnswer:"
    elif task == "bbh_date":
        return f"Answer the question correctly. Return ONLY the final answer string.\n\n{ x['input'] }\nAnswer:"
    elif task == "arc_c":
        q = x["question"]
        labels = x["choices"]["label"]
        texts  = x["choices"]["text"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, texts)])
        return (
          "You are given a multiple-choice science question. "
          "Return ONLY the correct option letter (A, B, C, D, or E).\n\n"
          f"{q}\n\n{opts}\n\nAnswer:"
        )
    else:
        raise ValueError(task)

def _gold_text(task, x):
    if task == "gsm8k":
        # gsm8k answer strings are like "... #### 42"
        ans = x["answer"]
        m = re.search(r"####\s*(.+)", ans)
        return m.group(1).strip() if m else ans.strip()
    elif task == "bbh_date":
        # bbh fields have 'target' as gold
        return str(x["target"]).strip()
    elif task == "arc_c":
        return str(x["answerKey"]).strip().upper()
    else:
        raise ValueError(task)

# ---------- Iterative VPS step (mirrors infer_vps.py) ----------
def vps_iterate_once(model, tok, hooks, prompt:str, gold_text:str, max_new_tokens:int):
    """
    Do the same two-phase you used:
      - compute entropy on prompt
      - deterministic decode for verifier text
      - verifier loss -> policy.update_outcome
      - short CE surrogate backward vs gold tail to populate grad_h
      - final VPS-enhanced generation (greedy)
    """
    hooks.clear_buffers()

    # entropy from prompt logits
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.enable_grad():
        out = model(**inputs)
        logits = out.logits
        entropy = compute_token_entropy(logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(entropy)

    # deterministic decode for verifier evaluation
    out_ids = model.generate(
        **inputs,
        max_new_tokens=min(64, max_new_tokens),
        do_sample=False,
        return_dict_in_generate=True,
    )
    pred_text = tok.decode(out_ids.sequences[0], skip_special_tokens=True)

    # verifier
    verifier = CompositeVerifier()
    losses = verifier.compute_loss(pred_text, gold_text, model, tok, prompt)
    vloss = losses["total"]
    prev_loss = float("inf")
    improved = vloss < prev_loss  # True
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.update_outcome(improved, prev_loss - vloss)

    # CE surrogate backward to populate grad_h
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()

    # final VPS-enhanced generation (greedy)
    final = generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                     temperature=0.0, top_p=1.0, do_sample=False)
    return final, float(vloss)

# ---------- Runner ----------
def run_task(task: str, n: int, seed: int, vps_on: bool, cfg_overrides: dict):
    set_seed(seed)
    random.seed(seed)
    torch.cuda.empty_cache()

    # Build model (we set fp16 for T4)
    cfg = VPSConfig()
    cfg.dtype = "fp16"
    max_new = int(cfg_overrides.get("max_new_tokens", cfg.max_new_tokens))
    # Greedy for evaluation
    cfg.temperature = 0.0
    cfg.top_p = 1.0

    # Toggle VPS: gamma=0 => baseline
    if not vps_on:
        cfg.gamma = 0.0
        cfg.adaptive_gamma = False

    tok, model, hooks = build(cfg)
    model.eval()

    # dataset
    if task == "gsm8k":
        ds = load_dataset("gsm8k", "main")["test"]
    elif task == "bbh_date":
        ds = load_dataset("lukaemon/bbh", "date_understanding")["test"]
    elif task == "arc_c":
        ds = load_dataset("ai2_arc", "ARC-Challenge")["test"]
    else:
        raise ValueError("unknown task")

    # sample n items deterministically
    idxs = list(range(len(ds)))
    random.shuffle(idxs)
    idxs = idxs[:n]

    # metrics
    correct = 0
    latencies = []
    tokens_gen = []

    vram_start = torch.cuda.max_memory_allocated() if torch.cuda.is_available() else 0

    for i, idx in enumerate(idxs, 1):
        ex = ds[idx]
        prompt = _format_prompt(task, ex)
        gold   = _gold_text(task, ex)

        # Measure
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        t0 = time.time()

        if vps_on:
            pred, _ = vps_iterate_once(model, tok, hooks, prompt, gold, max_new)
        else:
            pred = generate(model, tok, prompt, max_new_tokens=max_new,
                            temperature=0.0, top_p=1.0, do_sample=False)

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        dt = time.time() - t0
        latencies.append(dt)

        # crude token count (new tokens only)
        in_ids = tok(prompt, return_tensors="pt")["input_ids"]
        out_ids = tok(pred, return_tensors="pt")["input_ids"]
        tokens_gen.append(int(out_ids.shape[1]))

        ok = _answer_is_correct(task, pred, gold)
        correct += int(ok)

        if i % 10 == 0:
            print(f"[{task}] {i}/{n} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s")

    vram_end = torch.cuda.max_memory_allocated() if torch.cuda.is_available() else 0

    acc = correct / n
    t_mean = stats.mean(latencies)
    tokps = (sum(tokens_gen) / sum(latencies)) if sum(latencies) > 0 else 0.0
    vram_mb = (vram_end - vram_start) / (1024**2)

    return {
        "task": task,
        "n": n,
        "acc": acc,
        "latency_mean_s": t_mean,
        "tokens_per_sec": tokps,
        "vram_delta_mb": vram_mb,
    }

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=100)
    ap.add_argument("--n_bbh_date", type=int, default=100)
    ap.add_argument("--n_arc_c", type=int, default=100)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=64)
    args = ap.parse_args()

    cfg_over = {"max_new_tokens": args.max_new_tokens}

    report = {"baseline": {}, "vps": {}, "delta": {}}
    results_base = []
    results_vps = []

    # BASELINE
    for task, n in [("gsm8k", args.n_gsm8k),
                    ("bbh_date", args.n_bbh_date),
                    ("arc_c", args.n_arc_c)]:
        print(f"\n=== BASELINE (VPS OFF) :: {task} n={n} ===")
        res = run_task(task, n, args.seed, vps_on=False, cfg_overrides=cfg_over)
        results_base.append(res)
        print(res)

    # VPS
    for task, n in [("gsm8k", args.n_gsm8k),
                    ("bbh_date", args.n_bbh_date),
                    ("arc_c", args.n_arc_c)]:
        print(f"\n=== VPS ON :: {task} n={n} ===")
        res = run_task(task, n, args.seed, vps_on=True, cfg_overrides=cfg_over)
        results_vps.append(res)
        print(res)

    # Summarize deltas
    def _as_map(lst): return {r["task"]: r for r in lst}
    B = _as_map(results_base); V = _as_map(results_vps)

    print("\n========= SUMMARY =========")
    for task in ["gsm8k", "bbh_date", "arc_c"]:
        b, v = B[task], V[task]
        d_acc = (v["acc"] - b["acc"]) * 100.0
        d_lat = (v["latency_mean_s"] / max(1e-9, b["latency_mean_s"]) - 1.0) * 100.0
        d_tok = (v["tokens_per_sec"] / max(1e-9, b["tokens_per_sec"]) - 1.0) * 100.0
        print(f"{task:12s}  acc: {b['acc']:.3f} -> {v['acc']:.3f}  (Δ {d_acc:+.2f} pp) | "
              f"latency: {b['latency_mean_s']:.2f}s -> {v['latency_mean_s']:.2f}s (Δ {d_lat:+.1f}%) | "
              f"toks/s: {b['tokens_per_sec']:.1f} -> {v['tokens_per_sec']:.1f} (Δ {d_tok:+.1f}%)")

if __name__ == "__main__":
    main()


Writing vps/scripts/eval_suite.py


In [ ]:
%%bash
set -e

# vps/__init__.py
cat > vps/__init__.py <<'PY'
# VPS package init
# (keep this minimal; avoid notebook magics inside files)
from . import vpscore
__all__ = ["vpscore"]
PY

# vps/vpscore/__init__.py
cat > vps/vpscore/__init__.py <<'PY'
from .config import VPSConfig
from .patch_hf import patch_model_with_vps
from .vps_linear import VPSLinear
PY

# vps/vpscore/utils/__init__.py
cat > vps/vpscore/utils/__init__.py <<'PY'
from .generation import generate
PY

# vps/vpscore/verifiers/__init__.py
cat > vps/vpscore/verifiers/__init__.py <<'PY'
from .composite_verifier import CompositeVerifier
PY

# vps/vpscore/data/__init__.py
cat > vps/vpscore/data/__init__.py <<'PY'
# data pkg
PY


In [ ]:
import sys
mods = [m for m in list(sys.modules) if m == "vps" or m.startswith("vps.")]
for m in mods:
    sys.modules.pop(m, None)
print("Cleared:", mods)


Cleared: []


In [ ]:
%%writefile vps/vpscore/patch_hf.py
from __future__ import annotations
import weakref
import torch.nn as nn
from typing import List, Any
from .vps_linear import VPSLinear

# Canonical buckets and their common alias names inside HF models
NAMES = {
    "attn_q": ["q_proj", "wq", "query", "q"],
    "attn_k": ["k_proj", "wk", "key", "k"],
    "attn_v": ["v_proj", "wv", "value", "v"],
    "attn_o": ["o_proj", "wo", "out_proj", "o"],
    "mlp_up": ["up_proj", "w1", "fc_in", "dense_h_to_4h", "gate_proj", "w3", "gate"],
    "mlp_down": ["down_proj", "w2", "fc_out", "dense_4h_to_h"],
}

# Global registry for Q/K pairing across modules (keyed by parent block id)
PAIR_REGISTRY = {}

def _normalize_apply_to(apply_to: List[str]) -> List[str]:
    """
    Accept both bucket keys (e.g., 'attn_q') and raw aliases (e.g., 'q_proj').
    Return a list of canonical bucket keys that should be applied.
    """
    if not apply_to:
        return []
    want = set()
    a2 = [s.lower() for s in apply_to]
    for key, aliases in NAMES.items():
        if key in a2 or any(alias in a2 for alias in aliases):
            want.add(key)
    return list(want)

def _match_bucket(child_name: str, wanted_buckets: List[str]):
    """
    Return the canonical bucket key for this named child if it should be wrapped.
    """
    lname = child_name.lower()
    for key in wanted_buckets:
        aliases = NAMES.get(key, [])
        if any(alias in lname for alias in aliases):
            return key
    return None

def _wrap(module: nn.Module, wanted_buckets: List[str], vps_kwargs: dict, counter: dict):
    """
    Recursively replace selected nn.Linear children with VPSLinear.
    Also record Q/K modules per parent block for optional coupling.
    """
    for name, child in list(module.named_children()):
        # Recurse first
        _wrap(child, wanted_buckets, vps_kwargs, counter)

        # Replace target Linear layers
        if isinstance(child, nn.Linear):
            bucket = _match_bucket(name, wanted_buckets)
            if bucket is not None:
                vps = VPSLinear(child, **vps_kwargs)
                setattr(module, name, vps)  # install wrapper
                counter["wrapped"] += 1

                # Track possible Q/K pairs under the parent block
                if bucket in ("attn_q", "attn_k"):
                    key = id(module)
                    PAIR_REGISTRY.setdefault(key, {})[bucket] = vps

def patch_model_with_vps(model: nn.Module, apply_to: List[str], *args: Any, **kwargs: Any):
    """
    Flexible wrapper: accepts either
      A) long-form positional:
         (model, apply_to, rank, topk, clamp, gamma, builder, [softgrad_builder=False], policy_cfg=None)
         or extended:
         (model, apply_to, rank, topk, clamp, gamma, builder, order, qk_coupling, tau,
          lbfgs_enabled, adaptive_rank, adaptive_gamma, [softgrad_builder=False], policy_cfg=None)

      B) cfg-form:
         (model, apply_to, cfg)   where cfg is VPSConfig
    """
    # Defaults
    softgrad_builder = kwargs.pop("softgrad_builder", False)
    policy_cfg = kwargs.pop("policy_cfg", None)
    qk_coupling_flag = False

    # ---- Parse inputs ----
    if len(args) == 1 and not isinstance(args[0], (int, float, str, bool)):
        # cfg-form
        cfg = args[0]
        rank   = getattr(cfg, "rank", 2)
        topk   = getattr(cfg, "topk", 32)
        clamp  = getattr(cfg, "clamp", None)
        gamma  = getattr(cfg, "gamma", 0.5)
        builder= getattr(cfg, "builder", "hybrid")
        softgrad_builder = getattr(cfg, "softgrad_builder", softgrad_builder)
        policy_cfg = cfg
        qk_coupling_flag = bool(getattr(cfg, "qk_coupling", False))
    else:
        # long-form(s)
        if len(args) >= 6:
            rank, topk, clamp, gamma, builder = args[0:6]
            if len(args) >= 12:
                # args[6] is order (unused here), args[7] is qk_coupling
                qk_coupling_flag = bool(args[7])
            if len(args) >= 13 and isinstance(args[12], bool):
                softgrad_builder = args[12]
            if len(args) >= 14:
                policy_cfg = args[13]
        else:
            raise TypeError(
                "patch_model_with_vps: unsupported arguments. "
                "Pass VPSConfig as third arg OR full long-form params."
            )

    vps_kwargs = dict(
        rank=rank,
        topk=topk,
        clamp=clamp,
        gamma=gamma,
        builder=builder,
        softgrad_builder=softgrad_builder,
        policy_cfg=policy_cfg,
    )

    wanted_buckets = _normalize_apply_to(apply_to or [])
    counter = {"wrapped": 0}

    # Actually swap in VPSLinear for selected Linear layers
    _wrap(model, wanted_buckets, vps_kwargs, counter)

    # Finalize Q/K pairing if enabled — use WEAKREFS to avoid module graph cycles
    if qk_coupling_flag:
        for pair in PAIR_REGISTRY.values():
            q = pair.get("attn_q")
            k = pair.get("attn_k")
            if q is not None and k is not None:
                # flags are plain attrs
                q.is_Q = True
                k.is_K = True
                # store weak references (NOT modules) to avoid child-module cycles
                q._peer_ref = weakref.ref(k)
                k._peer_ref = weakref.ref(q)
                # ensure any accidental strong refs don't linger
                if hasattr(q, "_peer"): delattr(q, "_peer")
                if hasattr(k, "_peer"): delattr(k, "_peer")

    try:
        print(f"[VPS] Wrapped layers: {counter['wrapped']}")
    except Exception:
        pass

    return model


Overwriting vps/vpscore/patch_hf.py


In [ ]:
# Clear any previously-imported broken modules
import sys
for m in [m for m in list(sys.modules) if m == "vps" or m.startswith("vps.")]:
    sys.modules.pop(m, None)
print("Cleared vps imports")


Cleared vps imports


In [ ]:
!PYTHONPATH=/content:$PYTHONPATH TF_CPP_MIN_LOG_LEVEL=3 \
python -u vps/scripts/eval_suite.py \
  --n_gsm8k 40 --n_bbh_date 40 --n_arc_c 40 \
  --max_new_tokens 48 --seed 1234 \
  --print_every 1 --self_consistency_weight 0.0


usage: eval_suite.py [-h] [--n_gsm8k N_GSM8K] [--n_bbh_date N_BBH_DATE]
                     [--n_arc_c N_ARC_C] [--seed SEED]
                     [--max_new_tokens MAX_NEW_TOKENS]
eval_suite.py: error: unrecognized arguments: --print_every 1 --self_consistency_weight 0.0


In [ ]:
%%bash
set -e
cat > vps/scripts/eval_suite.py <<'PY'
import argparse, time, re, random, statistics as stats, sys
import torch
from datasets import load_dataset
from transformers import set_seed

from vps.scripts.infer_vps import build          # reuse your build()
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.hooks import HookManager
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier

NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _normalize(s: str):
    return (s or "").strip().lower()

def _extract_choice_letter(s: str):
    s = s.strip()
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    return None

def _answer_is_correct(task, pred, gold):
    if task == "gsm8k":
        pn, gn = _extract_number(pred), _extract_number(gold)
        return (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
    elif task == "bbh_date":
        return _normalize(pred) == _normalize(gold)
    elif task == "arc_c":
        return _extract_choice_letter(pred) == str(gold).upper()
    return False

def _format_prompt(task, x):
    if task == "gsm8k":
        q = x["question"] if "question" in x else x["question_text"]
        return f"Solve the problem. Return ONLY the final number.\n\nProblem: {q}\nAnswer:"
    if task == "bbh_date":
        return f"Answer correctly. Return ONLY the final answer string.\n\n{ x['input'] }\nAnswer:"
    if task == "arc_c":
        q = x["question"]
        labels = x["choices"]["label"]
        texts  = x["choices"]["text"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, texts)])
        return ("You are given a multiple-choice science question. "
                "Return ONLY the correct option letter (A, B, C, D, or E).\n\n"
                f"{q}\n\n{opts}\n\nAnswer:")
    raise ValueError(task)

def _gold_text(task, x):
    if task == "gsm8k":
        ans = x["answer"]
        m = re.search(r"####\s*(.+)", ans)
        return m.group(1).strip() if m else ans.strip()
    if task == "bbh_date":
        return str(x["target"]).strip()
    if task == "arc_c":
        return str(x["answerKey"]).strip().upper()
    raise ValueError(task)

def vps_iterate_once(model, tok, hooks, prompt:str, gold_text:str, max_new_tokens:int):
    hooks.clear_buffers()

    # entropy from prompt logits
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.enable_grad():
        out = model(**inputs)
        logits = out.logits
        entropy = compute_token_entropy(logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(entropy)

    # deterministic decode for verifier text
    out_ids = model.generate(
        **inputs,
        max_new_tokens=min(64, max_new_tokens),
        do_sample=False,
        return_dict_in_generate=True,
    )
    pred_text = tok.decode(out_ids.sequences[0], skip_special_tokens=True)

    # light verifier (no self-consistency to keep it fast)
    verifier = CompositeVerifier({"numeric":1.0,"units":0.5,"algebraic":0.2,"self_consistency":0.0})
    _ = verifier.compute_loss(pred_text, gold_text)

    # short CE surrogate to populate grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()

    # final VPS-enhanced generation (greedy)
    final = generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                     temperature=0.0, top_p=1.0, do_sample=False)
    return final

def run_phase(tasks, n_map, seed, vps_on, max_new_tokens, print_every=10):
    set_seed(seed)
    random.seed(seed)
    torch.cuda.empty_cache()

    # Build once per phase
    cfg = VPSConfig()
    cfg.dtype = "fp16"
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    if not vps_on:
        cfg.gamma = 0.0
        cfg.adaptive_gamma = False

    print(f"\n[Phase] {'VPS ON' if vps_on else 'BASELINE (VPS OFF)'} — building model...", flush=True)
    tok, model, hooks = build(cfg)
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []

    for task in tasks:
        n = int(n_map.get(task, 0))
        if n <= 0:
            print(f"\n=== {('VPS' if vps_on else 'BASELINE')} :: {task} n=0 — skipping ===", flush=True)
            continue

        print(f"\n=== {('VPS' if vps_on else 'BASELINE')} :: {task} n={n} ===", flush=True)

        # dataset load
        if task == "gsm8k":
            ds = load_dataset("gsm8k", "main")["test"]
        elif task == "bbh_date":
            ds = load_dataset("lukaemon/bbh", "date_understanding")["test"]
        elif task == "arc_c":
            ds = load_dataset("ai2_arc", "ARC-Challenge")["test"]
        else:
            raise ValueError(task)

        print(f"[{task}] dataset size={len(ds)}; sampling n={n}", flush=True)

        # sample n items
        idxs = list(range(len(ds)))
        random.shuffle(idxs)
        idxs = idxs[:n]

        correct = 0
        latencies = []
        tokens_gen = []
        ema_dt = None

        for i, idx in enumerate(idxs, 1):
            ex = ds[idx]
            prompt = _format_prompt(task, ex)
            gold   = _gold_text(task, ex)

            if torch.cuda.is_available(): torch.cuda.synchronize()
            t0 = time.time()

            if vps_on:
                pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens)
            else:
                pred = generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                                temperature=0.0, top_p=1.0, do_sample=False)

            if torch.cuda.is_available(): torch.cuda.synchronize()
            dt = time.time() - t0
            latencies.append(dt)
            ema_dt = dt if ema_dt is None else (0.9*ema_dt + 0.1*dt)

            out_ids = tok(pred, return_tensors="pt")["input_ids"]
            tokens_gen.append(int(out_ids.shape[1]))

            ok = _answer_is_correct(task, pred, gold)
            correct += int(ok)

            if (i % print_every == 0) or (i in (1, n)):
                rem = n - i
                eta = rem * (ema_dt or dt)
                mins = int(eta // 60); secs = int(eta % 60)
                print(f"[{task}] {i}/{n} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {mins}m{secs}s",
                      flush=True)

        acc = correct / n
        t_mean = stats.mean(latencies) if latencies else 0.0
        tokps = (sum(tokens_gen) / sum(latencies)) if sum(latencies) > 0 else 0.0

        res = {
            "task": task,
            "n": n,
            "acc": acc,
            "latency_mean_s": t_mean,
            "tokens_per_sec": tokps,
        }
        print(res, flush=True)
        results.append(res)

    return results

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=100)
    ap.add_argument("--n_bbh_date", type=int, default=100)
    ap.add_argument("--n_arc_c", type=int, default=100)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=64)
    ap.add_argument("--print_every", type=int, default=10)  # NEW but optional
    args = ap.parse_args()

    tasks = ["gsm8k", "bbh_date", "arc_c"]
    n_map = {"gsm8k": args.n_gsm8k, "bbh_date": args.n_bbh_date, "arc_c": args.n_arc_c}

    # Phase 1: Baseline
    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens,
                  print_every=args.print_every)

    # Phase 2: VPS
    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens,
                  print_every=args.print_every)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
python - <<'PY'
from transformers import AutoTokenizer, AutoModelForCausalLM
m = "Qwen/Qwen2.5-1.5B-Instruct"
print("Downloading model + tokenizer to local cache...", flush=True)
AutoTokenizer.from_pretrained(m, use_fast=True)
AutoModelForCausalLM.from_pretrained(m, device_map="cpu")
print("Done. Subsequent runs will start instantly.", flush=True)
PY



Process is terminated.


In [ ]:

%%bash
set -e
python - <<'PY'
import sys, os
os.environ["PYTHONUNBUFFERED"]="1"
print("[diag] importing eval_suite...", flush=True)
from vps.scripts.eval_suite import run_phase
print("[diag] import OK. Running 2 GSM8K items baseline...", flush=True)
_ = run_phase(["gsm8k"], {"gsm8k": 2}, seed=1234, vps_on=False, max_new_tokens=24, print_every=1)
print("[diag] baseline OK. Now VPS ON (2 items)...", flush=True)
_ = run_phase(["gsm8k"], {"gsm8k": 2}, seed=1234, vps_on=True, max_new_tokens=24, print_every=1)
print("[diag] done.", flush=True)
PY


Process is interrupted.


In [ ]:
%%bash
set -e
# ensure "vps/scripts" is a package
mkdir -p vps/scripts
if [ ! -f vps/scripts/__init__.py ]; then
  touch vps/scripts/__init__.py
fi
echo "ok"


ok


In [ ]:
%%bash
set -e
cat > vps/scripts/infer_vps.py <<'PY'
import argparse
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ---- robust imports: package-first, legacy fallback ----
try:
    from vps.vpscore.config import VPSConfig
    from vps.vpscore.patch_hf import patch_model_with_vps
    from vps.vpscore.verifiers.composite_verifier import CompositeVerifier
    from vps.vpscore.utils.generation import generate
    from vps.vpscore.hooks import HookManager
    from vps.vpscore.math_utils import compute_token_entropy
except Exception:
    # allow running with PYTHONPATH=/content/vps
    from vpscore.config import VPSConfig
    from vpscore.patch_hf import patch_model_with_vps
    from vpscore.verifiers.composite_verifier import CompositeVerifier
    from vpscore.utils.generation import generate
    from vpscore.hooks import HookManager
    from vpscore.math_utils import compute_token_entropy

DEFAULT_VERIFIER_WEIGHTS = {
    "exact_match": 0.8,
    "coherence": 0.2,
    "length_penalty": 0.0,
}

def build(cfg: VPSConfig):
    # ---- robust fallbacks so missing config fields don't crash ----
    seed = getattr(cfg, "seed", 1234)
    torch.manual_seed(seed)

    dtype_name = getattr(cfg, "dtype", "bf16")
    _dtype_map = {"bf16": torch.bfloat16, "fp16": torch.float16, "fp32": torch.float32}
    dtype = _dtype_map.get(dtype_name, torch.bfloat16)

    model_name = getattr(cfg, "model_name", None)
    if not model_name:
        raise ValueError("cfg.model_name is not set in VPSConfig")

    device_map = getattr(cfg, "device_map", "auto")

    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token_id = tok.eos_token_id

    # Use dtype= (not torch_dtype=) to avoid the deprecation warning.
    model = AutoModelForCausalLM.from_pretrained(model_name, dtype=dtype, device_map=device_map)

    # Your patcher: the variant we standardized earlier takes (model, apply_to, cfg)
    apply_to = getattr(cfg, "apply_to", None)
    model = patch_model_with_vps(model, apply_to, cfg)

    hooks = HookManager()
    hooks.attach(model)
    return tok, model, hooks

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--prompt", type=str, required=True)
    ap.add_argument("--gold", type=str, default=None)
    ap.add_argument("--iters", type=int, default=3)
    args = ap.parse_args()

    cfg = VPSConfig()

    tok, model, hooks = build(cfg)

    # Fall back if cfg.verifier_weights missing
    verifier_weights = getattr(cfg, "verifier_weights", DEFAULT_VERIFIER_WEIGHTS)
    verifier = CompositeVerifier(verifier_weights)

    # Also fallbacks for gen params
    max_new_tokens = getattr(cfg, "max_new_tokens", 128)
    temperature = getattr(cfg, "temperature", 0.7)
    top_p = getattr(cfg, "top_p", 0.9)

    # ----------------- Iteration 0 (plain decode) -----------------
    text = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
    print(f"=== Iteration 0 ===\n{text}\n")

    if args.gold is None or args.iters < 2:
        return

    prev_loss = float("inf")

    for it in range(1, args.iters):
        hooks.clear_buffers()

        # Clear any L-BFGS memories (keeps your technique intact)
        for m in model.modules():
            if hasattr(m, "clear_lbfgs"):
                m.clear_lbfgs()

        # Forward pass to compute token entropy for the adaptive policy
        inputs = tok(args.prompt, return_tensors="pt").to(model.device)
        with torch.enable_grad():
            out = model(**inputs)
            logits = out.logits
            entropy = compute_token_entropy(logits[0, -1, :])

        # Feed entropy to policies (again, preserving your mechanism)
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.set_token_entropy(entropy)

        # Deterministic decode for verifier evaluation
        out_ids = model.generate(
            **inputs,
            max_new_tokens=min(64, max_new_tokens),
            do_sample=False,
            return_dict_in_generate=True,
        )
        pred_text = tok.decode(out_ids.sequences[0], skip_special_tokens=True)

        # Multi-objective verifier (unchanged logic)
        losses = verifier.compute_loss(pred_text, args.gold, model, tok, args.prompt)
        vloss = losses["total"]

        # Feedback loop to policy
        improved = vloss < prev_loss
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.update_outcome(improved, prev_loss - vloss)
        prev_loss = vloss

        # Short CE surrogate to populate grads → VPS uses grad_h
        gold_ids = tok(args.gold, return_tensors="pt").to(model.device)["input_ids"][0]
        T = min(8, gold_ids.shape[0])
        target = gold_ids[-T:].unsqueeze(0)  # [1, T]

        model.zero_grad(set_to_none=True)
        with torch.enable_grad():
            out2 = model(**inputs)
            logits2 = out2.logits[:, -T:, :]  # [1, T, V]
            ce = torch.nn.functional.cross_entropy(
                logits2.reshape(-1, logits2.size(-1)),
                target.reshape(-1)
            )
            ce.backward()

        # VPS-enhanced generation
        text2 = generate(model, tok, args.prompt, max_new_tokens, temperature, top_p)
        print(f"=== Iteration {it} (VPS enhanced) ===")
        print(f"Losses: {losses}")
        print(f"Output:\n{text2}\n")

if __name__ == "__main__":
    main()
PY
echo "infer_vps.py rewritten"


infer_vps.py rewritten


In [ ]:

%%bash
python - <<'PY'
from pathlib import Path, re
p = Path('vps/vpscore/config.py')
s = p.read_text()
# Use 3B for more headroom (still fits on T4)
s = s.replace('Qwen/Qwen2.5-1.5B-Instruct','Qwen/Qwen2.5-3B-Instruct')
# Make VPS slightly more expressive + stable for MC tasks
s = s.replace('rank: int = 2','rank: int = 4')\
     .replace('topk: int = 32','topk: int = 64')\
     .replace('order: int = 1','order: int = 2')\
     .replace('gamma: float = 0.5','gamma: float = 0.6')\
     .replace('temperature: float = 0.2','temperature: float = 0.0')  # greedy for MC stability
# Ensure clamp is on (prevents rare spikes)
s = s.replace('clamp: Optional[float] = None','clamp: Optional[float] = 0.2')
p.write_text(s)
print("VPS config tuned for MC/short reasoning ✅")
PY


VPS config tuned for MC/short reasoning ✅


In [ ]:
# vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation with progress + OOM-safe VPS pass.
from __future__ import annotations

import os
import time
import math
import random
import argparse
from typing import Dict, List, Tuple

import torch
from datasets import load_dataset

# Reuse your build() and utilities exactly
from vps.scripts.infer_vps import build             # creates (tok, model, hooks)
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier


# -----------------------------
# Helper: toggle VPS on / off
# -----------------------------
def set_vps_enabled(model: torch.nn.Module, enabled: bool):
    """
    Make the VPS delta inert for baseline by forcing gamma=0 and disabling adaptivity.
    We do NOT remove wrappers; we just neutralize them. This keeps your patching intact.
    """
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            # Hard-disable effect for baseline
            if not enabled:
                if hasattr(m, "cfg"):
                    m.cfg.adaptive_gamma = False
                    m.cfg.gamma = 0.0
                if hasattr(m, "policy") and m.policy is not None:
                    # keep policy object but pin its gamma via cfg above
                    m.policy.cfg = m.cfg
            else:
                # Re-enable whatever the layer already had (do nothing)
                if hasattr(m, "policy") and m.policy is not None:
                    m.policy.cfg = m.cfg


# ------------------------------------------
# OOM-safe single VPS "iteration" (no grads)
# ------------------------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold: str | None, max_new_tokens: int):
    """
    A light VPS pass that:
      1) does a forward pass (no grad) to get token entropy,
      2) feeds entropy to per-layer policies (so they can adjust),
      3) runs greedy generation with VPS active.
    NOTE: we intentionally do NOT build any backward graph here (prevents OOM).
    """
    hooks.clear_buffers()

    # Step 1: entropy from a cheap forward (no cache -> lower mem)
    with torch.no_grad():
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model(**inputs, use_cache=False, return_dict=True)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])

    # Step 2: feed entropy to policies
    ent_val = float(ent) if torch.is_tensor(ent) else float(ent)
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(ent_val)
            except Exception:
                pass

    # Step 3: VPS-influenced generation (greedy for MC stability)
    pred_text = generate(
        model, tok, prompt,
        max_new_tokens=max_new_tokens,
        temperature=0.0, top_p=1.0, do_sample=False
    )
    return pred_text


# -----------------------------
# ARC-Challenge prompt helpers
# -----------------------------
def format_arc_prompt(q: str, choices: List[str]) -> str:
    letters = "ABCDE"
    lines = [f"Question: {q}", "Options:"]
    for i, c in enumerate(choices):
        lines.append(f"({letters[i]}) {c}")
    lines.append("Answer:")
    return "\n".join(lines)

def extract_arc_choice_letter(text: str) -> str | None:
    # Find last capital A–E in the output (common HF eval trick)
    for ch in reversed(text.strip()):
        if ch in "ABCDE":
            return ch
    return None


# -----------------------------
# BBH date prompt helpers
# -----------------------------
def format_bbh_date_prompt(q: str) -> str:
    # The BBH prompts are usually already formatted; we just ensure a clear cue.
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# GSM8K prompt helpers (short)
# -----------------------------
def format_gsm8k_prompt(q: str) -> str:
    # Keep short question → short answer; no CoT for speed/consistency.
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# Task runners
# -----------------------------
def run_arc_c(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[arc_c] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        choices = item["choices"]["text"]
        labels = item["choices"]["label"]
        gold_letter = item["answerKey"].strip()

        prompt = format_arc_prompt(q, choices)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold_letter, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        # decode letter
        got = extract_arc_choice_letter(pred) or ""
        if got == gold_letter:
            correct += 1

        # token estimate (approx decode length)
        gen_token_est += len(tok.encode(pred))  # rough but consistent

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            # naive ETA based on last example duration
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[arc_c] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "arc_c", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_bbh_date(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[bbh_date] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["input"]
        gold = item["target"].strip()

        prompt = format_bbh_date_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens=24)
        dt = time.time() - t0
        latencies.append(dt)

        got = pred.strip().splitlines()[-1].strip()
        if got == gold:
            correct += 1

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[bbh_date] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "bbh_date", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_gsm8k(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    # Use the short "main" split (1319 samples)
    ds = load_dataset("openai/gsm8k", "main", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[gsm8k] dataset size={total}; sampling n={len(idx)}")

    # Tiny numeric extractor (last number)
    import re
    num_re = re.compile(r"(-?\d+(?:\.\d+)?)")

    def extract_num(s: str):
        m = list(num_re.finditer(s))
        return m[-1].group(1) if m else None

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        gold_text = item["answer"]
        # gold is "#### 123" format; pull the number
        gnum = extract_num(gold_text)

        prompt = format_gsm8k_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gnum, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        pnum = extract_num(pred)
        if pnum is not None and gnum is not None:
            try:
                correct += float(pnum) == float(gnum)
            except Exception:
                pass

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[gsm8k] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "gsm8k", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


# -----------------------------
# Phase runner (build once)
# -----------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str, int], seed: int, vps_on: bool,
              max_new_tokens: int, print_every: int) -> List[Dict]:
    random.seed(seed)
    torch.manual_seed(seed)

    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    # Build model (your build() already patches; we neutralize for baseline below)
    cfg_overrides = None  # keep your defaults
    tok, model, hooks = build(cfg_overrides)
    if torch.cuda.is_available():
        torch.cuda.set_device(0)

    # Make baseline inert
    set_vps_enabled(model, enabled=vps_on)
    model.eval()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print("[Phase] model ready.")

    results: List[Dict] = []

    for task in tasks_order:
        n = int(n_map.get(task, 0))

        if n <= 0:
            print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
            continue

        print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n={n} ===")

        if task == "arc_c":
            res = run_arc_c(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        elif task == "bbh_date":
            res = run_bbh_date(model, tok, hooks, n, print_every, max_new_tokens=min(24, max_new_tokens))
        elif task == "gsm8k":
            res = run_gsm8k(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        else:
            print(f"[warn] unknown task '{task}', skipping.")
            continue

        print(res)
        results.append(res)

        # light GC between tasks
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return results


# -----------------------------
# CLI
# -----------------------------
def parse_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=100)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=10)
    return ap.parse_args()


def main():
    args = parse_args()

    # Recommended on Colab to reduce fragmentation
    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

    tasks_order = []
    n_map = {}

    if args.n_gsm8k >= 0:
        tasks_order.append("gsm8k")
        n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date >= 0:
        tasks_order.append("bbh_date")
        n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c >= 0:
        tasks_order.append("arc_c")
        n_map["arc_c"] = args.n_arc_c

    # BASELINE (VPS OFF)
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)

    # VPS ON
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)


if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] [--n_gsm8k N_GSM8K]
                                [--n_bbh_date N_BBH_DATE] [--n_arc_c N_ARC_C]
                                [--seed SEED]
                                [--max_new_tokens MAX_NEW_TOKENS]
                                [--print_every PRINT_EVERY]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-f94432f4-3a92-45ce-9723-3e91da0b75d6.json


SystemExit: 2

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation with progress + OOM-safe VPS pass.
from __future__ import annotations

import os
import sys
import time
import math
import random
import argparse
from typing import Dict, List

# --- Make top-level 'vpscore' imports work no matter how it's launched ---
# This points to the repo's /content/vps folder (parent of this scripts/).
try:
    import pathlib
    PROJ_ROOT = str(pathlib.Path(__file__).resolve().parents[1])  # .../vps
except Exception:
    PROJ_ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
if PROJ_ROOT not in sys.path:
    sys.path.insert(0, PROJ_ROOT)
os.environ["PYTHONPATH"] = PROJ_ROOT + ":" + os.environ.get("PYTHONPATH", "")

import torch
from datasets import load_dataset

# Reuse your build() and utilities exactly
from vps.scripts.infer_vps import build             # creates (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy


# -----------------------------
# Helper: toggle VPS on / off
# -----------------------------
def set_vps_enabled(model: torch.nn.Module, enabled: bool):
    """
    Make the VPS delta inert for baseline by forcing gamma=0 and disabling adaptivity.
    We do NOT remove wrappers; we just neutralize them.
    """
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not enabled:
                if hasattr(m, "cfg"):
                    m.cfg.adaptive_gamma = False
                    m.cfg.gamma = 0.0
                if hasattr(m, "policy") and m.policy is not None:
                    m.policy.cfg = m.cfg
            else:
                if hasattr(m, "policy") and m.policy is not None:
                    m.policy.cfg = m.cfg


# ------------------------------------------
# OOM-safe single VPS "iteration" (no grads)
# ------------------------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold: str | None, max_new_tokens: int):
    """
    Light VPS pass that:
      1) forward pass (no grad) to get token entropy,
      2) feed entropy to per-layer policies (adaptive tuning),
      3) greedy generation with VPS active.
    No backward() → avoids OOM on T4.
    """
    hooks.clear_buffers()

    with torch.no_grad():
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model(**inputs, use_cache=False, return_dict=True)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])

    ent_val = float(ent) if torch.is_tensor(ent) else float(ent)
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(ent_val)
            except Exception:
                pass

    pred_text = generate(
        model, tok, prompt,
        max_new_tokens=max_new_tokens,
        temperature=0.0, top_p=1.0, do_sample=False
    )
    return pred_text


# -----------------------------
# ARC-Challenge helpers
# -----------------------------
def format_arc_prompt(q: str, choices: List[str]) -> str:
    letters = "ABCDE"
    lines = [f"Question: {q}", "Options:"]
    for i, c in enumerate(choices):
        lines.append(f"({letters[i]}) {c}")
    lines.append("Answer:")
    return "\n".join(lines)

def extract_arc_choice_letter(text: str) -> str | None:
    for ch in reversed(text.strip()):
        if ch in "ABCDE":
            return ch
    return None


# -----------------------------
# BBH date helpers
# -----------------------------
def format_bbh_date_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# GSM8K helpers
# -----------------------------
def format_gsm8k_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# Task runners
# -----------------------------
def run_arc_c(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[arc_c] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        choices = item["choices"]["text"]
        gold_letter = item["answerKey"].strip()

        prompt = format_arc_prompt(q, choices)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold_letter, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        got = extract_arc_choice_letter(pred) or ""
        if got == gold_letter:
            correct += 1

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[arc_c] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "arc_c", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_bbh_date(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[bbh_date] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["input"]
        gold = item["target"].strip()

        prompt = format_bbh_date_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens=24)
        dt = time.time() - t0
        latencies.append(dt)

        got = pred.strip().splitlines()[-1].strip()
        if got == gold:
            correct += 1

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[bbh_date] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "bbh_date", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_gsm8k(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("openai/gsm8k", "main", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[gsm8k] dataset size={total}; sampling n={len(idx)}")

    import re
    num_re = re.compile(r"(-?\d+(?:\.\d+)?)")

    def extract_num(s: str):
        m = list(num_re.finditer(s))
        return m[-1].group(1) if m else None

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        gold_text = item["answer"]
        gnum = extract_num(gold_text)

        prompt = format_gsm8k_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gnum, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        pnum = extract_num(pred)
        if pnum is not None and gnum is not None:
            try:
                correct += float(pnum) == float(gnum)
            except Exception:
                pass

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[gsm8k] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "gsm8k", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


# -----------------------------
# Phase runner (build once)
# -----------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str, int], seed: int, vps_on: bool,
              max_new_tokens: int, print_every: int) -> List[Dict]:
    random.seed(seed)
    torch.manual_seed(seed)

    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    # Build model using your builder and default VPSConfig
    tok, model, hooks = build(VPSConfig())
    if torch.cuda.is_available():
        torch.cuda.set_device(0)

    # Neutralize or enable VPS effect
    set_vps_enabled(model, enabled=vps_on)
    model.eval()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print("[Phase] model ready.")

    results: List[Dict] = []

    for task in tasks_order:
        n = int(n_map.get(task, 0))

        if n <= 0:
            print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
            continue

        print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n={n} ===")

        if task == "arc_c":
            res = run_arc_c(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        elif task == "bbh_date":
            res = run_bbh_date(model, tok, hooks, n, print_every, max_new_tokens=min(24, max_new_tokens))
        elif task == "gsm8k":
            res = run_gsm8k(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        else:
            print(f"[warn] unknown task '{task}', skipping.")
            continue

        print(res)
        results.append(res)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return results


# -----------------------------
# CLI
# -----------------------------
def parse_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=100)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=10)
    return ap.parse_args()


def main():
    args = parse_args()
    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

    tasks_order = []
    n_map = {}

    if args.n_gsm8k >= 0:
        tasks_order.append("gsm8k")
        n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date >= 0:
        tasks_order.append("bbh_date")
        n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c >= 0:
        tasks_order.append("arc_c")
        n_map["arc_c"] = args.n_arc_c

    # BASELINE (VPS OFF)
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)

    # VPS ON
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)


if __name__ == "__main__":
    main()


NameError: name '__file__' is not defined

In [ ]:
# vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation with progress + OOM-safe VPS pass.
from __future__ import annotations

import os
import sys
import time
import math
import random
import argparse
from typing import Dict, List

# --- Robust path setup: works when run as a file or via stdin/heredoc ---
try:
    import pathlib
    _has_file = "__file__" in globals()
    if _has_file:
        HERE = pathlib.Path(__file__).resolve()                  # .../vps/scripts/eval_suite.py
        PKG_ROOT = str(HERE.parents[1])                          # .../vps
        REPO_ROOT = str(HERE.parents[2])                         # .../
    else:
        # e.g., executed via stdin in Colab; fall back to CWD
        CWD = pathlib.Path(os.getcwd()).resolve()
        # If a "vps" dir exists here, treat it as package root
        if (CWD / "vpscore").exists() or (CWD / "scripts").exists():
            PKG_ROOT = str(CWD)                                  # already inside .../vps
            REPO_ROOT = str(CWD.parent)
        elif (CWD / "vps").exists():
            PKG_ROOT = str((CWD / "vps").resolve())              # .../vps
            REPO_ROOT = str(CWD.resolve())                       # .../
        else:
            # last resort: just use cwd for both
            PKG_ROOT = str(CWD)
            REPO_ROOT = str(CWD)
except Exception:
    # absolute last resort if anything above fails
    PKG_ROOT = os.path.abspath(os.path.join(os.getcwd(), "vps"))
    REPO_ROOT = os.path.abspath(os.getcwd())

# Put BOTH on sys.path so that 'vps.scripts.*' and 'vpscore.*' work
for p in (REPO_ROOT, PKG_ROOT):
    if p and p not in sys.path:
        sys.path.insert(0, p)
os.environ["PYTHONPATH"] = REPO_ROOT + ":" + PKG_ROOT + ":" + os.environ.get("PYTHONPATH", "")

import torch
from datasets import load_dataset

# Reuse your build() and utilities exactly
from vps.scripts.infer_vps import build             # creates (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy


# -----------------------------
# Helper: toggle VPS on / off
# -----------------------------
def set_vps_enabled(model: torch.nn.Module, enabled: bool):
    """
    Make the VPS delta inert for baseline by forcing gamma=0 and disabling adaptivity.
    We do NOT remove wrappers; we just neutralize them.
    """
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not enabled:
                if hasattr(m, "cfg"):
                    m.cfg.adaptive_gamma = False
                    m.cfg.gamma = 0.0
                if hasattr(m, "policy") and m.policy is not None:
                    m.policy.cfg = m.cfg
            else:
                if hasattr(m, "policy") and m.policy is not None:
                    m.policy.cfg = m.cfg


# ------------------------------------------
# OOM-safe single VPS "iteration" (no grads)
# ------------------------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold: str | None, max_new_tokens: int):
    """
    Light VPS pass that:
      1) forward pass (no grad) to get token entropy,
      2) feed entropy to per-layer policies (adaptive tuning),
      3) greedy generation with VPS active.
    No backward() → avoids OOM on T4.
    """
    hooks.clear_buffers()

    with torch.no_grad():
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model(**inputs, use_cache=False, return_dict=True)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])

    ent_val = float(ent) if torch.is_tensor(ent) else float(ent)
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(ent_val)
            except Exception:
                pass

    pred_text = generate(
        model, tok, prompt,
        max_new_tokens=max_new_tokens,
        temperature=0.0, top_p=1.0, do_sample=False
    )
    return pred_text


# -----------------------------
# ARC-Challenge helpers
# -----------------------------
def format_arc_prompt(q: str, choices: List[str]) -> str:
    letters = "ABCDE"
    lines = [f"Question: {q}", "Options:"]
    for i, c in enumerate(choices):
        lines.append(f"({letters[i]}) {c}")
    lines.append("Answer:")
    return "\n".join(lines)

def extract_arc_choice_letter(text: str) -> str | None:
    for ch in reversed(text.strip()):
        if ch in "ABCDE":
            return ch
    return None


# -----------------------------
# BBH date helpers
# -----------------------------
def format_bbh_date_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# GSM8K helpers
# -----------------------------
def format_gsm8k_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


# -----------------------------
# Task runners
# -----------------------------
def run_arc_c(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[arc_c] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        choices = item["choices"]["text"]
        gold_letter = item["answerKey"].strip()

        prompt = format_arc_prompt(q, choices)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold_letter, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        got = extract_arc_choice_letter(pred) or ""
        if got == gold_letter:
            correct += 1

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[arc_c] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "arc_c", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_bbh_date(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[bbh_date] dataset size={total}; sampling n={len(idx)}")

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["input"]
        gold = item["target"].strip()

        prompt = format_bbh_date_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens=24)
        dt = time.time() - t0
        latencies.append(dt)

        got = pred.strip().splitlines()[-1].strip()
        if got == gold:
            correct += 1

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[bbh_date] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "bbh_date", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


def run_gsm8k(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("openai/gsm8k", "main", split="test")
    total = len(ds)
    idx = list(range(total))
    random.shuffle(idx)
    idx = idx[:n] if n > 0 else []

    correct = 0
    latencies: List[float] = []
    gen_token_est: int = 0

    print(f"[gsm8k] dataset size={total}; sampling n={len(idx)}")

    import re
    num_re = re.compile(r"(-?\d+(?:\.\d+)?)")

    def extract_num(s: str):
        m = list(num_re.finditer(s))
        return m[-1].group(1) if m else None

    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        q = item["question"]
        gold_text = item["answer"]
        gnum = extract_num(gold_text)

        prompt = format_gsm8k_prompt(q)

        t0 = time.time()
        pred = vps_iterate_once(model, tok, hooks, prompt, gnum, max_new_tokens)
        dt = time.time() - t0
        latencies.append(dt)

        pnum = extract_num(pred)
        if pnum is not None and gnum is not None:
            try:
                correct += float(pnum) == float(gnum)
            except Exception:
                pass

        gen_token_est += len(tok.encode(pred))

        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            acc_so_far = correct / i if i > 0 else 0.0
            rem = len(idx) - i
            eta_s = int(dt * rem)
            print(f"[gsm8k] {i}/{len(idx)} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")

    acc = (correct / len(idx)) if len(idx) else 0.0
    mean_lat = float(sum(latencies) / len(latencies)) if latencies else 0.0
    tps = float(gen_token_est / sum(latencies)) if latencies and gen_token_est > 0 else 0.0

    return {"task": "gsm8k", "n": len(idx), "acc": acc, "latency_mean_s": mean_lat, "tokens_per_sec": tps}


# -----------------------------
# Phase runner (build once)
# -----------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str, int], seed: int, vps_on: bool,
              max_new_tokens: int, print_every: int) -> List[Dict]:
    random.seed(seed)
    torch.manual_seed(seed)

    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    # Build model using your builder and default VPSConfig
    tok, model, hooks = build(VPSConfig())
    if torch.cuda.is_available():
        torch.cuda.set_device(0)

    # Neutralize or enable VPS effect
    set_vps_enabled(model, enabled=vps_on)
    model.eval()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print("[Phase] model ready.")

    results: List[Dict] = []

    for task in tasks_order:
        n = int(n_map.get(task, 0))

        if n <= 0:
            print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
            continue

        print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n={n} ===")

        if task == "arc_c":
            res = run_arc_c(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        elif task == "bbh_date":
            res = run_bbh_date(model, tok, hooks, n, print_every, max_new_tokens=min(24, max_new_tokens))
        elif task == "gsm8k":
            res = run_gsm8k(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        else:
            print(f"[warn] unknown task '{task}', skipping.")
            continue

        print(res)
        results.append(res)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return results


# -----------------------------
# CLI
# -----------------------------
def parse_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=100)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=10)
    return ap.parse_args()


def main():
    args = parse_args()
    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

    tasks_order = []
    n_map = {}

    if args.n_gsm8k >= 0:
        tasks_order.append("gsm8k")
        n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date >= 0:
        tasks_order.append("bbh_date")
        n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c >= 0:
        tasks_order.append("arc_c")
        n_map["arc_c"] = args.n_arc_c

    # BASELINE (VPS OFF)
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)

    # VPS ON
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)


if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] [--n_gsm8k N_GSM8K]
                                [--n_bbh_date N_BBH_DATE] [--n_arc_c N_ARC_C]
                                [--seed SEED]
                                [--max_new_tokens MAX_NEW_TOKENS]
                                [--print_every PRINT_EVERY]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-f94432f4-3a92-45ce-9723-3e91da0b75d6.json


SystemExit: 2

In [ ]:
# vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation with progress + OOM-safe VPS pass.
from __future__ import annotations

import os, sys, time, random, argparse
from typing import Dict, List

# --- Robust path setup: works when run as a file or via stdin/heredoc ---
try:
    import pathlib
    if "__file__" in globals():
        HERE = pathlib.Path(__file__).resolve()          # .../vps/scripts/eval_suite.py
        PKG_ROOT = str(HERE.parents[1])                  # .../vps
        REPO_ROOT = str(HERE.parents[2])                 # .../
    else:
        CWD = pathlib.Path(os.getcwd()).resolve()
        if (CWD / "vpscore").exists() or (CWD / "scripts").exists():
            PKG_ROOT = str(CWD)
            REPO_ROOT = str(CWD.parent)
        elif (CWD / "vps").exists():
            PKG_ROOT = str((CWD / "vps").resolve())
            REPO_ROOT = str(CWD.resolve())
        else:
            PKG_ROOT = str(CWD)
            REPO_ROOT = str(CWD)
except Exception:
    PKG_ROOT = os.path.abspath(os.path.join(os.getcwd(), "vps"))
    REPO_ROOT = os.path.abspath(os.getcwd())

for p in (REPO_ROOT, PKG_ROOT):
    if p and p not in sys.path:
        sys.path.insert(0, p)
os.environ["PYTHONPATH"] = REPO_ROOT + ":" + PKG_ROOT + ":" + os.environ.get("PYTHONPATH", "")

import torch
from datasets import load_dataset

# Reuse your existing pieces
from vps.scripts.infer_vps import build             # creates (tok, model, hooks)
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy


# ----------------------------- helpers -----------------------------
def set_vps_enabled(model: torch.nn.Module, enabled: bool):
    """Neutralize VPS delta by setting gamma=0 for baseline; keep wrappers intact."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if hasattr(m, "cfg"):
                if not enabled:
                    m.cfg.adaptive_gamma = False
                    m.cfg.gamma = 0.0
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.cfg = m.cfg


def vps_iterate_once(model, tok, hooks, prompt: str, gold: str | None, max_new_tokens: int):
    """OOM-safe VPS pass: entropy→policy only, no backward()."""
    hooks.clear_buffers()
    with torch.no_grad():
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model(**inputs, use_cache=False, return_dict=True)
        ent = compute_token_entropy(out.logits[0, -1, :])
    ent_val = float(ent) if torch.is_tensor(ent) else float(ent)
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(ent_val)
            except Exception:
                pass
    return generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                    temperature=0.0, top_p=1.0, do_sample=False)


def format_arc_prompt(q: str, choices: List[str]) -> str:
    letters = "ABCDE"
    lines = [f"Question: {q}", "Options:"]
    for i, c in enumerate(choices):
        lines.append(f"({letters[i]}) {c}")
    lines.append("Answer:")
    return "\n".join(lines)

def extract_arc_choice_letter(text: str) -> str | None:
    for ch in reversed(text.strip()):
        if ch in "ABCDE":
            return ch
    return None

def format_bbh_date_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"

def format_gsm8k_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


def run_arc_c(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[arc_c] dataset size={total}; sampling n={len(idx)}")
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        prompt = format_arc_prompt(item["question"], item["choices"]["text"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, item["answerKey"], max_new_tokens); dt = time.time() - t0
        latencies.append(dt)
        if (extract_arc_choice_letter(pred) or "") == item["answerKey"].strip():
            correct += 1
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[arc_c] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"arc_c","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}

def run_bbh_date(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[bbh_date] dataset size={total}; sampling n={len(idx)}")
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        prompt = format_bbh_date_prompt(item["input"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, item["target"].strip(), max_new_tokens=24); dt = time.time() - t0
        latencies.append(dt)
        got = pred.strip().splitlines()[-1].strip()
        if got == item["target"].strip():
            correct += 1
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[bbh_date] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"bbh_date","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}

def run_gsm8k(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("openai/gsm8k", "main", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[gsm8k] dataset size={total}; sampling n={len(idx)}")
    import re
    num_re = re.compile(r"(-?\d+(?:\.\d+)?)")
    def extract_num(s: str):
        m = list(num_re.finditer(s)); return m[-1].group(1) if m else None
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        gnum = extract_num(item["answer"])
        prompt = format_gsm8k_prompt(item["question"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, gnum, max_new_tokens); dt = time.time() - t0
        latencies.append(dt)
        pnum = extract_num(pred)
        if pnum is not None and gnum is not None:
            try:
                if float(pnum) == float(gnum):
                    correct += 1
            except Exception:
                pass
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[gsm8k] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"gsm8k","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}


def run_phase(tasks_order: List[str], n_map: Dict[str, int], seed: int, vps_on: bool,
              max_new_tokens: int, print_every: int) -> List[Dict]:
    random.seed(seed); torch.manual_seed(seed)
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")
    tok, model, hooks = build()  # uses your VPSConfig defaults internally
    if torch.cuda.is_available(): torch.cuda.set_device(0)
    set_vps_enabled(model, enabled=vps_on); model.eval()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("[Phase] model ready.")
    results: List[Dict] = []
    for task in tasks_order:
        n = int(n_map.get(task, 0))
        if n <= 0:
            print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
            continue
        print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n={n} ===")
        if task == "arc_c":
            res = run_arc_c(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        elif task == "bbh_date":
            res = run_bbh_date(model, tok, hooks, n, print_every, max_new_tokens=min(24, max_new_tokens))
        elif task == "gsm8k":
            res = run_gsm8k(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        else:
            print(f"[warn] unknown task '{task}', skipping."); continue
        print(res); results.append(res)
        if torch.cuda.is_available(): torch.cuda.empty_cache()
    return results


# ----------------------------- CLI -----------------------------
def parse_args():
    ap = argparse.ArgumentParser(prog="eval_suite.py")
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=10)
    # IMPORTANT: tolerate unknown args injected by Jupyter/Colab (-f kernel.json)
    args, _unknown = ap.parse_known_args()
    return args

def main():
    os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
    args = parse_args()
    tasks_order = []; n_map = {}
    tasks_order.append("gsm8k");   n_map["gsm8k"]   = args.n_gsm8k
    tasks_order.append("bbh_date"); n_map["bbh_date"] = args.n_bbh_date
    tasks_order.append("arc_c");   n_map["arc_c"]   = args.n_arc_c
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)

if __name__ == "__main__":
    main()



[Phase] BASELINE (VPS OFF) — building model...


TypeError: build() missing 1 required positional argument: 'cfg'

In [ ]:
# vps/scripts/eval_suite.py
# Colab/T4-friendly evaluation with progress + OOM-safe VPS pass.
from __future__ import annotations

import os, sys, time, random, argparse
from typing import Dict, List

# --- Robust path setup: works when run as a file or via stdin/heredoc ---
try:
    import pathlib
    if "__file__" in globals():
        HERE = pathlib.Path(__file__).resolve()          # .../vps/scripts/eval_suite.py
        PKG_ROOT = str(HERE.parents[1])                  # .../vps
        REPO_ROOT = str(HERE.parents[2])                 # .../
    else:
        CWD = pathlib.Path(os.getcwd()).resolve()
        if (CWD / "vpscore").exists() or (CWD / "scripts").exists():
            PKG_ROOT = str(CWD)
            REPO_ROOT = str(CWD.parent)
        elif (CWD / "vps").exists():
            PKG_ROOT = str((CWD / "vps").resolve())
            REPO_ROOT = str(CWD.resolve())
        else:
            PKG_ROOT = str(CWD)
            REPO_ROOT = str(CWD)
except Exception:
    PKG_ROOT = os.path.abspath(os.path.join(os.getcwd(), "vps"))
    REPO_ROOT = os.path.abspath(os.getcwd())

for p in (REPO_ROOT, PKG_ROOT):
    if p and p not in sys.path:
        sys.path.insert(0, p)
os.environ["PYTHONPATH"] = REPO_ROOT + ":" + PKG_ROOT + ":" + os.environ.get("PYTHONPATH", "")

import torch
from datasets import load_dataset

# Reuse your existing pieces
from vps.scripts.infer_vps import build             # creates (tok, model, hooks)
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.config import VPSConfig                # <-- NEW: import the config


# ----------------------------- helpers -----------------------------
def set_vps_enabled(model: torch.nn.Module, enabled: bool):
    """Neutralize VPS delta by setting gamma=0 for baseline; keep wrappers intact."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if hasattr(m, "cfg"):
                if not enabled:
                    m.cfg.adaptive_gamma = False
                    m.cfg.gamma = 0.0
            if hasattr(m, "policy") and m.policy is not None:
                m.policy.cfg = m.cfg


def vps_iterate_once(model, tok, hooks, prompt: str, gold: str | None, max_new_tokens: int):
    """OOM-safe VPS pass: entropy→policy only, no backward()."""
    hooks.clear_buffers()
    with torch.no_grad():
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model(**inputs, use_cache=False, return_dict=True)
        ent = compute_token_entropy(out.logits[0, -1, :])
    ent_val = float(ent) if torch.is_tensor(ent) else float(ent)
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(ent_val)
            except Exception:
                pass
    return generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                    temperature=0.0, top_p=1.0, do_sample=False)


def format_arc_prompt(q: str, choices: List[str]) -> str:
    letters = "ABCDE"
    lines = [f"Question: {q}", "Options:"]
    for i, c in enumerate(choices):
        lines.append(f"({letters[i]}) {c}")
    lines.append("Answer:")
    return "\n".join(lines)

def extract_arc_choice_letter(text: str) -> str | None:
    for ch in reversed(text.strip()):
        if ch in "ABCDE":
            return ch
    return None

def format_bbh_date_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"

def format_gsm8k_prompt(q: str) -> str:
    return f"{q.strip()}\nAnswer:"


def run_arc_c(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[arc_c] dataset size={total}; sampling n={len(idx)}")
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        prompt = format_arc_prompt(item["question"], item["choices"]["text"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, item["answerKey"], max_new_tokens); dt = time.time() - t0
        latencies.append(dt)
        if (extract_arc_choice_letter(pred) or "") == item["answerKey"].strip():
            correct += 1
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[arc_c] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"arc_c","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}

def run_bbh_date(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[bbh_date] dataset size={total}; sampling n={len(idx)}")
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        prompt = format_bbh_date_prompt(item["input"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, item["target"].strip(), max_new_tokens=24); dt = time.time() - t0
        latencies.append(dt)
        got = pred.strip().splitlines()[-1].strip()
        if got == item["target"].strip():
            correct += 1
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[bbh_date] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"bbh_date","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}

def run_gsm8k(model, tok, hooks, n: int, print_every: int, max_new_tokens: int) -> Dict:
    ds = load_dataset("openai/gsm8k", "main", split="test")
    total = len(ds)
    idx = list(range(total)); random.shuffle(idx); idx = idx[:n] if n > 0 else []
    correct = 0; latencies = []; gen_token_est = 0
    print(f"[gsm8k] dataset size={total}; sampling n={len(idx)}")
    import re
    num_re = re.compile(r"(-?\d+(?:\.\d+)?)")
    def extract_num(s: str):
        m = list(num_re.finditer(s)); return m[-1].group(1) if m else None
    for i, k in enumerate(idx, 1):
        item = ds[int(k)]
        gnum = extract_num(item["answer"])
        prompt = format_gsm8k_prompt(item["question"])
        t0 = time.time(); pred = vps_iterate_once(model, tok, hooks, prompt, gnum, max_new_tokens); dt = time.time() - t0
        latencies.append(dt)
        pnum = extract_num(pred)
        if pnum is not None and gnum is not None:
            try:
                if float(pnum) == float(gnum):
                    correct += 1
            except Exception:
                pass
        gen_token_est += len(tok.encode(pred))
        if print_every > 0 and (i % print_every == 0 or i == len(idx)):
            rem = len(idx) - i; eta_s = int(dt * rem)
            print(f"[gsm8k] {i}/{len(idx)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {eta_s//60}m{eta_s%60}s")
    acc = correct/len(idx) if idx else 0.0
    mean_lat = sum(latencies)/len(latencies) if latencies else 0.0
    tps = gen_token_est/sum(latencies) if latencies and gen_token_est>0 else 0.0
    return {"task":"gsm8k","n":len(idx),"acc":acc,"latency_mean_s":mean_lat,"tokens_per_sec":tps}


def run_phase(tasks_order: List[str], n_map: Dict[str, int], seed: int, vps_on: bool,
              max_new_tokens: int, print_every: int) -> List[Dict]:
    random.seed(seed); torch.manual_seed(seed)
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")
    cfg = VPSConfig()                          # <-- NEW
    tok, model, hooks = build(cfg)             # <-- NEW: pass cfg into build()
    if torch.cuda.is_available(): torch.cuda.set_device(0)
    set_vps_enabled(model, enabled=vps_on); model.eval()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("[Phase] model ready.")
    results: List[Dict] = []
    for task in tasks_order:
        n = int(n_map.get(task, 0))
        if n <= 0:
            print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
            continue
        print(f"\n=== {'VPS' if vps_on else 'BASELINE'} :: {task} n={n} ===")
        if task == "arc_c":
            res = run_arc_c(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        elif task == "bbh_date":
            res = run_bbh_date(model, tok, hooks, n, print_every, max_new_tokens=min(24, max_new_tokens))
        elif task == "gsm8k":
            res = run_gsm8k(model, tok, hooks, n, print_every, max_new_tokens=max_new_tokens)
        else:
            print(f"[warn] unknown task '{task}', skipping."); continue
        print(res); results.append(res)
        if torch.cuda.is_available(): torch.cuda.empty_cache()
    return results


# ----------------------------- CLI -----------------------------
def parse_args():
    ap = argparse.ArgumentParser(prog="eval_suite.py")
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=10)
    # IMPORTANT: tolerate unknown args injected by Jupyter/Colab (-f kernel.json)
    args, _unknown = ap.parse_known_args()
    return args

def main():
    os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
    args = parse_args()
    tasks_order = []; n_map = {}
    tasks_order.append("gsm8k");   n_map["gsm8k"]   = args.n_gsm8k
    tasks_order.append("bbh_date"); n_map["bbh_date"] = args.n_bbh_date
    tasks_order.append("arc_c");   n_map["arc_c"]   = args.n_arc_c
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)
    _ = run_phase(tasks_order, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every)

if __name__ == "__main__":
    main()



[Phase] BASELINE (VPS OFF) — building model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_headers.py:154: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  cached_token = get_token()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

[VPS] Wrapped layers: 252
[Phase] model ready.

=== BASELINE :: gsm8k n=0 — skipping ===

=== BASELINE :: bbh_date n=0 — skipping ===

=== BASELINE :: arc_c n=0 — skipping ===

[Phase] VPS ON — building model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[VPS] Wrapped layers: 252
[Phase] model ready.

=== VPS :: gsm8k n=0 — skipping ===

=== VPS :: bbh_date n=0 — skipping ===

=== VPS :: arc_c n=0 — skipping ===


In [ ]:
%%bash
set -e
# Make your packages importable
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
# Show logs immediately (stdout + stderr)
export PYTHONUNBUFFERED=1
export TF_CPP_MIN_LOG_LEVEL=3
export TRANSFORMERS_VERBOSITY=info

# Run Python with faulthandler and line-buffered stdio
stdbuf -oL -eL python -X faulthandler -u - <<'PY'
import os, sys, time, threading
# Ensure import paths even inside Python
for p in ["/content", "/content/vps"]:
    if p not in sys.path: sys.path.insert(0, p)

alive = True
def heartbeat():
    t = 0
    while alive:
        print(f"[hb] alive {t}s", flush=True)
        time.sleep(1)
        t += 1

hb = threading.Thread(target=heartbeat, daemon=True)
hb.start()

try:
    print("[step] quick imports...", flush=True)
    from datasets import load_dataset
    from transformers import AutoTokenizer, AutoModelForCausalLM
    print("[ok] imports ready", flush=True)

    print("[step] cache tiny dataset slice (ARC-Challenge 2 items)...", flush=True)
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation[:2]")
    print(f"[ok] dataset cached: {len(ds)} items", flush=True)

    MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
    print(f"[step] load tokenizer: {MODEL}", flush=True)
    tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
    print("[ok] tokenizer ready", flush=True)

    print("[step] load base model (this is the slowest part; HF logs will appear)...", flush=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL, device_map="auto")
    print("[ok] base model in memory", flush=True)

    print("[step] import your eval harness (run_phase)...", flush=True)
    from vps.scripts.eval_suite import run_phase
    print("[ok] eval harness ready", flush=True)

    # Tiny baseline run (prints per-item progress)
    print("\n[phase] BASELINE (no VPS) :: ARC-C n=1", flush=True)
    res_base = run_phase(["arc_c"], {"arc_c": 1}, seed=1234, vps_on=False, max_new_tokens=24, print_every=1)
    print("[result] baseline:", res_base, flush=True)

    # Tiny VPS run (prints per-item progress)
    print("\n[phase] VPS ON :: ARC-C n=1", flush=True)
    res_vps = run_phase(["arc_c"], {"arc_c": 1}, seed=1234, vps_on=True, max_new_tokens=24, print_every=1)
    print("[result] vps:", res_vps, flush=True)

except Exception as e:
    import traceback
    print("[error] exception during run:", repr(e), flush=True)
    traceback.print_exc()
finally:
    alive = False
    time.sleep(0.2)
    print("[done] diagnostic run finished.", flush=True)
PY



In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, math, random, gc
import argparse
from typing import Dict, List, Tuple

# --- make both "vpscore" and "vps.scripts" importable no matter how we launch ---
THIS_DIR = os.path.abspath(os.path.dirname(__file__))
PROJ_ROOT = os.path.abspath(os.path.join(THIS_DIR, ".."))         # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))      # .../
for p in (PROJ_ROOT, PROJ_PARENT):
    if p not in sys.path: sys.path.insert(0, p)

# Helps with allocator fragmentation on Colab T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

from vps.scripts.infer_vps import build                         # your build(cfg)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier


# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def build_cfg(args, vps_on: bool) -> VPSConfig:
    cfg = VPSConfig()
    # model / dtype
    cfg.model_name = args.model_name
    cfg.device_map = "auto"
    cfg.dtype = args.dtype
    cfg.torch_dtype_str = args.dtype
    # generation
    cfg.max_new_tokens = args.max_new_tokens
    cfg.temperature = 0.0  # deterministic for eval
    cfg.top_p = 1.0
    cfg.top_k = 0
    # VPS knobs (kept same as your defaults, but allow attn-only)
    if args.attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    # make sure qk coupling stays enabled (your default)
    cfg.qk_coupling = True
    # lbfgs is on by default; you can disable with --no_lbfgs if you add it later
    return cfg


# ------------------------- task loaders -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    """Return list of (prompt, gold_text) for ARC-Challenge validation."""
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    # Stable sample
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        # simple instruction prompt
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        gold = gold_label  # compare on letter
        items.append((prompt, gold))
    return items


# ------------------------- VPS iteration -------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    verifier = CompositeVerifier()

    # ---- Iteration 0 (plain decode, no grad) ----
    pred0 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    # Prepare inputs once
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    # ---- Lightweight policy signal: token entropy (no grad) ----
    with torch.no_grad():
        out = model(**tail_inputs)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(float(ent))

    # ---- CE surrogate on tiny tail to populate grads for VPS ----
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1,T]

    # Important for memory: no cache + grad ckpt on VPS runs
    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False

    model.zero_grad(set_to_none=True)
    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]  # [1,T,V]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # ---- VPS-enhanced decode (deterministic) ----
    pred1 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
    return pred1


# ------------------------- phases -------------------------
def set_vps_enabled(model, enabled: bool):
    """Optionally gate VPS at runtime (if you add flags later)."""
    # Your VPSLinear path is injected already; generation will use it.
    # Nothing to toggle here unless you add a runtime switch inside VPSLinear.

def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, hooks, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C grading: compare letter
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if f"{ch}" in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s")

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    # tokens_per_sec is approximate without tokenizer length; keep simple
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}


def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    set_seed(seed)
    cfg = build_cfg(argparse.Namespace(
        model_name=model_name, dtype=dtype,
        max_new_tokens=max_new_tokens, attn_only=attn_only
    ), vps_on=vps_on)

    # Build model
    tok, model, hooks = build(cfg)
    if vps_on:
        # memory saving tricks for the CE step
        try:
            model.gradient_checkpointing_enable()
        except Exception:
            pass
        set_vps_enabled(model, True)
    model.eval()
    print("[Phase] model ready.")

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, hooks, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
    finally:
        # free memory before leaving the phase
        del model; del tok
        free_cuda()

    return results


# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=64, help="number of prompt tokens used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O to save VRAM")
    args = ap.parse_args()

    tasks = []
    n_map = {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k  # (not implemented in this trimmed file)
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date  # (not implemented here)
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c

    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    # BASELINE
    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    # VPS
    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, math, random, gc
import argparse
from typing import Dict, List, Tuple

# --- make both "vpscore" and "vps.scripts" importable no matter how we launch ---
THIS_DIR = os.path.abspath(os.path.dirname(__file__))
PROJ_ROOT = os.path.abspath(os.path.join(THIS_DIR, ".."))         # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))      # .../
for p in (PROJ_ROOT, PROJ_PARENT):
    if p not in sys.path: sys.path.insert(0, p)

# Helps with allocator fragmentation on Colab T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

from vps.scripts.infer_vps import build                         # your build(cfg)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier


# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def build_cfg(args, vps_on: bool) -> VPSConfig:
    cfg = VPSConfig()
    # model / dtype
    cfg.model_name = args.model_name
    cfg.device_map = "auto"
    cfg.dtype = args.dtype
    cfg.torch_dtype_str = args.dtype
    # generation
    cfg.max_new_tokens = args.max_new_tokens
    cfg.temperature = 0.0  # deterministic for eval
    cfg.top_p = 1.0
    cfg.top_k = 0
    # VPS knobs (kept same as your defaults, but allow attn-only)
    if args.attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    # make sure qk coupling stays enabled (your default)
    cfg.qk_coupling = True
    # lbfgs is on by default; you can disable with --no_lbfgs if you add it later
    return cfg


# ------------------------- task loaders -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    """Return list of (prompt, gold_text) for ARC-Challenge validation."""
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    # Stable sample
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        # simple instruction prompt
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        gold = gold_label  # compare on letter
        items.append((prompt, gold))
    return items


# ------------------------- VPS iteration -------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    verifier = CompositeVerifier()

    # ---- Iteration 0 (plain decode, no grad) ----
    pred0 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    # Prepare inputs once
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    # ---- Lightweight policy signal: token entropy (no grad) ----
    with torch.no_grad():
        out = model(**tail_inputs)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(float(ent))

    # ---- CE surrogate on tiny tail to populate grads for VPS ----
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1,T]

    # Important for memory: no cache + grad ckpt on VPS runs
    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False

    model.zero_grad(set_to_none=True)
    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]  # [1,T,V]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # ---- VPS-enhanced decode (deterministic) ----
    pred1 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
    return pred1


# ------------------------- phases -------------------------
def set_vps_enabled(model, enabled: bool):
    """Optionally gate VPS at runtime (if you add flags later)."""
    # Your VPSLinear path is injected already; generation will use it.
    # Nothing to toggle here unless you add a runtime switch inside VPSLinear.

def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, hooks, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C grading: compare letter
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if f"{ch}" in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s")

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    # tokens_per_sec is approximate without tokenizer length; keep simple
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}


def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    set_seed(seed)
    cfg = build_cfg(argparse.Namespace(
        model_name=model_name, dtype=dtype,
        max_new_tokens=max_new_tokens, attn_only=attn_only
    ), vps_on=vps_on)

    # Build model
    tok, model, hooks = build(cfg)
    if vps_on:
        # memory saving tricks for the CE step
        try:
            model.gradient_checkpointing_enable()
        except Exception:
            pass
        set_vps_enabled(model, True)
    model.eval()
    print("[Phase] model ready.")

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, hooks, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
    finally:
        # free memory before leaving the phase
        del model; del tok
        free_cuda()

    return results


# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=64, help="number of prompt tokens used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O to save VRAM")
    args = ap.parse_args()

    tasks = []
    n_map = {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k  # (not implemented in this trimmed file)
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date  # (not implemented here)
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c

    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    # BASELINE
    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    # VPS
    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, math, random, gc, argparse
from typing import Dict, List, Tuple

# -------- make both "vpscore" and "vps.scripts" importable everywhere --------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))  # script mode
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())                # notebook/REPL
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))       # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))      # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if os.path.isdir(cand) and cand not in sys.path:
        sys.path.insert(0, cand)

# allocator hygiene on Colab T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# reuse your build() and utilities
from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier


# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def build_cfg(args, vps_on: bool) -> VPSConfig:
    cfg = VPSConfig()
    # model / dtype
    cfg.model_name = args.model_name
    cfg.device_map = "auto"
    cfg.dtype = args.dtype
    cfg.torch_dtype_str = args.dtype
    # generation
    cfg.max_new_tokens = args.max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    # patch selection
    if args.attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    cfg.qk_coupling = True
    return cfg


# ------------------------- task loaders -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items


# ------------------------- VPS iteration -------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter-0 (deterministic)
    _ = CompositeVerifier()  # kept for parity; not used here directly
    pred0 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(float(ent))

    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1,T]

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # Iter-1 (deterministic with VPS deltas active)
    pred1 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
    return pred1


# ------------------------- phases -------------------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, hooks, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C: extract letter
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s")

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...")

    set_seed(seed)
    cfg = build_cfg(argparse.Namespace(
        model_name=model_name, dtype=dtype,
        max_new_tokens=max_new_tokens, attn_only=attn_only
    ), vps_on=vps_on)

    tok, model, hooks = build(cfg)
    if vps_on:
        try: model.gradient_checkpointing_enable()
        except Exception: pass
    model.eval()
    print("[Phase] model ready.")

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, hooks, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
    finally:
        del model; del tok
        free_cuda()

    return results


# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=64, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    args = ap.parse_args()

    tasks = []
    n_map = {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not enabled in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not enabled in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# ---------- make both "vpscore" and "vps.scripts" importable (script or notebook) ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))    # script mode
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())                  # notebook/REPL
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))  # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, "..")) # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if os.path.isdir(cand) and cand not in sys.path:
        sys.path.insert(0, cand)

# CUDA allocator hygiene on Colab T4 (helps OOM/fragmentation)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# Reuse your project’s build + utils
from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy
from vpscore.verifiers.composite_verifier import CompositeVerifier


# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]  # attention only (OOM safer)
    cfg.qk_coupling = True
    return cfg


# ------------------------- task loader(s) -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items


# ------------------------- VPS iteration (1 step) -------------------------
def vps_iterate_once(model, tok, hooks, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter-0 (deterministic) — not used for scoring here, but keeps your flow
    _ = CompositeVerifier()
    _pred0 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        logits = out.logits
        ent = compute_token_entropy(logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            m.policy.set_token_entropy(float(ent))

    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1,T]

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # Iter-1 (deterministic with VPS)
    pred1 = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
    return pred1


# ------------------------- run one task -------------------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, hooks, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===")
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, hooks, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C scoring: check for option letter
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}


# ------------------------- whole phase -------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)

    # tiny mem wins for VPS phase
    if vps_on:
        try: model.gradient_checkpointing_enable()
        except Exception: pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, hooks, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
    finally:
        del model; del tok
        free_cuda()

    return results


# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=64, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    # IMPORTANT: ignore unknown args (e.g., Colab’s `-f <kernel.json>`)
    args, _unknown = ap.parse_known_args()

    tasks = []
    n_map = {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not enabled in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not enabled in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_arc_c 5 \
  --max_new_tokens 24 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --attn_only \
  --tail_tokens 32


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# ---------- Make imports work whether launched as a script or inside a notebook ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))    # script mode
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())                  # notebook/REPL
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator hygiene on Colab T4 (helps fragmentation/OOM)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# Reuse your project constructs
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """
    Baseline toggle: when disabled, set gamma=0 so VPS adds no delta.
    Keeps wrapping intact (so code paths stay identical).
    """
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            if enabled:
                m.cfg.gamma = float(getattr(m, "_saved_gamma", 0.5))
            else:
                m.cfg.gamma = 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # your build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]  # attention only (lower memory)
    cfg.qk_coupling = True
    return cfg

# ------------------------- loader(s) -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# ------------------------- VPS iteration (1 step) -------------------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter-0 (deterministic) — keeps your original flow; not used for scoring
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE on the last few gold tokens to populate grads → VPS uses grad_h internally
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1, T]

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # VPS-enhanced decode (deterministic)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# ------------------------- run one task -------------------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C scoring: best-effort letter extraction
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

# ------------------------- whole phase -------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)

    # Toggle VPS effect for 'baseline'
    set_vps_enabled(model, enabled=vps_on)

    # small memory wins
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    # NOTE: parse_known_args so Colab's "-f <kernel.json>" won't crash
    args, _unknown = ap.parse_known_args()

    tasks = []
    n_map = {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not enabled in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not enabled in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_arc_c 5 \
  --max_new_tokens 24 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --attn_only \
  --tail_tokens 32


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# ---------- Make imports work whether launched as a script or inside a notebook ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))    # script mode
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())                  # notebook/REPL
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator hygiene (reduces OOM on Colab T4)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# Reuse your project constructs
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Baseline toggle: when disabled, set gamma=0 so VPS adds no delta."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(getattr(m._saved_gamma, "__float__", lambda: m._saved_gamma)()) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]  # attention only (lower memory)
    cfg.qk_coupling = True
    return cfg

# ------------------------- loader(s) -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# ------------------------- VPS iteration (1 step) -------------------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter-0 (deterministic) — keeps your original flow; not used for scoring
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE on the last few gold tokens to populate grads → VPS uses grad_h internally
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1, T]

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # VPS-enhanced decode (deterministic)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# ------------------------- run one task -------------------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C scoring: best-effort letter extraction
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

# ------------------------- whole phase -------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)

    # Toggle VPS effect for 'baseline'
    set_vps_enabled(model, enabled=vps_on)

    # small memory wins
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    # parse_known_args → ignores Colab’s "-f <kernel.json>"
    args, _unknown = ap.parse_known_args()

    tasks = []
    n_map = {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not enabled in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not enabled in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_arc_c 5 \
  --max_new_tokens 24 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --attn_only \
  --tail_tokens 32


In [ ]:
# vps/scripts/eval_suite.py
from __future__ import annotations
import os, sys, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# ---------- Make imports work in both script and notebook ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))    # script mode
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())                  # notebook/REPL
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator hygiene (reduces OOM on Colab T4)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# Reuse your project constructs
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ------------------------- small helpers -------------------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_inputs_to_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(0)==1 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Baseline toggle: when disabled, set gamma=0 so VPS adds no delta."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]  # attention only (lower memory)
    cfg.qk_coupling = True
    return cfg

# ------------------------- loader(s) -------------------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc","ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# ------------------------- VPS iteration (1 step) -------------------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter-0 (deterministic) — keeps your original flow; not used for scoring
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_inputs_to_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE on the last few gold tokens to populate grads → VPS uses grad_h internally
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)  # [1, T]

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # VPS-enhanced decode (deterministic)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# ------------------------- run one task -------------------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        if vps_on:
            pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens)
        else:
            pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # ARC-C scoring: best-effort letter extraction
        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

# ------------------------- whole phase -------------------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)

    # Toggle VPS effect for 'baseline'
    set_vps_enabled(model, enabled=vps_on)

    # small memory wins
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# ------------------------- CLI -------------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    # parse_known_args → ignores Colab’s "-f <kernel.json>"
    args, _unknown = ap.parse_known_args()

    tasks = []
    n_map = {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not enabled in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not enabled in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()


In [ ]:
%%bash
set -e
# Show the first ~140 lines (should contain model_name/dtype/attn_only/tail_tokens)
sed -n '1,140p' vps/scripts/eval_suite.py | nl | sed -n '1,140p'
echo "---- grep flags ----"
grep -nE 'model_name|dtype|attn_only|tail_tokens|parse_known_args' vps/scripts/eval_suite.py || true


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_arc_c 5 \
  --max_new_tokens 24 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --attn_only \
  --tail_tokens 32


In [ ]:
%%bash
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# --- Make imports work in Colab or as a script ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator = fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# --- Reuse your project pieces ---
from vps.scripts.infer_vps import build            # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- helpers ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """For baseline: keep VPS modules present but turn their effect off."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # your build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention only
    cfg.qk_coupling = True
    return cfg

# --------------- data (ARC-C only in this lite file) ---------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# --------------- one VPS iteration (safe on T4) ---------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter 0 (greedy) to get context; ignore output
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE on last few gold tokens to populate grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

    # VPS-enhanced decode (greedy)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# --------------- run one task ---------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            # graceful fallback: free, shrink tail, try once more (or baseline)
            free_cuda()
            if vps_on and tail_tokens > 8:
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens // 2)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        got = None
        for ch in ["A","B","C","D","E","a","b","c","d","e"]:
            if ch in pred:
                got = ch.upper(); break
        is_ok = (got == gold.upper())
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

# --------------- whole phase ---------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)
    set_vps_enabled(model, enabled=vps_on)
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# --------------- CLI ---------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   print("[warn] gsm8k loader not in this lite file — set n_gsm8k=0")
    if args.n_bbh_date:print("[warn] bbh_date loader not in this lite file — set n_bbh_date=0")
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_arc_c 5 \
  --max_new_tokens 24 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --attn_only \
  --tail_tokens 32


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_arc_c 50 \
  --n_gsm8k 0 \
  --n_bbh_date 0 \
  --max_new_tokens 24 \
  --print_every 5


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_bbh_date 40 \
  --n_arc_c 0 \
  --n_gsm8k 0 \
  --max_new_tokens 16 \
  --print_every 5


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_bbh_date 40 \
  --n_gsm8k 0 \
  --n_arc_c 0 \
  --max_new_tokens 16 \
  --print_every 5


In [ ]:
%%bash
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse, re, datetime as dt
from typing import Dict, List, Tuple

# --- Make imports work in Colab or as a script ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator = fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# --- Reuse your project pieces ---
from vps.scripts.infer_vps import build            # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- helpers ----------------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """For baseline: keep VPS modules present but turn their effect off."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # your build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention only
    cfg.qk_coupling = True
    return cfg

# --------------- data loaders ---------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    # ARC has "test" split for Challenge; some mirrors provide "validation" too.
    try:
        ds = load_dataset("ai2_arc", "ARC-Challenge", split="test")
    except Exception:
        ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the following multiple-choice science question.\n"
            "Respond with the BEST option letter only (A, B, C, D, or E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append(("arc_c", prompt, str(gold_label).upper()))
    return items

def load_gsm8k(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    # Use "main" config; fall back to train if test not present.
    try:
        ds = load_dataset("openai/gsm8k", "main", split="test")
    except Exception:
        ds = load_dataset("openai/gsm8k", "main", split="train")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"] if "question" in ex else ex.get("question_text", "")
        # gold: extract final number after "####"
        ans = ex["answer"]
        m = re.search(r"####\s*(.+)", ans)
        gold = (m.group(1) if m else ans).strip()
        prompt = (
            "Solve the problem. Return ONLY the final number, no words.\n\n"
            f"Problem: {q}\nAnswer:"
        )
        items.append(("gsm8k", prompt, gold))
    return items

def _bbh_try_load():
    tried = []
    for name, subset, split in [
        ("lukaemon/bbh", "date_understanding", "test"),
        ("google/bigbench-hard", "date_understanding", "test"),
        ("maveriq/bbh", "date_understanding", "test"),
    ]:
        try:
            ds = load_dataset(name, subset, split=split)
            return ds, f"{name}/{subset}:{split}"
        except Exception as e:
            tried.append(f"{name}/{subset}:{split} -> {type(e).__name__}")
    return None, tried

def _weekday_name(d: dt.date):
    return ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"][d.weekday()]

def _synthetic_bbh_date(n, seed=1234):
    rng = random.Random(seed)
    rows = []
    for _ in range(n):
        y = rng.randint(2001, 2030)
        m = rng.randint(1, 12)
        # choose a valid day for the month
        for _try in range(5):
            d = rng.randint(1, 31)
            try:
                base = dt.date(y, m, d); break
            except: continue
        k = rng.choice([-400,-90,-31,-30,-7,-1,1,2,3,7,30,31,90,400])
        target = base + dt.timedelta(days=k)
        if rng.random() < 0.6:
            prompt = f"If the date is {base.isoformat()}, what day of the week is it {k} days later? Answer only the weekday."
            answer = _weekday_name(target)
        else:
            prompt = f"Starting from {base.strftime('%B %d, %Y')}, what is the exact date {k} days later? Answer as YYYY-MM-DD."
            answer = target.isoformat()
        rows.append({"input": prompt, "target": answer})
    from datasets import Dataset
    return Dataset.from_list(rows)

def load_bbh_date(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds, msg = _bbh_try_load()
    if ds is None:
        print(f"[warn] could not fetch BBH Date from HF ({msg}); using synthetic {n} items.", flush=True)
        ds = _synthetic_bbh_date(n, seed)
    else:
        ds = ds.shuffle(seed=seed).select(range(min(n, len(ds))))
    items = []
    for ex in ds:
        prompt = (
            "Answer the question. Return ONLY the final weekday name or ISO date (YYYY-MM-DD).\n\n"
            f"{ex['input']}\nAnswer:"
        )
        gold = str(ex.get("target", ex.get("label", ""))).strip()
        items.append(("bbh_date", prompt, gold))
    return items

# --------------- parsing + correctness ---------------
def _extract_last_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return m[-1].group(1) if m else None

def _extract_choice_letter(s: str):
    s = (s or "").strip()
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    # fallback: first letter in text beginning with A-E:
    m = re.findall(r"\b([A-E])[.)\s]", s.upper())
    return m[0] if m else None

def _normalize(s: str):
    return (s or "").strip().lower()

def is_correct(task: str, pred: str, gold: str) -> bool:
    if task == "arc_c":
        got = _extract_choice_letter(pred)
        return (got == gold.upper())
    if task == "gsm8k":
        pn = _extract_last_number(pred)
        gn = _extract_last_number(gold)
        try:
            return pn is not None and gn is not None and abs(float(pn) - float(gn)) < 1e-6
        except:
            return False
    if task == "bbh_date":
        return _normalize(pred) == _normalize(gold)
    return False

# --------------- one VPS iteration (safe on T4) ---------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int) -> str:
    # Iter 0 (greedy) to warm caches; ignore output
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                # ensure plain float (some policies expect scalar)
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE on last few gold tokens to populate grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    try:
        out2 = model(**tail_inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()
    finally:
        model.config.use_cache = old_cache

    # VPS-enhanced decode (greedy)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# --------------- run one task ---------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (_, prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            # graceful fallback: free, shrink tail, try once more (or baseline)
            free_cuda()
            if vps_on and tail_tokens > 8:
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens // 2)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        ok = is_correct(task, pred, gold)
        correct += int(ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    lat = sum(times)/n
    result = {"task": task, "n": n, "acc": correct / n, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}
    return result

# --------------- whole phase ---------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)
    set_vps_enabled(model, enabled=vps_on)
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            elif task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# --------------- CLI ---------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only)

if __name__ == "__main__":
    main()
PY



In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTHONUNBUFFERED=1
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True

python -u vps/scripts/eval_suite.py \
  --n_bbh_date 5 \
  --n_gsm8k 5 \
  --max_new_tokens 12 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --tail_tokens 8 \
  --attn_only \
  --seed 1234


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTHONUNBUFFERED=1
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 25 \
  --n_bbh_date 0 \
  --max_new_tokens 16 \
  --print_every 5 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --tail_tokens 8 \
  --attn_only \
  --seed 1234


In [ ]:
%%bash
mkdir -p vps/scripts
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, re, json, time, random, gc, argparse
from typing import Dict, List, Tuple

# ---------- Make imports work both in Colab and as a script ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# ---------- Friendly defaults for Colab/T4 ----------
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

# ---------- Reuse your project pieces ----------
from vps.scripts.infer_vps import build            # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- helpers ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Keep VPS modules present but turn their effect off for baseline."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            if enabled:
                m.cfg.gamma = float(getattr(m, "_saved_gamma", 0.5))
                m.cfg.adaptive_gamma = True
            else:
                m.cfg.gamma = 0.0
                m.cfg.adaptive_gamma = False

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # your build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention only
    cfg.qk_coupling = True
    return cfg

# --------------- tiny parsers/scorers ---------------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _normalize(s: str) -> str:
    return (s or "").strip().lower()

def _extract_choice_letter(s: str):
    s = (s or "").strip()
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    return None

# --------------- prompt formatters ---------------
def format_prompt_gsm8k(x):
    q = x["question"] if "question" in x else x.get("question_text", "")
    # Allow brief reasoning but enforce a clean final line
    return (
        "Solve the math problem. You may use a brief scratchpad, "
        "but end with a single line 'Final: <number>'.\n\n"
        f"Problem: {q}\n\nAnswer:"
    )

def format_prompt_arc_c(x):
    q = x["question"]; labels = x["choices"]["label"]; texts = x["choices"]["text"]
    opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, texts)])
    return (
        "You are given a multiple-choice science question. "
        "Return ONLY the correct option letter (A, B, C, D, or E).\n\n"
        f"{q}\n\n{opts}\n\nAnswer:"
    )

def format_prompt_bbh_date(x):
    return (
        "Answer the task correctly. Respond with the exact expected output string.\n\n"
        f"{x['input']}\nAnswer:"
    )

# --------------- dataset loaders ---------------
def load_arc_c(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        items.append((format_prompt_arc_c(ex), str(ex["answerKey"]).strip().upper()))
    return items

def load_gsm8k(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("gsm8k", "main", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        # gold is after "#### "
        ans = ex["answer"]
        m = re.search(r"####\s*(.+)", ans)
        gold = m.group(1).strip() if m else ans.strip()
        items.append((format_prompt_gsm8k(ex), gold))
    return items

def load_bbh_date(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    # BBH community mirror
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        items.append((format_prompt_bbh_date(ex), str(ex["target"]).strip()))
    return items

# --------------- one VPS iteration (T4-safe, supports >1 iters) ---------------
def vps_iterate(model, tok, prompt: str, gold_text: str,
                max_new_tokens: int, tail_tokens: int, iters:int = 1) -> str:
    pred = ""
    for _ in range(max(1, iters)):
        # warm-up forward on tail to get entropy
        base_inputs = tok(prompt, return_tensors="pt").to(model.device)
        tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

        with torch.enable_grad():
            out = model(**tail_inputs)
            ent = compute_token_entropy(out.logits[0, -1, :])

        # pass entropy to policies if present
        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                try:
                    m.policy.set_token_entropy(float(ent))
                except Exception:
                    pass

        # short CE on last few gold tokens to populate grads
        gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
        T = min(8, gold_ids.shape[0])
        target = gold_ids[-T:].unsqueeze(0)

        old_cache = getattr(model.config, "use_cache", True)
        model.config.use_cache = False
        model.zero_grad(set_to_none=True)
        out2 = model(**tail_inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()
        model.config.use_cache = old_cache

        # VPS-enhanced decode (greedy)
        pred = generate(model, tok, prompt, max_new_tokens,
                        temperature=0.0, top_p=1.0, do_sample=False)
    return pred

# --------------- run one task ---------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int, vps_iters:int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                pred = vps_iterate(model, tok, prompt, gold, max_new_tokens, tail_tokens, iters=vps_iters)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            # graceful fallback: free, shrink tail, try once more (or baseline)
            free_cuda()
            if vps_on and tail_tokens > 8:
                pred = vps_iterate(model, tok, prompt, gold, max_new_tokens, tail_tokens // 2, iters=max(1, vps_iters//2))
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        # scoring
        if task == "gsm8k":
            pn, gn = _extract_number(pred), _extract_number(gold)
            is_ok = (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
        elif task == "bbh_date":
            is_ok = (_normalize(pred) == _normalize(gold))
        elif task == "arc_c":
            got = _extract_choice_letter(pred)
            is_ok = (str(got).upper() == str(gold).upper())
        else:
            is_ok = False
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    lat = sum(times)/n
    result = {
        "task": task, "n": n,
        "acc": correct / n,
        "latency_mean_s": lat,
        "tokens_per_sec": max(1e-9, 1.0/lat)
    }
    return result

# --------------- whole phase ---------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool, vps_iters:int) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens, attn_only=attn_only)
    tok, model, hooks = build(cfg)   # your build(cfg)
    set_vps_enabled(model, enabled=vps_on)
    try:
        if vps_on:
            model.gradient_checkpointing_enable()  # helps T4 OOM
    except Exception:
        pass
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "arc_c":
                items = load_arc_c(n, seed)
            elif task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            else:
                items = []

            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens, vps_iters)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# --------------- CLI ---------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=48)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=16, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    ap.add_argument("--vps_iters", type=int, default=1, help="number of VPS refinement iters per sample")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only, vps_iters=args.vps_iters)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only, vps_iters=args.vps_iters)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
export HF_HUB_ENABLE_PROGRESS_BARS=1
export TRANSFORMERS_VERBOSITY=info
export HF_HUB_DISABLE_TELEMETRY=1

# heartbeat
( while :; do echo "[hb] alive $(date +%T)"; sleep 1; done ) &
HB=$!

# hard timeout so it can't hang forever
timeout 300s stdbuf -oL -eL python -X faulthandler -u vps/scripts/eval_suite.py \
  --n_gsm8k 1 \
  --n_bbh_date 1 \
  --max_new_tokens 16 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --tail_tokens 8 || STATUS=$?

kill $HB || true
exit ${STATUS:-0}



In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_gsm8k 10 \
  --n_bbh_date 10 \
  --max_new_tokens 24 \
  --print_every 5 \
  --model_name Qwen/Qwen2.5-1.5B-Instruct \
  --dtype fp16 \
  --tail_tokens 12


In [ ]:
%%bash
set -euo pipefail
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
export TOKENIZERS_PARALLELISM=false

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 6 \
  --n_bbh_date 6 \
  --max_new_tokens 48 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 16 \
  --attn_only


In [ ]:
%%bash
mkdir -p vps/scripts
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse, threading
from typing import Dict, List, Tuple
# ---------------- path setup (works in Colab or as plain script) ----------------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator settings – help fragmentation on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

import torch
from datasets import load_dataset

# ----- import from your project (REAL VPS path) -----
from vps.scripts.infer_vps import build             # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- misc utils ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool, gamma: float):
    """
    Baseline: keep VPS modules present but neutralize by setting gamma=0.
    VPS ON: restore configured gamma.
    """
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                # remember the module's original gamma if present; or use CLI gamma
                g = float(getattr(getattr(m, "cfg", None), "gamma", gamma))
                m._saved_gamma = g
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              attn_only: bool, gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name       = model_name
    cfg.device_map       = "auto"
    cfg.dtype            = dtype          # your build() maps this to torch dtype
    cfg.torch_dtype_str  = dtype
    cfg.max_new_tokens   = max_new_tokens
    cfg.temperature      = 0.0
    cfg.top_p            = 1.0
    cfg.top_k            = 0
    cfg.gamma            = float(gamma)
    cfg.adaptive_gamma   = bool(adaptive_gamma)
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # target attention-only
    # Encourage Q/K coupling (often helps)
    try:
        cfg.qk_coupling = True
    except Exception:
        pass
    return cfg

# ---------------- Data loaders ----------------
def load_gsm8k(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    ds = load_dataset("gsm8k", "main", split="test")  # 1319 examples
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        ans = ex["answer"]
        # gold = number after '####'
        g = ans.split("####")[-1].strip()
        prompt = (
            "Solve the problem carefully. Return ONLY the final number.\n\n"
            f"Problem: {q}\nAnswer:"
        )
        items.append((prompt, g))
    return items

def load_bbh_date(n:int, seed:int) -> List[Tuple[str,str]]:
    """
    Try lukaemon/bbh (common mirror). Fallback prints a warning and returns [].
    """
    if n <= 0: return []
    try:
        ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    except Exception:
        print("[warn] Could not load BBH date_understanding; skipping.", flush=True)
        return []
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        inp = ex.get("inputs") or ex.get("input") or ""
        tgt = (ex.get("targets") or ex.get("target") or "").strip()
        prompt = (
            "Answer the question. Return ONLY the final answer string (no punctuation).\n\n"
            f"{inp}\nAnswer:"
        )
        items.append((prompt, tgt))
    return items

def load_arc_c(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    # 'validation' has 1172 and is fine for quick runs
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = str(ex["answerKey"]).strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the multiple-choice question. Return ONLY the correct option letter.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# ---------------- Answer checkers ----------------
def _extract_number(s: str):
    import re
    m = list(re.finditer(r"(-?\d+(?:\.\d+)?)", s or ""))
    return float(m[-1].group(1)) if m else None

def is_correct(task:str, pred:str, gold:str)->bool:
    if task == "gsm8k":
        pn, gn = _extract_number(pred), _extract_number(gold)
        return (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
    if task == "bbh_date":
        return (pred or "").strip().lower() == (gold or "").strip().lower()
    if task == "arc_c":
        import re
        m = re.findall(r"\b([A-E])\b", (pred or "").upper())
        choice = m[-1] if m else None
        return choice == str(gold).upper()
    return False

# ---------------- VPS iteration (strong but T4-safe) ----------------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     max_new_tokens: int, tail_tokens: int, iters:int=1) -> str:
    """
    1) First greedy decode to establish context (no sampling).
    2) For 'iters' times: tail-slice forward, compute entropy, set policy; short CE on last 12 gold tokens; backward.
    3) Final greedy decode with VPS influence.
    """
    # Step 1: context decode (ignored)
    _ = generate(model, tok, prompt, max_new_tokens=min(64, max_new_tokens),
                 temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    old_cache = getattr(model.config, "use_cache", True)
    try:
        for k in range(max(1, iters)):
            tail_inputs = slice_tail(base_inputs, tail=max(8, tail_tokens))
            with torch.no_grad():
                out = model(**tail_inputs)
                ent = compute_token_entropy(out.logits[0, -1, :])

            # pass token entropy to policies
            for m in model.modules():
                if hasattr(m, "policy") and m.policy is not None:
                    try:
                        m.policy.set_token_entropy(float(ent))
                    except Exception:
                        pass

            # small CE on last few gold tokens (stronger than 8, but still safe)
            gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
            T = min(12, gold_ids.shape[0])
            target = gold_ids[-T:].unsqueeze(0)

            model.config.use_cache = False
            model.zero_grad(set_to_none=True)
            out2 = model(**tail_inputs)
            logits2 = out2.logits[:, -T:, :]
            ce = torch.nn.functional.cross_entropy(
                logits2.reshape(-1, logits2.size(-1)),
                target.reshape(-1)
            )
            ce.backward()
    finally:
        model.config.use_cache = old_cache

    # Step 3: final greedy decode (VPS effect applied)
    return generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                    temperature=0.0, top_p=1.0, do_sample=False)

# ---------------- runner ----------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int, iters:int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, tail_tokens, iters=iters)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            if vps_on and tail_tokens > 8:
                # auto-fallback: shrink tail
                pred = vps_iterate_once(model, tok, prompt, gold, max_new_tokens, max(8, tail_tokens//2), iters=1)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        ok = is_correct(task, pred, gold)
        correct += int(ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool, gamma: float, adaptive_gamma: bool,
              iters:int) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens,
                    attn_only=attn_only, gamma=gamma, adaptive_gamma=adaptive_gamma)
    tok, model, hooks = build(cfg)
    # Baseline neutralizes VPS by setting gamma=0 at modules; VPS ON restores
    set_vps_enabled(model, enabled=vps_on, gamma=gamma)
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            elif task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens, iters)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()
    return results

# ---------------- heartbeat (always prints) ----------------
def start_heartbeat():
    stop = {"flag": False}
    def _hb():
        while not stop["flag"]:
            print(f"[hb] alive {time.strftime('%H:%M:%S')}", flush=True)
            time.sleep(1.0)
    th = threading.Thread(target=_hb, daemon=True)
    th.start()
    return stop

# ---------------- CLI ----------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=48)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=16, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    ap.add_argument("--gamma", type=float, default=0.35, help="VPS strength (higher = stronger effect)")
    ap.add_argument("--adaptive_gamma", action="store_true", help="scale gamma with token entropy")
    ap.add_argument("--iters", type=int, default=1, help="number of VPS CE iterations (>=1); keep small on T4")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"]   = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"]= args.n_bbh_date
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"]   = args.n_arc_c
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    hb = start_heartbeat()

    try:
        _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                      max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                      model_name=args.model_name, dtype=args.dtype,
                      tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                      gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                      iters=1)  # baseline ignores VPS CE anyway

        _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                      max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                      model_name=args.model_name, dtype=args.dtype,
                      tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                      gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                      iters=args.iters)

    finally:
        hb["flag"] = True

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1
python -u vps/scripts/eval_suite.py \
  --n_gsm8k 8 \
  --n_bbh_date 8 \
  --max_new_tokens 48 \
  --print_every 2 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 16 \
  --attn_only \
  --gamma 0.35 \
  --adaptive_gamma \
  --iters 1


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 12 \
  --n_bbh_date 12 \
  --max_new_tokens 48 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.55 \
  --adaptive_gamma \
  --iters 2


In [ ]:
%%bash
mkdir -p vps/scripts
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse, re, threading, datetime
from typing import Dict, List, Tuple

# ---------- Make imports work in Colab or as a script ----------
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))   # .../vps
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))  # .../
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# T4-friendly CUDA allocator & quiet logs
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("PYTHONUNBUFFERED", "1")

import torch
from datasets import load_dataset

# ---- Your project pieces ----
from vps.scripts.infer_vps import build            # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---- optional date parser (for BBH-Date normalization) ----
try:
    from dateutil import parser as _dateparse
    HAVE_DATEUTIL = True
except Exception:
    HAVE_DATEUTIL = False

# ---------------- utilities ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Keep VPS modules present but turn their effect off for baseline."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              attn_only: bool, gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype              # your build() maps this to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention only
    # VPS knobs:
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    cfg.qk_coupling = True
    return cfg

# --------------- data loaders ---------------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _norm_date(s: str) -> str:
    s = (s or "").strip()
    if HAVE_DATEUTIL:
        try:
            dt = _dateparse.parse(s, fuzzy=True, dayfirst=False)
            return dt.strftime("%Y-%m-%d")  # canonical YYYY-MM-DD
        except Exception:
            pass
    return s.lower()

def load_gsm8k(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    try:
        ds = load_dataset("openai/gsm8k", "main", split="test")
    except Exception:
        ds = load_dataset("gsm8k", "main", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q   = ex.get("question", ex.get("question_text", ""))
        ans = ex["answer"]
        m = re.search(r"####\s*(.+)", ans)
        gold = m.group(1).strip() if m else ans.strip()
        prompt = (
            "Solve the following math word problem. Return ONLY the final number.\n\n"
            f"Problem: {q}\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_bbh_date(n:int, seed:int) -> List[Tuple[str, str]]:
    if n <= 0: return []
    items = []
    # Try common BBH sources; fall back to a tiny built-in set if needed
    ds = None
    for spec in [("lukaemon/bbh","date_understanding"), ("tasksource/bbh","date_understanding")]:
        try:
            ds = load_dataset(spec[0], spec[1], split="test")
            break
        except Exception:
            continue
    if ds is None:
        # fallback: a few deterministic samples
        samples = [
            ("If today is March 1, 2020, what date is 10 days later? Answer succinctly.", "2020-03-11"),
            ("The day after 2021-12-31 is?", "2022-01-01"),
            ("Two weeks before July 4, 2022 is?", "2022-06-20"),
            ("What date is 3 days after 14 Feb 2021?", "2021-02-17"),
            ("What date is yesterday if today is 1/1/2023?", "2022-12-31"),
        ]
        random.Random(seed).shuffle(samples)
        return samples[:n]
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    for i in idx:
        ex = ds[i]
        inp = ex.get("input", ex.get("question", ""))
        tgt = ex.get("target", ex.get("answer", ""))
        prompt = "Answer correctly. Return ONLY the final date.\n\n" + str(inp) + "\nAnswer:"
        items.append((prompt, str(tgt)))
    return items

def score(task: str, pred: str, gold: str) -> bool:
    if task == "gsm8k":
        pn, gn = _extract_number(pred), _extract_number(gold)
        return (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
    if task == "bbh_date":
        return _norm_date(pred) == _norm_date(gold)
    if task == "arc_c":
        # not used by default in this file, but keep for completeness
        def _letter(s):
            m = re.findall(r"\b([A-E])\b", s.upper())
            if m: return m[-1]
            m = re.findall(r"\(([A-E])\)", s.upper())
            return m[-1] if m else None
        return _letter(pred) == str(gold).upper()
    return False

# --------------- one or more VPS iterations ---------------
def vps_iterate(model, tok, prompt: str, gold_text: str,
                max_new_tokens: int, tail_tokens: int, iters: int) -> str:
    # Warmup decode (no sample) to stabilize entropy/policies
    _ = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    for k in range(max(1, iters)):
        # 1) entropy signal
        with torch.no_grad():
            out = model(**tail_inputs)
            ent = compute_token_entropy(out.logits[0, -1, :])

        for m in model.modules():
            if hasattr(m, "policy") and m.policy is not None:
                try:
                    m.policy.set_token_entropy(float(ent))
                except Exception:
                    pass

        # 2) short CE to populate grads
        gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
        T = min(8, gold_ids.shape[0])
        target = gold_ids[-T:].unsqueeze(0)

        old_cache = getattr(model.config, "use_cache", True)
        model.config.use_cache = False
        model.zero_grad(set_to_none=True)
        out2 = model(**tail_inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()
        model.config.use_cache = old_cache

    # Final VPS-enhanced decode (greedy)
    return generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)

# --------------- runner ---------------
def run_task(task: str, items: List[Tuple[str,str]], tok, model,
             print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int, iters: int) -> Dict:
    if len(items) == 0:
        print(f"=== {'VPS' if vps_on else 'BASELINE'} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                pred = vps_iterate(model, tok, prompt, gold, max_new_tokens, tail_tokens, iters)
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            # fallback strategy: shrink tail or drop to baseline
            if vps_on and tail_tokens > 8:
                pred = vps_iterate(model, tok, prompt, gold, max_new_tokens, max(8, tail_tokens//2), max(1, iters//2))
            else:
                pred = generate(model, tok, prompt, max_new_tokens, temperature=0.0, top_p=1.0, do_sample=False)
        dt = time.time() - t0
        times.append(dt)

        is_ok = score(task, pred, gold)
        correct += int(is_ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times)) * (len(items)-i)
            print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    acc = correct / n
    lat = sum(times)/n
    return {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}

def heartbeat(stop_evt: threading.Event):
    while not stop_evt.is_set():
        now = datetime.datetime.now().strftime("%H:%M:%S")
        print(f"[hb] alive {now}", flush=True)
        stop_evt.wait(1.0)

def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool, gamma: float, adaptive_gamma: bool,
              iters: int) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens,
                    attn_only=attn_only, gamma=gamma, adaptive_gamma=adaptive_gamma)

    tok, model, hooks = build(cfg)
    # Turn effect off for baseline:
    set_vps_enabled(model, enabled=vps_on)

    # ---- GAMMA DIAGNOSTIC PRINT (what you asked for) ----
    any_gamma = None
    for n, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma
            break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)
    # -----------------------------------------------------

    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass
    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens, vps_on, tail_tokens, iters)
            results.append(res)
            print(json.dumps(res), flush=True)
    finally:
        del model; del tok
        free_cuda()

    return results

# --------------- CLI ---------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)  # not used by default here
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--attn_only", action="store_true")
    ap.add_argument("--gamma", type=float, default=0.55)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--iters", type=int, default=2, help="number of VPS CE steps")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c:   print("[warn] arc_c loader not in this file — set n_arc_c=0")
    if not tasks:
        print("No tasks requested; nothing to run."); return

    # heartbeat thread so you always see progress
    stop_evt = threading.Event()
    t = threading.Thread(target=heartbeat, args=(stop_evt,), daemon=True)
    t.start()
    try:
        _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                      max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                      model_name=args.model_name, dtype=args.dtype,
                      tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                      gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=max(1, args.iters//2))

        _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                      max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                      model_name=args.model_name, dtype=args.dtype,
                      tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                      gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=args.iters)
    finally:
        stop_evt.set()
        t.join(timeout=0.1)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 6 \
  --n_bbh_date 6 \
  --max_new_tokens 48 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.55 \
  --adaptive_gamma \
  --iters 2


In [ ]:
%%bash
mkdir -p vps/scripts
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse, re
from typing import Dict, List, Tuple

# ----- Colab / script import hygiene -----
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# Keep TF/XLA chatty logs down and make CUDA alloc robust on T4
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")

import torch
from datasets import load_dataset

# ---- Your project pieces ----
from vps.scripts.infer_vps import build            # build(cfg) -> (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- utils ----------------
def set_seed(seed:int):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Toggle VPS effect without removing wrappers (gamma=0 for baseline)."""
    for m in model.modules():
        if getattr(m, "__class__", None) and m.__class__.__name__ == "VPSLinear":
            # remember original gamma the first time we see it
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              attn_only: bool, gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype               # your build() maps to torch dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # limit to attention
    return cfg

# -------------- scoring helpers --------------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _normalize(s: str):
    return (s or "").strip().lower()

def _extract_choice_letter(s: str):
    s = (s or "").strip()
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    return None

# ----- BBH date normalization -----
def _to_iso_date(text: str) -> str|None:
    if not text: return None
    t = text.strip()
    # direct ISO match
    m = re.search(r"\b(\d{4})-(\d{2})-(\d{2})\b", t)
    if m:
        y,mn,dd = m.groups()
        return f"{int(y):04d}-{int(mn):02d}-{int(dd):02d}"
    # try dateutil if present
    try:
        from dateutil import parser as du
        dt = du.parse(t, dayfirst=False, yearfirst=True, fuzzy=True)
        return f"{dt.year:04d}-{dt.month:02d}-{dt.day:02d}"
    except Exception:
        pass
    # very light fallback like "March 5, 2020"
    m2 = re.search(r"\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\.?\s+(\d{1,2}),?\s+(\d{4})", t, re.I)
    if m2:
        mon_map = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"sept":9,"oct":10,"nov":11,"dec":12}
        mon = mon_map[m2.group(1).lower()[:3]]
        d   = int(m2.group(2)); y = int(m2.group(3))
        return f"{y:04d}-{mon:02d}-{d:02d}"
    return None

# -------------- datasets --------------
def load_gsm8k(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("openai/gsm8k", "main", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"] if "question" in ex else ex["question_text"]
        ans = ex["answer"]
        m = re.search(r"####\s*(.+)", ans)
        gold = (m.group(1).strip() if m else ans.strip())
        prompt = (
            "Solve the problem step by step, but return ONLY the final number.\n\n"
            f"Problem: {q}\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_bbh_date(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        # bbh date has fields "input" and "target"
        inp = ex["input"]
        gold = str(ex["target"]).strip()
        gold_iso = _to_iso_date(gold) or gold
        prompt = (
            "You are given a date reasoning problem.\n"
            "Return ONLY the final date in the exact format YYYY-MM-DD.\n\n"
            f"{inp}\n\nAnswer:"
        )
        items.append((prompt, gold_iso))
    return items

def load_arc_c(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Answer the multiple-choice science question.\n"
            "Return ONLY the option letter (A, B, C, D, or E).\n\n"
            f"{q}\n\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# -------------- one VPS step --------------
def vps_step(model, tok, inputs, gold_text: str, tail_tokens: int):
    """One light CE backprop on last few gold tokens to drive VPS."""
    tail_inputs = slice_tail(inputs, tail=tail_tokens)

    # entropy for policy
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except Exception: pass

    # short CE surrogate to create grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**tail_inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()
    model.config.use_cache = old_cache

def _score(task:str, pred:str, gold:str) -> bool:
    if task == "gsm8k":
        pn, gn = _extract_number(pred), _extract_number(gold)
        return (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
    if task == "bbh_date":
        p_iso = _to_iso_date(pred)
        g_iso = _to_iso_date(gold) or gold
        return (p_iso == g_iso)
    if task == "arc_c":
        got = _extract_choice_letter(pred)
        return (got == str(gold).upper())
    return False

# -------------- run one task --------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int, iters:int) -> Dict:
    if len(items) == 0:
        print(f"=== {('VPS' if vps_on else 'BASELINE')} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times: List[float] = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            if vps_on:
                # optional warm decode to anchor context (no-op for scoring)
                _ = generate(model, tok, prompt, max_new_tokens=min(24, max_new_tokens),
                             temperature=0.0, top_p=1.0, do_sample=False)
                for _ in range(max(1, iters)):
                    vps_step(model, tok, inputs, gold, tail_tokens)
            # final decode (greedy)
            pred = generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                            temperature=0.0, top_p=1.0, do_sample=False)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            # shrink tail and retry once if VPS, else vanilla
            if vps_on and tail_tokens > 8:
                try:
                    vps_step(model, tok, inputs, gold, max(8, tail_tokens//2))
                except Exception:
                    pass
            pred = generate(model, tok, prompt, max_new_tokens=max_new_tokens,
                            temperature=0.0, top_p=1.0, do_sample=False)

        dt = time.time() - t0
        times.append(dt)
        ok = _score(task, pred, gold)
        correct += int(ok)

        eta = (sum(times)/len(times)) * (len(items)-i)
        print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    lat = sum(times)/n
    res = {"task": task, "n": n, "acc": correct/n, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}
    return res

# -------------- whole phase --------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool, gamma: float, adaptive_gamma: bool,
              iters:int) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens,
                    attn_only=attn_only, gamma=gamma, adaptive_gamma=adaptive_gamma)
    tok, model, hooks = build(cfg)

    # Toggle VPS effect
    set_vps_enabled(model, enabled=vps_on)

    # gamma diagnostic (first VPSLinear)
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma
            break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            elif task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens,
                           vps_on, tail_tokens, iters)
            print(json.dumps(res), flush=True)
            results.append(res)
    finally:
        del model; del tok
        free_cuda()

    return results

# -------------- CLI --------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=24, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    ap.add_argument("--gamma", type=float, default=0.6, help="VPS gamma (ignored in baseline)")
    ap.add_argument("--adaptive_gamma", action="store_true", help="enable adaptive gamma")
    ap.add_argument("--iters", type=int, default=1, help="count of VPS CE steps per item")
    args, _unknown = ap.parse_known_args()  # tolerate Colab's -f kernel.json

    tasks, n_map = [], {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if not tasks:
        print("No tasks requested; nothing to run.")
        return

    # BASELINE
    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                  gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=0)

    # VPS ON
    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                  gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=max(1,args.iters))

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
pip -q install python-dateutil


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

# install normalizer just in case
pip -q install python-dateutil >/dev/null

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 8 \
  --n_bbh_date 8 \
  --max_new_tokens 16 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 12 \
  --iters 1 \
  --gamma 0.7 \
  --adaptive_gamma \
  --attn_only


In [ ]:
%%bash
mkdir -p vps/scripts
cat > vps/scripts/eval_suite.py << 'PY'
from __future__ import annotations
import os, sys, json, time, random, gc, argparse, re
from typing import Dict, List, Tuple

# ---- Colab / script import hygiene
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {PROJ_PARENT, PROJ_ROOT, "/content", "/content/vps"}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")

import torch
from datasets import load_dataset

# ---- Your project pieces
from vps.scripts.infer_vps import build            # build(cfg) -> (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

# ---------------- utils ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Toggle VPS effect without removing wrappers (gamma=0 for baseline)."""
    for m in model.modules():
        if getattr(m, "__class__", None) and m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              attn_only: bool, gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    return cfg

# -------------- decode ONLY new tokens (fixes scoring errors) --------------
def greedy_new_text(model, tok, prompt: str, max_new_tokens: int) -> str:
    ins = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **ins, max_new_tokens=max_new_tokens,
        do_sample=False, return_dict_in_generate=True
    )
    seq = out.sequences[0]
    inp_len = ins["input_ids"].shape[1]
    new_ids = seq[inp_len:]
    return tok.decode(new_ids, skip_special_tokens=True)

# -------------- scoring helpers --------------
NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_number(s: str):
    m = list(NUM_RE.finditer(s or ""))
    return float(m[-1].group(1)) if m else None

def _normalize(s: str):
    return (s or "").strip().lower()

def _extract_choice_letter(s: str):
    s = (s or "").strip()
    m = re.findall(r"\b([A-E])\b", s.upper())
    if m: return m[-1]
    m = re.findall(r"\(([A-E])\)", s.upper())
    if m: return m[-1]
    return None

# ----- BBH date normalization -----
def _to_iso_date(text: str) -> str|None:
    if not text: return None
    t = text.strip()

    # Prefer the LAST ISO-looking date in the generated delta
    iso_all = list(re.finditer(r"(\d{4})-(\d{2})-(\d{2})", t))
    if iso_all:
        y,mn,dd = iso_all[-1].groups()
        return f"{int(y):04d}-{int(mn):02d}-{int(dd):02d}"

    # try python-dateutil if present
    try:
        from dateutil import parser as du
        dt = du.parse(t, dayfirst=False, yearfirst=True, fuzzy=True)
        return f"{dt.year:04d}-{dt.month:02d}-{dt.day:02d}"
    except Exception:
        pass

    # light fallback like "March 5, 2020"
    m2 = re.search(r"\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\.?\s+(\d{1,2}),?\s+(\d{4})", t, re.I)
    if m2:
        mon_map = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"sept":9,"oct":10,"nov":11,"dec":12}
        mon = mon_map[m2.group(1).lower()[:3]]
        d   = int(m2.group(2)); y = int(m2.group(3))
        return f"{y:04d}-{mon:02d}-{d:02d}"
    return None

# -------------- datasets --------------
def load_gsm8k(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("openai/gsm8k", "main", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex.get("question", ex.get("question_text", ""))
        ans = ex["answer"]
        m = re.search(r"####\s*(.+)", ans)
        gold = (m.group(1).strip() if m else ans.strip())
        prompt = (
            "Solve the problem. Return ONLY the final number with no words.\n\n"
            f"Problem: {q}\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_bbh_date(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("lukaemon/bbh", "date_understanding", split="test")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        inp = ex["input"]
        gold = str(ex["target"]).strip()
        gold_iso = _to_iso_date(gold) or gold
        prompt = (
            "You are given a date reasoning problem.\n"
            "Return ONLY the final date in EXACT format YYYY-MM-DD (no words).\n\n"
            f"{inp}\n\nAnswer:"
        )
        items.append((prompt, gold_iso))
    return items

def load_arc_c(n:int, seed:int) -> List[Tuple[str,str]]:
    if n <= 0: return []
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold_label = ex["answerKey"]
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "Answer the multiple-choice science question.\n"
            "Return ONLY the option letter (A, B, C, D, or E).\n\n"
            f"{q}\n\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold_label))
    return items

# -------------- one VPS step --------------
def vps_step(model, tok, inputs, gold_text: str, tail_tokens: int):
    tail_inputs = slice_tail(inputs, tail=tail_tokens)

    # entropy for policy
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])

    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except Exception: pass

    # short CE surrogate
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(8, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**tail_inputs)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            target.reshape(-1)
        )
        ce.backward()
    model.config.use_cache = old_cache

def _score(task:str, pred_delta:str, gold:str) -> bool:
    if task == "gsm8k":
        pn, gn = _extract_number(pred_delta), _extract_number(gold)
        return (pn is not None and gn is not None and abs(pn - gn) < 1e-6)
    if task == "bbh_date":
        p_iso = _to_iso_date(pred_delta)
        g_iso = _to_iso_date(gold) or gold
        return (p_iso == g_iso)
    if task == "arc_c":
        got = _extract_choice_letter(pred_delta)
        return (got == str(gold).upper())
    return False

# -------------- run one task --------------
def run_task(task: str, items: List[Tuple[str,str]],
             tok, model, print_every: int, max_new_tokens: int, vps_on: bool,
             tail_tokens: int, iters:int) -> Dict:
    if len(items) == 0:
        print(f"=== {('VPS' if vps_on else 'BASELINE')} :: {task} n=0 — skipping ===", flush=True)
        return {"task": task, "n": 0, "acc": 0.0, "latency_mean_s": 0.0, "tokens_per_sec": 0.0}

    correct = 0
    times: List[float] = []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            if vps_on:
                # quick warm decode (no scoring)
                _ = greedy_new_text(model, tok, prompt, max_new_tokens=min(24, max_new_tokens))
                for _ in range(max(1, iters)):
                    vps_step(model, tok, inputs, gold, tail_tokens)
            # final decode (delta only)
            pred_delta = greedy_new_text(model, tok, prompt, max_new_tokens=max_new_tokens)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            if vps_on and tail_tokens > 8:
                try:
                    vps_step(model, tok, inputs, gold, max(8, tail_tokens//2))
                except Exception:
                    pass
            pred_delta = greedy_new_text(model, tok, prompt, max_new_tokens=max_new_tokens)

        dt = time.time() - t0
        times.append(dt)
        ok = _score(task, pred_delta, gold)
        correct += int(ok)

        eta = (sum(times)/len(times)) * (len(items)-i)
        print(f"[{task}] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    n = len(items)
    lat = sum(times)/n
    res = {"task": task, "n": n, "acc": correct/n, "latency_mean_s": lat, "tokens_per_sec": max(1e-9, 1.0/lat)}
    return res

# -------------- whole phase --------------
def run_phase(tasks_order: List[str], n_map: Dict[str,int], seed: int,
              vps_on: bool, max_new_tokens: int, print_every: int, model_name: str,
              dtype: str, tail_tokens: int, attn_only: bool, gamma: float, adaptive_gamma: bool,
              iters:int) -> List[Dict]:
    phase_name = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase_name} — building model...", flush=True)

    set_seed(seed)
    cfg = build_cfg(model_name=model_name, dtype=dtype, max_new_tokens=max_new_tokens,
                    attn_only=attn_only, gamma=gamma, adaptive_gamma=adaptive_gamma)
    tok, model, hooks = build(cfg)

    # Toggle VPS
    set_vps_enabled(model, enabled=vps_on)

    # gamma diagnostic
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    model.eval()
    print("[Phase] model ready.", flush=True)

    results = []
    try:
        for task in tasks_order:
            n = int(n_map.get(task, 0))
            if task == "gsm8k":
                items = load_gsm8k(n, seed)
            elif task == "bbh_date":
                items = load_bbh_date(n, seed)
            elif task == "arc_c":
                items = load_arc_c(n, seed)
            else:
                items = []
            res = run_task(task, items, tok, model, print_every, max_new_tokens,
                           vps_on, tail_tokens, iters)
            print(json.dumps(res), flush=True)
            results.append(res)
    finally:
        del model; del tok
        free_cuda()

    return results

# -------------- CLI --------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n_gsm8k", type=int, default=0)
    ap.add_argument("--n_bbh_date", type=int, default=0)
    ap.add_argument("--n_arc_c", type=int, default=0)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--max_new_tokens", type=int, default=48)
    ap.add_argument("--print_every", type=int, default=5)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=24, help="prompt tail used for CE/backprop (VPS only)")
    ap.add_argument("--attn_only", action="store_true", help="apply VPS only to attention Q/K/V/O")
    ap.add_argument("--gamma", type=float, default=0.6, help="VPS gamma (ignored in baseline)")
    ap.add_argument("--adaptive_gamma", action="store_true", help="enable adaptive gamma")
    ap.add_argument("--iters", type=int, default=1, help="VPS CE steps per item")
    args, _unknown = ap.parse_known_args()

    tasks, n_map = [], {}
    if args.n_gsm8k:   tasks.append("gsm8k");     n_map["gsm8k"] = args.n_gsm8k
    if args.n_bbh_date:tasks.append("bbh_date");  n_map["bbh_date"] = args.n_bbh_date
    if args.n_arc_c:   tasks.append("arc_c");     n_map["arc_c"] = args.n_arc_c
    if not tasks:
        print("No tasks requested; nothing to run."); return

    _ = run_phase(tasks, n_map, args.seed, vps_on=False,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                  gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=0)

    _ = run_phase(tasks, n_map, args.seed, vps_on=True,
                  max_new_tokens=args.max_new_tokens, print_every=args.print_every,
                  model_name=args.model_name, dtype=args.dtype,
                  tail_tokens=args.tail_tokens, attn_only=args.attn_only,
                  gamma=args.gamma, adaptive_gamma=args.adaptive_gamma, iters=max(1,args.iters))

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_gsm8k 8 \
  --n_bbh_date 8 \
  --max_new_tokens 64 \
  --print_every 1 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.6 \
  --adaptive_gamma \
  --iters 2


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_arc_c 20 \
  --max_new_tokens 32 \
  --print_every 5 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.6 \
  --adaptive_gamma \
  --iters 2


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/eval_suite.py \
  --n_arc_c 80 \
  --max_new_tokens 24 \
  --print_every 5 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.7 \
  --adaptive_gamma \
  --iters 2 \
  --attn_only


In [ ]:
%%bash
set -e
mkdir -p vps/scripts
cat > vps/scripts/arc_mc_eval.py << 'PY'
from __future__ import annotations
import os, sys, re, random, time, json, argparse, gc
from typing import List, Tuple, Dict

# --- Make imports work no matter how this is launched ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")

import torch
from datasets import load_dataset

# --- Your project pieces ---
from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate
from vpscore.math_utils import compute_token_entropy

LETTER_RE = re.compile(r"Answer\s*[:\-]?\s*([A-E])\b", re.IGNORECASE)

def set_seed(seed:int):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool,
              gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention-only VPS
    cfg.qk_coupling = True
    return cfg

def load_arc_items(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def extract_letter(s: str) -> str | None:
    if not s: return None
    m = LETTER_RE.search(s)
    if m: return m.group(1).upper()
    # Fallback: last 10 chars
    tail = s.strip().upper()[-10:]
    m2 = re.search(r"\b([A-E])\b", tail)
    return m2.group(1) if m2 else None

def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     tail_tokens: int, ce_T: int = 6) -> None:
    """
    One VPS pass that populates grads using a tiny CE on the tail.
    No return: it just conditions internal VPS modules via gradients.
    """
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    # entropy probe (no grad)
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # tiny CE to populate grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(ce_T, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)),
        target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

def run_phase(n:int, seed:int, vps_on:bool, model_name:str, dtype:str,
              tail_tokens:int, gamma:float, adaptive_gamma:bool,
              iters:int, attn_only:bool, print_every:int) -> Dict:
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, max_new_tokens=1, attn_only=attn_only,
                    gamma=gamma, adaptive_gamma=adaptive_gamma)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    # baseline: disable effect but keep modules present
    set_vps_enabled(model, enabled=vps_on)
    # show a gamma sample
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    model.eval()
    print("[Phase] model ready.", flush=True)

    items = load_arc_items(n, seed=seed)

    correct, times = 0, []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                # small number of VPS iterations
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=6)
            # final decode: 1 token greedy (must be the letter)
            out_ids = tok(prompt, return_tensors="pt").to(model.device)
            gen = model.generate(**out_ids, max_new_tokens=1, do_sample=False)
            pred_text = tok.decode(gen[0], skip_special_tokens=True)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            # fallback: zero VPS and decode
            out_ids = tok(prompt, return_tensors="pt").to(model.device)
            gen = model.generate(**out_ids, max_new_tokens=1, do_sample=False)
            pred_text = tok.decode(gen[0], skip_special_tokens=True)

        dt = time.time() - t0
        times.append(dt)

        got = extract_letter(pred_text)
        ok = (got == gold.upper())
        correct += int(ok)

        if (i % max(1, print_every) == 0) or (i == n):
            eta = (sum(times)/len(times))*(n-i)
            print(f"[arc_c] {i}/{n} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = correct / n if n else 0.0
    lat = sum(times)/max(1,len(times))
    res = {"task":"arc_c", "n": n, "acc": acc, "latency_mean_s": lat}
    print(json.dumps(res), flush=True)

    del model, tok
    free_cuda()
    return res

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n", type=int, default=40)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--attn_only", action="store_true")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    # Baseline
    _ = run_phase(n=args.n, seed=args.seed, vps_on=False, model_name=args.model_name,
                  dtype=args.dtype, tail_tokens=args.tail_tokens, gamma=args.gamma,
                  adaptive_gamma=args.adaptive_gamma, iters=args.iters,
                  attn_only=args.attn_only, print_every=args.print_every)

    # VPS
    _ = run_phase(n=args.n, seed=args.seed, vps_on=True, model_name=args.model_name,
                  dtype=args.dtype, tail_tokens=args.tail_tokens, gamma=args.gamma,
                  adaptive_gamma=args.adaptive_gamma, iters=args.iters,
                  attn_only=args.attn_only, print_every=args.print_every)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/arc_mc_eval.py \
  --n 40 \
  --seed 1234 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 24 \
  --gamma 0.65 \
  --adaptive_gamma \
  --iters 2 \
  --attn_only \
  --print_every 5


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/arc_mc_eval.py \
  --n 80 \
  --seed 1234 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 32 \
  --gamma 0.70 \
  --adaptive_gamma \
  --iters 3 \
  --attn_only \
  --print_every 5


In [ ]:
from __future__ import annotations
import os, sys, re, random, time, json, argparse, gc, math
from typing import List, Tuple, Dict

# --- robust imports (Colab/script) ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# less noise + fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")

import torch
from datasets import load_dataset

# --- your project pieces ---
from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate  # kept for --score gen option
from vpscore.math_utils import compute_token_entropy

LETTER_RE = re.compile(r"\b([A-E])\b")

def set_seed(seed:int):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """For baseline: keep VPS modules present but turn their effect off."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int, attn_only: bool,
              gamma: float, adaptive_gamma: bool) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    if attn_only:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]   # attention-only
    cfg.qk_coupling = True
    return cfg

def load_arc_items(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds))); random.Random(seed).shuffle(idx); idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output ONE LETTER only (A/B/C/D/E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

# ---------- robust, generation-free scoring ----------
def _candidate_ids(tok, letter:str):
    """Return the plausible single next-token ids for letter, with/without leading space."""
    ids_bare = tok(letter, add_special_tokens=False)["input_ids"]
    ids_sp   = tok(" " + letter, add_special_tokens=False)["input_ids"]
    cand = []
    if len(ids_bare) >= 1: cand.append(ids_bare[0])
    if len(ids_sp)   >= 1: cand.append(ids_sp[0])
    # de-dup
    return list(dict.fromkeys(cand))

@torch.no_grad()
def predict_by_logits(model, tok, prompt:str) -> str | None:
    # Ensure the next-token position is right after "Answer:"
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model(**inputs)
    logits = out.logits[0, -1, :]  # next-token logits

    # candidate id sets per letter
    letters = ["A","B","C","D","E"]
    scores = {}
    for L in letters:
        cand_ids = _candidate_ids(tok, L)
        if not cand_ids:
            scores[L] = float("-inf")
            continue
        # use maximum logit among variants to be robust to spacing/BPE
        scores[L] = float(torch.max(logits[cand_ids]).item())
    # argmax
    best = max(letters, key=lambda L: scores[L])
    # if all -inf, return None
    return best if math.isfinite(scores[best]) else None

def extract_letter_from_gen(text:str) -> str | None:
    # parse last 10 chars fallback
    tail = (text or "").strip()[-10:].upper()
    m = LETTER_RE.search(tail)
    return m.group(1) if m else None

# ---------- one VPS conditioning pass ----------
def vps_iterate_once(model, tok, prompt: str, gold_text: str,
                     tail_tokens: int, ce_T: int = 6) -> None:
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    # entropy probe (no grad)
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except Exception: pass

    # tiny CE to populate grads
    gold_ids = tok(gold_text, return_tensors="pt").to(model.device)["input_ids"][0]
    T = min(ce_T, gold_ids.shape[0])
    target = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)

    out2 = model(**tail_inputs)
    logits2 = out2.logits[:, -T:, :]
    ce = torch.nn.functional.cross_entropy(
        logits2.reshape(-1, logits2.size(-1)), target.reshape(-1)
    )
    ce.backward()
    model.config.use_cache = old_cache

# ---------- run both phases on the SAME items ----------
def run_phase(items:List[Tuple[str,str]], vps_on:bool, model_name:str, dtype:str,
              tail_tokens:int, gamma:float, adaptive_gamma:bool,
              iters:int, attn_only:bool, print_every:int,
              score_mode:str) -> Dict:
    cfg = build_cfg(model_name, dtype, max_new_tokens=1, attn_only=attn_only,
                    gamma=gamma, adaptive_gamma=adaptive_gamma)
    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    # (de)activate VPS effect
    set_vps_enabled(model, enabled=vps_on)
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)
    model.eval()
    print("[Phase] model ready.", flush=True)

    correct, times, preds = 0, [], []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=6)

            if score_mode == "logit":
                got = predict_by_logits(model, tok, prompt)
            else:
                # legacy: 1-token greedy then parse
                out_ids = tok(prompt, return_tensors="pt").to(model.device)
                gen = model.generate(**out_ids, max_new_tokens=1, do_sample=False)
                text = tok.decode(gen[0], skip_special_tokens=True)
                got = extract_letter_from_gen(text)

        except torch.cuda.OutOfMemoryError:
            free_cuda()
            got = None

        dt = time.time() - t0
        times.append(dt)
        preds.append(got)
        ok = (got == gold.upper())
        correct += int(ok)

        if (i % max(1, print_every) == 0) or (i == len(items)):
            eta = (sum(times)/len(times))*(len(items)-i)
            print(f"[arc_c] {i}/{len(items)} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = correct / max(1,len(items))
    lat = sum(times)/max(1,len(times))
    res = {"task":"arc_c", "n": len(items), "acc": acc, "latency_mean_s": lat, "preds": preds}
    print(json.dumps({k:v for k,v in res.items() if k!='preds'}), flush=True)

    del model, tok
    free_cuda()
    return res

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--tail_tokens", type=int, default=32)
    ap.add_argument("--gamma", type=float, default=0.70)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--iters", type=int, default=3)
    ap.add_argument("--attn_only", action="store_true")
    ap.add_argument("--print_every", type=int, default=10)
    ap.add_argument("--score", type=str, choices=["logit","gen"], default="logit",
                    help="logit = next-token argmax over letters; gen = 1-token greedy then parse")
    args, _ = ap.parse_known_args()

    set_seed(args.seed)
    items = load_arc_items(args.n, args.seed)

    base = run_phase(items, vps_on=False, model_name=args.model_name, dtype=args.dtype,
                     tail_tokens=args.tail_tokens, gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                     iters=args.iters, attn_only=args.attn_only, print_every=args.print_every,
                     score_mode=args.score)

    vps  = run_phase(items, vps_on=True, model_name=args.model_name, dtype=args.dtype,
                     tail_tokens=args.tail_tokens, gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                     iters=args.iters, attn_only=args.attn_only, print_every=args.print_every,
                     score_mode=args.score)

    # paired flips summary
    b = c = 0
    for (_, gold), pb, pv in zip(items, base["preds"], vps["preds"]):
        ok_b = (pb == gold)
        ok_v = (pv == gold)
        if (not ok_b) and ok_v: b += 1
        if ok_b and (not ok_v): c += 1
    # McNemar (with continuity correction)
    mcnemar = ((abs(b - c) - 1)**2 / max(1, (b + c))) if (b + c) > 0 else 0.0
    print(f"[paired] flips: baseline→VPS improvements b={b}, regressions c={c}, McNemar≈{mcnemar:.2f}", flush=True)

if __name__ == "__main__":
    main()


In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/arc_mc_eval.py \
  --n 80 \
  --seed 1234 \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --dtype fp16 \
  --tail_tokens 32 \
  --gamma 0.70 \
  --adaptive_gamma \
  --iters 3 \
  --attn_only \
  --score logit \
  --print_every 10


In [ ]:
from __future__ import annotations
import os, sys, re, random, time, json, argparse, gc, math
from typing import List, Tuple, Dict

# --- Make imports work in Colab or as a script ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator = fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
import torch.nn.functional as F
from datasets import load_dataset

# --- Your project pieces (FULL VPS) ---
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate     # not used for scoring, but kept for parity
from vpscore.math_utils import compute_token_entropy

LETTERS = ["A","B","C","D","E"]

# ---------------- utils ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Baseline: keep VPS modules present but disable their effect (gamma=0)."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              gamma: float, adaptive_gamma: bool,
              apply_to: str) -> VPSConfig:
    """
    apply_to: 'full' (default) or 'attn' for attention-only.
    """
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    cfg.qk_coupling = True
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    # else: full (no restriction)
    return cfg

# --------------- datasets ---------------
def load_arc(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]  # ['A','B','C','D','E'] subset
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_csqa(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("commonsense_qa", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = [c["text"] for c in ex["choices"]]
        labels  = [c["label"].upper() for c in ex["choices"]]  # A..E
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

# --------------- log-prob MC scoring ---------------
def _letter_token_ids(tok) -> Dict[str, List[int]]:
    """
    For robustness across BPEs, allow both 'A' and ' A' (space-prefixed) etc.
    Return a dict: letter -> [candidate_token_ids...]
    """
    out = {}
    for L in LETTERS:
        cand = set()
        for s in (L, " "+L):
            ids = tok.encode(s, add_special_tokens=False)
            if len(ids)>=1:
                # use only the FIRST token (position 0) as the first-step choice
                cand.add(ids[0])
        out[L] = sorted(cand)
    return out

@torch.no_grad()
def score_letters_logprob(model, tok, prompt_ids, letter_id_map:Dict[str,List[int]]) -> Dict[str,float]:
    """
    Returns log-prob for each letter at the *first next token* after prompt.
    """
    out = model(**prompt_ids)
    logits = out.logits[:, -1, :]           # (1, vocab)
    logp   = F.log_softmax(logits, dim=-1)  # (1, vocab)
    scores = {}
    for L, ids in letter_id_map.items():
        if len(ids)==0:
            scores[L] = -1e9
        elif len(ids)==1:
            scores[L] = float(logp[0, ids[0]].item())
        else:
            # logsumexp over multiple tokenizations (e.g., 'A' vs ' A')
            vals = torch.stack([logp[0, j] for j in ids])
            scores[L] = float(torch.logsumexp(vals, dim=0).item())
    return scores

# --------------- VPS iteration (aligned CE on gold) ---------------
def vps_iterate_once(model, tok, prompt: str, gold_letter: str,
                     tail_tokens: int, ce_T: int = 1) -> None:
    """
    Full-power VPS pass:
      - entropy probe on tail
      - tiny CE on the *gold letter* to populate grads (aligned / teacher-forced)
    """
    # Base prompt tensors
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    # 1) entropy probe (no grad)
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # 2) CE on gold letter (teacher-forced, aligned)
    gold_ids_full = tok(" "+gold_letter, add_special_tokens=False, return_tensors=None)
    if len(gold_ids_full)==0:
        gold_ids_full = tok(gold_letter, add_special_tokens=False, return_tensors=None)
    gold_ids_full = torch.tensor(gold_ids_full, dtype=torch.long, device=base_inputs["input_ids"].device)
    T = max(1, min(ce_T, gold_ids_full.shape[0]))

    # Input for CE must include prompt + gold[:-1], target = gold
    aug = torch.cat([base_inputs["input_ids"][0], gold_ids_full[:T-1]], dim=0) if T>1 else base_inputs["input_ids"][0]
    aug = aug.unsqueeze(0)
    attn = torch.ones_like(aug, device=aug.device)

    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    out2 = model(input_ids=aug, attention_mask=attn)
    logits2 = out2.logits[:, -T:, :]             # predict the gold segment
    target  = gold_ids_full[:T].unsqueeze(0)
    ce = F.cross_entropy(logits2.reshape(-1, logits2.size(-1)), target.reshape(-1))
    ce.backward()
    model.config.use_cache = True

# --------------- run one task ---------------
def run_task(task:str, n:int, seed:int, vps_on:bool,
             model_name:str, dtype:str, gamma:float, adaptive_gamma:bool,
             tail_tokens:int, iters:int, apply_to:str, print_every:int) -> Dict:
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, max_new_tokens=1,
                    gamma=gamma, adaptive_gamma=adaptive_gamma, apply_to=apply_to)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    # Baseline keeps VPS modules but disables effect
    set_vps_enabled(model, enabled=vps_on)

    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    # show a gamma sample
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    model.eval()
    print("[Phase] model ready.", flush=True)

    if task == "arc_c":
        items = load_arc(n, seed)
    elif task == "csqa":
        items = load_csqa(n, seed)
    else:
        raise ValueError(f"Unknown task {task}")

    letter_id_map = _letter_token_ids(tok)

    preds, golds, times = [], [], []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=1)
            # Score letters by log-prob *without* appending the gold
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        dt = time.time() - t0

        preds.append(pred); golds.append(gold)
        times.append(dt)

        if (i % max(1, print_every) == 0) or (i == n):
            acc_so_far = sum(int(p==g) for p,g in zip(preds,golds)) / len(preds)
            eta = (sum(times)/len(times))*(n-i)
            print(f"[{task}] {i}/{n} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = sum(int(p==g) for p,g in zip(preds,golds)) / max(1,len(golds))
    lat = sum(times)/max(1,len(times))
    res = {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "preds": preds, "golds": golds}
    print(json.dumps({k:v for k,v in res.items() if k!='preds' and k!='golds'}), flush=True)

    # cleanup
    del model, tok
    free_cuda()
    return res

def mcnemar_from_pairs(base_preds, vps_preds, golds):
    b = c = 0
    for pb, pv, g in zip(base_preds, vps_preds, golds):
        if pb != g and pv == g: b += 1       # improved
        if pb == g and pv != g: c += 1       # regressed
    # continuity-corrected chi^2 (McNemar)
    if (b+c)==0:
        stat = 0.0; p = 1.0
    else:
        stat = (abs(b - c) - 1.0)**2 / (b + c)
        # approximate p-value with survival function of chi2(1)
        # since we avoid scipy, use simple exp approximation
        # p ~ exp(-stat/2) * 1.2533 / sqrt(stat+1e-9)  (rough)
        p = math.exp(-stat/2.0)
    return b, c, stat, p

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", type=str, choices=["arc_c", "csqa"], default="arc_c")
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","fp32"], default="fp16")  # T4: fp16
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--apply_to", type=str, choices=["full","attn"], default="full")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    # Baseline
    base = run_task(task=args.task, n=args.n, seed=args.seed, vps_on=False,
                    model_name=args.model_name, dtype=args.dtype,
                    gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                    tail_tokens=args.tail_tokens, iters=args.iters,
                    apply_to=args.apply_to, print_every=args.print_every)

    # VPS
    vps = run_task(task=args.task, n=args.n, seed=args.seed, vps_on=True,
                   model_name=args.model_name, dtype=args.dtype,
                   gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                   tail_tokens=args.tail_tokens, iters=args.iters,
                   apply_to=args.apply_to, print_every=args.print_every)

    # Paired stats
    b, c, stat, p = mcnemar_from_pairs(base["preds"], vps["preds"], base["golds"])
    delta = vps["acc"] - base["acc"]
    summary = {
        "task": args.task,
        "n": args.n,
        "baseline_acc": base["acc"],
        "vps_acc": vps["acc"],
        "delta_acc": delta,
        "improved_pairs_b": b,
        "regressed_pairs_c": c,
        "mcnemar_chi2": stat,
        "mcnemar_p_approx": p,
        "baseline_latency_mean_s": base["latency_mean_s"],
        "vps_latency_mean_s": vps["latency_mean_s"],
    }
    print("[paired]", json.dumps(summary), flush=True)

if __name__ == "__main__":
    main()



[Phase] BASELINE (VPS OFF) — building model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[Phase] model ready.


README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

[arc_c] 5/80 acc_so_far=0.600 last_dt=0.38s ~ETA 0m49s
[arc_c] 10/80 acc_so_far=0.600 last_dt=0.36s ~ETA 0m36s
[arc_c] 15/80 acc_so_far=0.667 last_dt=0.39s ~ETA 0m30s
[arc_c] 20/80 acc_so_far=0.700 last_dt=0.44s ~ETA 0m27s
[arc_c] 25/80 acc_so_far=0.760 last_dt=0.37s ~ETA 0m24s
[arc_c] 30/80 acc_so_far=0.767 last_dt=0.37s ~ETA 0m21s
[arc_c] 35/80 acc_so_far=0.771 last_dt=0.39s ~ETA 0m19s
[arc_c] 40/80 acc_so_far=0.775 last_dt=0.38s ~ETA 0m16s
[arc_c] 45/80 acc_so_far=0.778 last_dt=0.37s ~ETA 0m14s
[arc_c] 50/80 acc_so_far=0.780 last_dt=0.42s ~ETA 0m12s
[arc_c] 55/80 acc_so_far=0.782 last_dt=0.39s ~ETA 0m10s
[arc_c] 60/80 acc_so_far=0.800 last_dt=0.37s ~ETA 0m8s
[arc_c] 65/80 acc_so_far=0.800 last_dt=0.37s ~ETA 0m6s
[arc_c] 70/80 acc_so_far=0.814 last_dt=0.40s ~ETA 0m4s
[arc_c] 75/80 acc_so_far=0.800 last_dt=0.39s ~ETA 0m2s
[arc_c] 80/80 acc_so_far=0.775 last_dt=0.37s ~ETA 0m0s
{"task": "arc_c", "n": 80, "acc": 0.775, "latency_mean_s": 0.402903813123703}

[Phase] VPS ON — building model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[VPS] Wrapped layers: 252
[diag] sample gamma = 0.65
[Phase] model ready.


TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
# vps/scripts/mc_logprob_eval.py
from __future__ import annotations
import os, sys, re, random, time, json, argparse, gc, math
from typing import List, Tuple, Dict

# --- Make imports work in Colab or as a script ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator = fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
import torch.nn.functional as F
from datasets import load_dataset

# --- Your project pieces (FULL VPS) ---
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate     # not used for scoring, but kept for parity
from vpscore.math_utils import compute_token_entropy

LETTERS = ["A","B","C","D","E"]

# ---------------- utils ----------------
def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    """Baseline: keep VPS modules present but disable their effect (gamma=0)."""
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              gamma: float, adaptive_gamma: bool,
              apply_to: str) -> VPSConfig:
    """
    apply_to: 'full' (default) or 'attn' for attention-only.
    """
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    cfg.qk_coupling = True
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    # else: full (no restriction)
    return cfg

# Robust encode helper: always returns List[int]
def encode_ids(tok, text: str) -> List[int]:
    try:
        ids = tok.encode(text, add_special_tokens=False)
    except Exception:
        out = tok(text, add_special_tokens=False, return_tensors=None)
        if isinstance(out, dict):
            ids = out.get("input_ids", [])
            if isinstance(ids, list) and len(ids) > 0 and isinstance(ids[0], list):
                ids = ids[0]
        else:
            ids = out
    if isinstance(ids, torch.Tensor):
        ids = ids.tolist()
    # ensure ints
    ids = [int(x) for x in ids] if isinstance(ids, list) else []
    return ids

# --------------- datasets ---------------
def load_arc(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]  # ['A','B','C','D','E'] subset
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_csqa(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("commonsense_qa", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = [c["text"] for c in ex["choices"]]
        labels  = [c["label"].upper() for c in ex["choices"]]  # A..E
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

# --------------- log-prob MC scoring ---------------
def _letter_token_ids(tok) -> Dict[str, List[int]]:
    """
    Allow both 'A' and ' A' (space-prefixed). Return letter -> [candidate_token_ids...]
    """
    out = {}
    for L in LETTERS:
        cand = set()
        for s in (L, " "+L):
            ids = encode_ids(tok, s)
            if len(ids) >= 1:
                cand.add(ids[0])  # next-token choice is the first subtoken
        out[L] = sorted(cand)
    return out

@torch.no_grad()
def score_letters_logprob(model, tok, prompt_ids, letter_id_map:Dict[str,List[int]]) -> Dict[str,float]:
    """
    Returns log-prob for each letter at the *first next token* after prompt.
    """
    out = model(**prompt_ids)
    logits = out.logits[:, -1, :]           # (1, vocab)
    logp   = F.log_softmax(logits, dim=-1)  # (1, vocab)
    scores = {}
    for L, ids in letter_id_map.items():
        if len(ids)==0:
            scores[L] = -1e9
        elif len(ids)==1:
            scores[L] = float(logp[0, ids[0]].item())
        else:
            vals = torch.stack([logp[0, j] for j in ids])
            scores[L] = float(torch.logsumexp(vals, dim=0).item())
    return scores

# --------------- VPS iteration (aligned CE on gold) ---------------
def vps_iterate_once(model, tok, prompt: str, gold_letter: str,
                     tail_tokens: int, ce_T: int = 1) -> None:
    """
    Full-power VPS pass:
      - entropy probe on tail
      - tiny CE on the *gold letter* to populate grads (aligned / teacher-forced)
    """
    gold_letter = (gold_letter or "").strip().upper()
    if len(gold_letter) == 0:
        return
    gold_letter = gold_letter[0]  # ensure single char

    # Base prompt tensors
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    # 1) entropy probe (no grad)
    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    # 2) CE on gold letter (teacher-forced, aligned)
    gold_ids_full = encode_ids(tok, " " + gold_letter)
    if len(gold_ids_full) == 0:
        gold_ids_full = encode_ids(tok, gold_letter)
    if len(gold_ids_full) == 0:
        return  # nothing to backprop

    gold_ids_full = torch.tensor(gold_ids_full, dtype=torch.long, device=base_inputs["input_ids"].device)
    T = max(1, min(ce_T, gold_ids_full.shape[0]))

    # Input for CE must include prompt + gold[:-1], target = gold
    if T > 1:
        aug = torch.cat([base_inputs["input_ids"][0], gold_ids_full[:T-1]], dim=0)
    else:
        aug = base_inputs["input_ids"][0]
    aug = aug.unsqueeze(0)
    attn = torch.ones_like(aug, device=aug.device)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    out2 = model(input_ids=aug, attention_mask=attn)
    logits2 = out2.logits[:, -T:, :]             # predict the gold segment
    target  = gold_ids_full[:T].unsqueeze(0)
    ce = F.cross_entropy(logits2.reshape(-1, logits2.size(-1)), target.reshape(-1))
    ce.backward()
    model.config.use_cache = old_cache

# --------------- run one task ---------------
def run_task(task:str, n:int, seed:int, vps_on:bool,
             model_name:str, dtype:str, gamma:float, adaptive_gamma:bool,
             tail_tokens:int, iters:int, apply_to:str, print_every:int) -> Dict:
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, max_new_tokens=1,
                    gamma=gamma, adaptive_gamma=adaptive_gamma, apply_to=apply_to)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    # Baseline keeps VPS modules but disables effect
    set_vps_enabled(model, enabled=vps_on)

    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    # show a gamma sample
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    model.eval()
    print("[Phase] model ready.", flush=True)

    if task == "arc_c":
        items = load_arc(n, seed)
    elif task == "csqa":
        items = load_csqa(n, seed)
    else:
        raise ValueError(f"Unknown task {task}")

    letter_id_map = _letter_token_ids(tok)

    preds, golds, times = [], [], []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=1)
            # Score letters by log-prob *without* appending the gold
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        dt = time.time() - t0

        preds.append(pred); golds.append(gold)
        times.append(dt)

        if (i % max(1, print_every) == 0) or (i == n):
            acc_so_far = sum(int(p==g) for p,g in zip(preds,golds)) / len(preds)
            eta = (sum(times)/len(times))*(n-i)
            print(f"[{task}] {i}/{n} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = sum(int(p==g) for p,g in zip(preds,golds)) / max(1,len(golds))
    lat = sum(times)/max(1,len(times))
    res = {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "preds": preds, "golds": golds}
    print(json.dumps({k:v for k,v in res.items() if k!='preds' and k!='golds'}), flush=True)

    # cleanup
    del model, tok
    free_cuda()
    return res

def mcnemar_from_pairs(base_preds, vps_preds, golds):
    b = c = 0
    for pb, pv, g in zip(base_preds, vps_preds, golds):
        if pb != g and pv == g: b += 1       # improved
        if pb == g and pv != g: c += 1       # regressed
    # continuity-corrected chi^2 (McNemar)
    if (b+c)==0:
        stat = 0.0; p = 1.0
    else:
        stat = (abs(b - c) - 1.0)**2 / (b + c)
        p = math.exp(-stat/2.0)  # rough approx without scipy
    return b, c, stat, p

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", type=str, choices=["arc_c", "csqa"], default="arc_c")
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","fp32"], default="fp16")  # T4: fp16
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--apply_to", type=str, choices=["full","attn"], default="full")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    # Baseline
    base = run_task(task=args.task, n=args.n, seed=args.seed, vps_on=False,
                    model_name=args.model_name, dtype=args.dtype,
                    gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                    tail_tokens=args.tail_tokens, iters=args.iters,
                    apply_to=args.apply_to, print_every=args.print_every)

    # VPS
    vps = run_task(task=args.task, n=args.n, seed=args.seed, vps_on=True,
                   model_name=args.model_name, dtype=args.dtype,
                   gamma=args.gamma, adaptive_gamma=args.adaptive_gamma,
                   tail_tokens=args.tail_tokens, iters=args.iters,
                   apply_to=args.apply_to, print_every=args.print_every)

    # Paired stats
    b, c, stat, p = mcnemar_from_pairs(base["preds"], vps["preds"], base["golds"])
    delta = vps["acc"] - base["acc"]
    summary = {
        "task": args.task,
        "n": args.n,
        "baseline_acc": base["acc"],
        "vps_acc": vps["acc"],
        "delta_acc": delta,
        "improved_pairs_b": b,
        "regressed_pairs_c": c,
        "mcnemar_chi2": stat,
        "mcnemar_p_approx": p,
        "baseline_latency_mean_s": base["latency_mean_s"],
        "vps_latency_mean_s": vps["latency_mean_s"],
    }
    print("[paired]", json.dumps(summary), flush=True)

if __name__ == "__main__":
    main()



[Phase] BASELINE (VPS OFF) — building model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[Phase] model ready.
[arc_c] 5/80 acc_so_far=0.600 last_dt=0.38s ~ETA 0m29s
[arc_c] 10/80 acc_so_far=0.600 last_dt=0.37s ~ETA 0m27s
[arc_c] 15/80 acc_so_far=0.667 last_dt=0.37s ~ETA 0m25s
[arc_c] 20/80 acc_so_far=0.700 last_dt=0.45s ~ETA 0m24s
[arc_c] 25/80 acc_so_far=0.760 last_dt=0.37s ~ETA 0m21s


KeyboardInterrupt: 

In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

# ARC-Challenge, n=80
python -u vps/scripts/mc_logprob_eval.py \
  --task arc_c --n 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \
  --apply_to full --print_every 5



python3: can't open file '/content/vps/scripts/mc_logprob_eval.py': [Errno 2] No such file or directory


CalledProcessError: Command 'b'set -e\nexport PYTHONPATH=/content:/content/vps:$PYTHONPATH\nexport PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True\nexport PYTHONUNBUFFERED=1\n\n# ARC-Challenge, n=80\npython -u vps/scripts/mc_logprob_eval.py \\\n  --task arc_c --n 80 --seed 1234 \\\n  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \\\n  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \\\n  --apply_to full --print_every 5\n\n'' returned non-zero exit status 2.

In [ ]:
%%bash
set -e

# --- write the evaluator (full, fixed) ---
mkdir -p vps/scripts
cat > vps/scripts/mc_logprob_eval.py << 'PY'
from __future__ import annotations
import os, sys, re, random, time, json, argparse, gc, math
from typing import List, Tuple, Dict

# --- Make imports work in Colab or as a script ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# CUDA allocator = fewer OOMs on T4
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
import torch.nn.functional as F
from datasets import load_dataset

# --- Your project pieces (FULL VPS) ---
from vps.scripts.infer_vps import build           # returns (tok, model, hooks)
from vpscore.config import VPSConfig
from vpscore.utils.generation import generate     # kept for parity
from vpscore.math_utils import compute_token_entropy

LETTERS = ["A","B","C","D","E"]

def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def set_vps_enabled(model, enabled: bool):
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def build_cfg(model_name: str, dtype: str, max_new_tokens: int,
              gamma: float, adaptive_gamma: bool,
              apply_to: str) -> VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.max_new_tokens = max_new_tokens
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    cfg.qk_coupling = True
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    return cfg

def encode_ids(tok, text: str) -> List[int]:
    try:
        ids = tok.encode(text, add_special_tokens=False)
    except Exception:
        out = tok(text, add_special_tokens=False, return_tensors=None)
        if isinstance(out, dict):
            ids = out.get("input_ids", [])
            if isinstance(ids, list) and len(ids) > 0 and isinstance(ids[0], list):
                ids = ids[0]
        else:
            ids = out
    if isinstance(ids, torch.Tensor):
        ids = ids.tolist()
    ids = [int(x) for x in ids] if isinstance(ids, list) else []
    return ids

def load_arc(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def load_csqa(n:int, seed:int) -> List[Tuple[str,str]]:
    ds = load_dataset("commonsense_qa", split="validation")
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    idx = idx[:n]
    items = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = [c["text"] for c in ex["choices"]]
        labels  = [c["label"].upper() for c in ex["choices"]]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. IMPORTANT: Output **one letter only** (A/B/C/D/E) and nothing else.\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        items.append((prompt, gold))
    return items

def _letter_token_ids(tok) -> Dict[str, List[int]]:
    out = {}
    for L in ["A","B","C","D","E"]:
        cand = set()
        for s in (L, " "+L):
            ids = encode_ids(tok, s)
            if len(ids) >= 1:
                cand.add(ids[0])
        out[L] = sorted(cand)
    return out

@torch.no_grad()
def score_letters_logprob(model, tok, prompt_ids, letter_id_map:Dict[str,List[int]]) -> Dict[str,float]:
    out = model(**prompt_ids)
    logits = out.logits[:, -1, :]
    logp   = F.log_softmax(logits, dim=-1)
    scores = {}
    for L, ids in letter_id_map.items():
        if len(ids)==0:
            scores[L] = -1e9
        elif len(ids)==1:
            scores[L] = float(logp[0, ids[0]].item())
        else:
            vals = torch.stack([logp[0, j] for j in ids])
            scores[L] = float(torch.logsumexp(vals, dim=0).item())
    return scores

def vps_iterate_once(model, tok, prompt: str, gold_letter: str,
                     tail_tokens: int, ce_T: int = 1) -> None:
    gold_letter = (gold_letter or "").strip().upper()
    if len(gold_letter) == 0:
        return
    gold_letter = gold_letter[0]

    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    with torch.no_grad():
        out = model(**tail_inputs)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try:
                m.policy.set_token_entropy(float(ent))
            except Exception:
                pass

    gold_ids_full = encode_ids(tok, " " + gold_letter)
    if len(gold_ids_full) == 0:
        gold_ids_full = encode_ids(tok, gold_letter)
    if len(gold_ids_full) == 0:
        return

    gold_ids_full = torch.tensor(gold_ids_full, dtype=torch.long, device=base_inputs["input_ids"].device)
    T = max(1, min(ce_T, gold_ids_full.shape[0]))

    if T > 1:
        aug = torch.cat([base_inputs["input_ids"][0], gold_ids_full[:T-1]], dim=0)
    else:
        aug = base_inputs["input_ids"][0]
    aug = aug.unsqueeze(0)
    attn = torch.ones_like(aug, device=aug.device)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    out2 = model(input_ids=aug, attention_mask=attn)
    logits2 = out2.logits[:, -T:, :]
    target  = gold_ids_full[:T].unsqueeze(0)
    ce = F.cross_entropy(logits2.reshape(-1, logits2.size(-1)), target.reshape(-1))
    ce.backward()
    model.config.use_cache = old_cache

def run_task(task:str, n:int, seed:int, vps_on:bool,
             model_name:str, dtype:str, gamma:float, adaptive_gamma:bool,
             tail_tokens:int, iters:int, apply_to:str, print_every:int) -> Dict:
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, max_new_tokens=1,
                    gamma=gamma, adaptive_gamma=adaptive_gamma, apply_to=apply_to)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    set_vps_enabled(model, enabled=vps_on)
    try:
        if vps_on:
            model.gradient_checkpointing_enable()
    except Exception:
        pass

    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[diag] sample gamma = {any_gamma}", flush=True)
    print("[Phase] model ready.", flush=True)

    if task == "arc_c":
        items = load_arc(n, seed)
    elif task == "csqa":
        items = load_csqa(n, seed)
    else:
        raise ValueError(f"Unknown task {task}")

    letter_id_map = _letter_token_ids(tok)

    preds, golds, times = [], [], []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=1)
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        dt = time.time() - t0

        preds.append(pred); golds.append(gold); times.append(dt)

        if (i % max(1, print_every) == 0) or (i == n):
            acc_so_far = sum(int(p==g) for p,g in zip(preds,golds)) / len(preds)
            eta = (sum(times)/len(times))*(n-i)
            print(f"[{task}] {i}/{n} acc_so_far={acc_so_far:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = sum(int(p==g) for p,g in zip(preds,golds)) / max(1,len(golds))
    lat = sum(times)/max(1,len(times))
    res = {"task": task, "n": n, "acc": acc, "latency_mean_s": lat, "preds": preds, "golds": golds}
    print(json.dumps({k:v for k,v in res.items() if k not in ('preds','golds')}), flush=True)

    del model, tok
    free_cuda()
    return res

def mcnemar_from_pairs(base_preds, vps_preds, golds):
    b = c = 0
    for pb, pv, g in zip(base_preds, vps_preds, golds):
        if pb != g and pv == g: b += 1
        if pb == g and pv != g: c += 1
    if (b+c)==0:
        stat = 0.0; p = 1.0
    else:
        stat = (abs(b - c) - 1.0)**2 / (b + c)
        p = math.exp(-stat/2.0)
    return b, c, stat, p

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", type=str, choices=["arc_c", "csqa"], default="arc_c")
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","fp32"], default="fp16")
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--apply_to", type=str, choices=["full","attn"], default="full")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    base = run_task(args.task, args.n, args.seed, False,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)
    vps  = run_task(args.task, args.n, args.seed, True,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)
    b, c, stat, p = mcnemar_from_pairs(base["preds"], vps["preds"], base["golds"])
    summary = {
        "task": args.task,
        "n": args.n,
        "baseline_acc": base["acc"],
        "vps_acc": vps["acc"],
        "delta_acc": vps["acc"] - base["acc"],
        "improved_pairs_b": b,
        "regressed_pairs_c": c,
        "mcnemar_chi2": stat,
        "mcnemar_p_approx": p,
        "baseline_latency_mean_s": base["latency_mean_s"],
        "vps_latency_mean_s": vps["latency_mean_s"],
    }
    print("[paired]", json.dumps(summary), flush=True)

if __name__ == "__main__":
    main()
PY

In [ ]:
%%bash
set -e
export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/mc_logprob_eval.py \
  --task arc_c --n 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \
  --apply_to full --print_every 5




[Phase] BASELINE (VPS OFF) — building model...
[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[Phase] model ready.


E0000 00:00:1757365444.191697    9345 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757365444.196898    9345 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757365444.210175    9345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757365444.210194    9345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757365444.210197    9345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757365444.210199    9345 computation_placer.cc:177] computation placer already registered. Please check linka

CalledProcessError: Command 'b'set -e\nexport PYTHONPATH=/content:/content/vps:$PYTHONPATH\nexport PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True\nexport PYTHONUNBUFFERED=1\n\npython -u vps/scripts/mc_logprob_eval.py \\\n  --task arc_c --n 80 --seed 1234 \\\n  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \\\n  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \\\n  --apply_to full --print_every 5\n\n'' returned non-zero exit status 1.

In [ ]:
%%bash
set -e
# Free up ~2.4 GiB that JAX often grabs on Colab/T4:
export JAX_PLATFORMS=cpu
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_MEM_FRACTION=.0

export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

# ARC-Challenge, n=80, full VPS
python -u vps/scripts/mc_logprob_eval.py \
  --task arc_c --n 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \
  --apply_to full --print_every 5


Process is terminated.


In [ ]:
%%bash
set -e
mkdir -p vps/scripts
cat > vps/scripts/mc_logprob_eval.py << 'PY'
from __future__ import annotations
import os, sys, re, json, time, gc, argparse, random
from typing import Dict, List, Tuple

# --- robust imports (Colab / script) ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.math_utils import compute_token_entropy

LETTER_SET = ["A","B","C","D","E"]

def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def build_cfg(model_name:str, dtype:str, apply_to:str, gamma:float, adaptive_gamma:bool)->VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"     # accelerate handles T4
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.max_new_tokens = 1
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    else:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]
    cfg.qk_coupling = True
    return cfg

def set_vps_enabled(model, enabled:bool):
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def robust_letter_id(tok, letter:str)->int:
    # try single-token encodings first
    cands = [
        letter,                 # "A"
        " " + letter,           # " A"
        "\n" + letter,          # "\nA"
    ]
    for s in cands:
        ids = tok(s, add_special_tokens=False)["input_ids"]
        if len(ids) == 1:
            return ids[0]
    # fallback: take the last token id (still works for next-token logit)
    ids = tok(" " + letter, add_special_tokens=False)["input_ids"]
    return ids[-1]

def load_arc(n:int, seed:int)->List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds))); random.Random(seed).shuffle(idx)
    idx = idx[:n]
    out = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]
        labels  = ex["choices"]["label"]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        out.append((prompt, gold))
    return out

def load_csqa(n:int, seed:int)->List[Tuple[str,str]]:
    ds = load_dataset("tau/commonsense_qa", split="validation")
    idx = list(range(len(ds))); random.Random(seed).shuffle(idx)
    idx = idx[:n]
    out = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = [c["text"] for c in ex["choices"]]
        labels  = [c["label"] for c in ex["choices"]]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        out.append((prompt, gold))
    return out

@torch.inference_mode()
def score_letters_logprob(model, tok, prompt_ids:Dict[str,torch.Tensor], letter_id_map:Dict[str,int]) -> Dict[str,float]:
    # ensure no KV cache to save memory
    model.config.use_cache = False
    out = model(**prompt_ids, use_cache=False)
    logits = out.logits[0, -1, :]  # next-token distribution
    scores = {L: float(logits[letter_id_map[L]].item()) for L in LETTER_SET}
    return scores

def vps_iterate_once(model, tok, prompt:str, gold_letter:str, tail_tokens:int, ce_T:int=6):
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)
    # entropy probe (no grad)
    with torch.inference_mode():
        out = model(**tail_inputs, use_cache=False)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except: pass
    # tiny CE on the letter token(s)
    gold_ids_full = tok(gold_letter, add_special_tokens=False, return_tensors=None)
    gold_ids_full = torch.tensor(gold_ids_full, dtype=torch.long, device=base_inputs["input_ids"].device)
    T = max(1, min(ce_T, gold_ids_full.shape[0]))
    targets = gold_ids_full[-T:].unsqueeze(0)
    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**tail_inputs, use_cache=False)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(logits2.reshape(-1, logits2.size(-1)),
                                               targets.reshape(-1))
        ce.backward()
    model.config.use_cache = old_cache

def run_task(task:str, n:int, seed:int, vps_on:bool,
             model_name:str, dtype:str, gamma:float, adaptive_gamma:bool,
             tail_tokens:int, iters:int, apply_to:str, print_every:int):
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, apply_to, gamma, adaptive_gamma)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)

    # enable/disable VPS effect
    set_vps_enabled(model, enabled=vps_on)

    sample_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            sample_gamma = m.cfg.gamma; break
    print("[VPS] Wrapped layers:", sum(1 for _ in model.modules() if _.__class__.__name__=="VPSLinear"))
    print(f"[diag] sample gamma = {sample_gamma}", flush=True)

    model.eval()
    model.config.use_cache = False

    # data
    if task == "arc_c":
        items = load_arc(n, seed)
    elif task == "csqa":
        items = load_csqa(n, seed)
    else:
        raise ValueError(f"Unknown task: {task}")

    # letter ids once
    letter_id_map = {L: robust_letter_id(tok, L) for L in LETTER_SET}

    correct, times = 0, []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=6)

            inputs = tok(prompt, return_tensors="pt", add_special_tokens=True).to(model.device)
            # no grad scoring
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            # fall back: shrink tail and try once more (or bypass VPS if already in baseline)
            if vps_on and tail_tokens > 8:
                try:
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=8, ce_T=4)
                    inputs = tok(prompt, return_tensors="pt").to(model.device)
                    scores = score_letters_logprob(model, tok, inputs, letter_id_map)
                    pred = max(scores.items(), key=lambda kv: kv[1])[0]
                except Exception:
                    inputs = tok(prompt, return_tensors="pt").to(model.device)
                    scores = score_letters_logprob(model, tok, inputs, letter_id_map)
                    pred = max(scores.items(), key=lambda kv: kv[1])[0]
            else:
                inputs = tok(prompt, return_tensors="pt").to(model.device)
                scores = score_letters_logprob(model, tok, inputs, letter_id_map)
                pred = max(scores.items(), key=lambda kv: kv[1])[0]

        dt = time.time() - t0
        times.append(dt)
        correct += int(pred == gold)

        if (i % max(1, print_every) == 0) or (i == n):
            eta = (sum(times)/len(times))*(n-i)
            print(f"[{task}] {i}/{n} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = correct / n if n else 0.0
    lat = sum(times)/max(1,len(times))
    res = {"task": task, "n": n, "acc": acc, "latency_mean_s": lat}
    print(json.dumps(res), flush=True)

    del model, tok
    free_cuda()
    return res

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", type=str, choices=["arc_c","csqa"], default="arc_c")
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--apply_to", type=str, choices=["full","attn"], default="full")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    # BASELINE
    base = run_task(args.task, args.n, args.seed, False,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)
    # VPS
    vps  = run_task(args.task, args.n, args.seed, True,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
# Free up ~2.4 GiB that JAX often grabs on Colab/T4:
export JAX_PLATFORMS=cpu
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_MEM_FRACTION=.0

export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

# ARC-Challenge, n=80, full VPS
python -u vps/scripts/mc_logprob_eval.py \
  --task arc_c --n 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \
  --apply_to full --print_every 5



[Phase] BASELINE (VPS OFF) — building model...
[VPS] Wrapped layers: 252
[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[arc_c] 5/80 acc_so_far=0.600 last_dt=0.36s ~ETA 0m39s
[arc_c] 10/80 acc_so_far=0.600 last_dt=0.35s ~ETA 0m30s
[arc_c] 15/80 acc_so_far=0.600 last_dt=0.35s ~ETA 0m26s
[arc_c] 20/80 acc_so_far=0.650 last_dt=0.34s ~ETA 0m23s
[arc_c] 25/80 acc_so_far=0.720 last_dt=0.40s ~ETA 0m21s
[arc_c] 30/80 acc_so_far=0.733 last_dt=0.34s ~ETA 0m19s
[arc_c] 35/80 acc_so_far=0.743 last_dt=0.37s ~ETA 0m17s
[arc_c] 40/80 acc_so_far=0.750 last_dt=0.36s ~ETA 0m15s
[arc_c] 45/80 acc_so_far=0.756 last_dt=0.35s ~ETA 0m13s
[arc_c] 50/80 acc_so_far=0.760 last_dt=0.35s ~ETA 0m11s
[arc_c] 55/80 acc_so_far=0.764 last_dt=0.35s ~ETA 0m9s
[arc_c] 60/80 acc_so_far=0.783 last_dt=0.47s ~ETA 0m7s
[arc_c] 65/80 acc_so_far=0.785 last_dt=0.35s ~ETA 0m5s
[arc_c] 70/80 acc_so_far=0.800 last_dt=0.35s ~ETA 0m3s
[arc_c] 75/80 acc_so_far=0.787 last_dt=0.37s ~ETA 0m1s
[arc_c] 80/80 acc_so_far=0.762 last_dt=0

E0000 00:00:1757366233.077714   12626 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757366233.106226   12626 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757366233.156884   12626 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366233.156968   12626 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366233.156984   12626 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366233.156996   12626 computation_placer.cc:177] computation placer already registered. Please check linka

CalledProcessError: Command 'b'set -e\n# Free up ~2.4 GiB that JAX often grabs on Colab/T4:\nexport JAX_PLATFORMS=cpu\nexport XLA_PYTHON_CLIENT_PREALLOCATE=false\nexport XLA_PYTHON_CLIENT_MEM_FRACTION=.0\n\nexport PYTHONPATH=/content:/content/vps:$PYTHONPATH\nexport PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True\nexport PYTHONUNBUFFERED=1\n\n# ARC-Challenge, n=80, full VPS\npython -u vps/scripts/mc_logprob_eval.py \\\n  --task arc_c --n 80 --seed 1234 \\\n  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \\\n  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \\\n  --apply_to full --print_every 5\n'' returned non-zero exit status 1.

In [ ]:
%%bash
set -e
mkdir -p vps/scripts
cat > vps/scripts/mc_logprob_eval.py << 'PY'
from __future__ import annotations
import os, sys, re, json, time, gc, argparse, random
from typing import Dict, List, Tuple

# --- robust imports ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

# calm logs + allocator
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.math_utils import compute_token_entropy

LETTER_SET = ["A","B","C","D","E"]

def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def build_cfg(model_name:str, dtype:str, apply_to:str, gamma:float, adaptive_gamma:bool)->VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.max_new_tokens = 1
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive_gamma)
    # keep VRAM under control on T4
    cfg.max_memory = {"cuda:0": "10GiB", "cpu": "64GiB"}
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    else:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]
    cfg.qk_coupling = True
    return cfg

def set_vps_enabled(model, enabled:bool):
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def slice_tail(inputs:Dict[str,torch.Tensor], tail:int)->Dict[str,torch.Tensor]:
    if tail <= 0: return inputs
    L = inputs["input_ids"].shape[1]
    if L <= tail: return inputs
    sl = slice(L - tail, L)
    out = {}
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor) and v.dim()==2 and v.size(1)==L:
            out[k] = v[:, sl]
        else:
            out[k] = v
    return out

def robust_letter_id(tok, letter:str)->int:
    """Return a single token id for this letter, trying common variants."""
    for s in (letter, " "+letter, "\n"+letter):
        ids = tok(s, add_special_tokens=False)["input_ids"]
        if len(ids)==1: return ids[0]
    ids = tok(" "+letter, add_special_tokens=False)["input_ids"]
    return ids[-1]

def robust_letter_ids(tok, letter:str)->List[int]:
    """Return a non-empty list of ids for the letter (never a string)."""
    for s in (letter, " "+letter, "\n"+letter):
        ids = tok(s, add_special_tokens=False)["input_ids"]
        if len(ids) >= 1: return ids
    # last resort: wrap single id
    return [robust_letter_id(tok, letter)]

def load_arc(n:int, seed:int)->List[Tuple[str,str]]:
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
    idx = list(range(len(ds))); random.Random(seed).shuffle(idx)
    idx = idx[:n]
    out = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = ex["choices"]["text"]; labels = ex["choices"]["label"]
        gold = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        out.append((prompt, gold))
    return out

def load_csqa(n:int, seed:int)->List[Tuple[str,str]]:
    ds = load_dataset("tau/commonsense_qa", split="validation")
    idx = list(range(len(ds))); random.Random(seed).shuffle(idx)
    idx = idx[:n]
    out = []
    for i in idx:
        ex = ds[i]
        q = ex["question"]
        choices = [c["text"] for c in ex["choices"]]
        labels  = [c["label"] for c in ex["choices"]]
        gold    = ex["answerKey"].strip().upper()
        opts = "\n".join([f"{l}) {t}" for l,t in zip(labels, choices)])
        prompt = (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {q}\n\nOptions:\n{opts}\n\nAnswer:"
        )
        out.append((prompt, gold))
    return out

@torch.inference_mode()
def score_letters_logprob(model, tok, prompt_ids:Dict[str,torch.Tensor], letter_id_map:Dict[str,int]) -> Dict[str,float]:
    model.config.use_cache = False
    out = model(**prompt_ids, use_cache=False)
    logits = out.logits[0, -1, :]
    return {L: float(logits[letter_id_map[L]].item()) for L in LETTER_SET}

def vps_iterate_once(model, tok, prompt:str, gold_letter:str, tail_tokens:int, ce_T:int=6):
    base_inputs = tok(prompt, return_tensors="pt").to(model.device)
    tail_inputs = slice_tail(base_inputs, tail=tail_tokens)

    # entropy probe
    with torch.inference_mode():
        out = model(**tail_inputs, use_cache=False)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except: pass

    # CE on gold letter ids (always ints)
    gold_ids = robust_letter_ids(tok, gold_letter)            # <- FIXED: list[int], never str
    gold_ids = torch.tensor(gold_ids, dtype=torch.long, device=base_inputs["input_ids"].device)
    T = max(1, min(ce_T, gold_ids.shape[0]))
    targets = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**tail_inputs, use_cache=False)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            targets.reshape(-1)
        )
        ce.backward()
    model.config.use_cache = old_cache

def run_task(task:str, n:int, seed:int, vps_on:bool,
             model_name:str, dtype:str, gamma:float, adaptive_gamma:bool,
             tail_tokens:int, iters:int, apply_to:str, print_every:int):
    set_seed(seed)
    cfg = build_cfg(model_name, dtype, apply_to, gamma, adaptive_gamma)

    phase = "VPS ON" if vps_on else "BASELINE (VPS OFF)"
    print(f"\n[Phase] {phase} — building model...", flush=True)
    tok, model, hooks = build(cfg)
    set_vps_enabled(model, enabled=vps_on)

    wrapped = sum(1 for _ in model.modules() if _.__class__.__name__=="VPSLinear")
    any_gamma = None
    for _, m in model.named_modules():
        if hasattr(m, "cfg") and hasattr(m.cfg, "gamma"):
            any_gamma = m.cfg.gamma; break
    print(f"[VPS] Wrapped layers: {wrapped}")
    print(f"[diag] sample gamma = {any_gamma}", flush=True)

    model.eval()
    model.config.use_cache = False

    items = load_arc(n, seed) if task=="arc_c" else load_csqa(n, seed)
    letter_id_map = {L: robust_letter_id(tok, L) for L in LETTER_SET}

    correct, times = 0, []
    for i, (prompt, gold) in enumerate(items, 1):
        t0 = time.time()
        try:
            if vps_on:
                for _ in range(max(1, iters)):
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=tail_tokens, ce_T=6)
            inputs = tok(prompt, return_tensors="pt", add_special_tokens=True).to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            if vps_on:
                try:
                    vps_iterate_once(model, tok, prompt, gold, tail_tokens=max(8, tail_tokens//2), ce_T=4)
                except Exception:
                    pass
            inputs = tok(prompt, return_tensors="pt").to(model.device)
            scores = score_letters_logprob(model, tok, inputs, letter_id_map)
            pred = max(scores.items(), key=lambda kv: kv[1])[0]

        dt = time.time() - t0
        times.append(dt)
        correct += int(pred == gold)

        if (i % max(1, print_every) == 0) or (i == n):
            eta = (sum(times)/len(times))*(n-i)
            print(f"[{task}] {i}/{n} acc_so_far={correct/i:.3f} last_dt={dt:.2f}s ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    acc = correct / n if n else 0.0
    lat = sum(times)/max(1,len(times))
    print(json.dumps({"task":task,"n":n,"acc":acc,"latency_mean_s":lat}), flush=True)

    del model, tok
    free_cuda()
    return {"task":task,"n":n,"acc":acc,"latency_mean_s":lat}

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", type=str, choices=["arc_c","csqa"], default="arc_c")
    ap.add_argument("--n", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", type=str, choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=2)
    ap.add_argument("--apply_to", type=str, choices=["full","attn"], default="full")
    ap.add_argument("--print_every", type=int, default=5)
    args, _ = ap.parse_known_args()

    base = run_task(args.task, args.n, args.seed, False,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)

    vps  = run_task(args.task, args.n, args.seed, True,
                    args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
                    args.tail_tokens, args.iters, args.apply_to, args.print_every)

if __name__ == "__main__":
    main()
PY


In [ ]:
%%bash
set -e
# free up stray JAX VRAM on Colab/T4
export JAX_PLATFORMS=cpu
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_MEM_FRACTION=.0

export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

python -u vps/scripts/mc_logprob_eval.py \
  --task arc_c --n 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 2 \
  --apply_to full --print_every 5



[Phase] BASELINE (VPS OFF) — building model...
[VPS] Wrapped layers: 252
[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[arc_c] 5/80 acc_so_far=0.600 last_dt=0.36s ~ETA 0m38s
[arc_c] 10/80 acc_so_far=0.600 last_dt=0.34s ~ETA 0m30s
[arc_c] 15/80 acc_so_far=0.600 last_dt=0.36s ~ETA 0m26s
[arc_c] 20/80 acc_so_far=0.650 last_dt=0.34s ~ETA 0m23s
[arc_c] 25/80 acc_so_far=0.720 last_dt=0.44s ~ETA 0m21s
[arc_c] 30/80 acc_so_far=0.733 last_dt=0.34s ~ETA 0m19s
[arc_c] 35/80 acc_so_far=0.743 last_dt=0.37s ~ETA 0m17s
[arc_c] 40/80 acc_so_far=0.750 last_dt=0.34s ~ETA 0m15s
[arc_c] 45/80 acc_so_far=0.756 last_dt=0.35s ~ETA 0m13s
[arc_c] 50/80 acc_so_far=0.760 last_dt=0.34s ~ETA 0m11s
[arc_c] 55/80 acc_so_far=0.764 last_dt=0.35s ~ETA 0m9s
[arc_c] 60/80 acc_so_far=0.783 last_dt=0.46s ~ETA 0m7s
[arc_c] 65/80 acc_so_far=0.785 last_dt=0.35s ~ETA 0m5s
[arc_c] 70/80 acc_so_far=0.800 last_dt=0.36s ~ETA 0m3s
[arc_c] 75/80 acc_so_far=0.787 last_dt=0.36s ~ETA 0m1s
[arc_c] 80/80 acc_so_far=0.762 last_dt=0

E0000 00:00:1757366544.812666   13948 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757366544.820173   13948 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757366544.854844   13948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366544.854873   13948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366544.854876   13948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757366544.854880   13948 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
%%bash
set -e
mkdir -p vps/scripts
cat > vps/scripts/confidence_filtered_eval.py << 'PY'
from __future__ import annotations
import os, sys, json, math, time, argparse, random, gc
from typing import Dict, List, Tuple

# --- robust imports and quiet logs ---
try:
    THIS_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    THIS_DIR = os.path.abspath(os.getcwd())
PROJ_ROOT   = os.path.abspath(os.path.join(THIS_DIR, ".."))
PROJ_PARENT = os.path.abspath(os.path.join(PROJ_ROOT, ".."))
for cand in {"/content", "/content/vps", PROJ_PARENT, PROJ_ROOT}:
    if cand not in sys.path and os.path.isdir(cand):
        sys.path.insert(0, cand)

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128,expandable_segments:True")
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

import torch
from datasets import load_dataset

from vps.scripts.infer_vps import build
from vpscore.config import VPSConfig
from vpscore.math_utils import compute_token_entropy

LETTERS = ["A","B","C","D","E"]

def set_seed(seed:int):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def free_cuda():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()

def build_cfg(model_name:str, dtype:str, apply_to:str, gamma:float, adaptive:bool)->VPSConfig:
    cfg = VPSConfig()
    cfg.model_name = model_name
    cfg.device_map = "auto"
    cfg.dtype = dtype
    cfg.torch_dtype_str = dtype
    cfg.temperature = 0.0
    cfg.top_p = 1.0
    cfg.top_k = 0
    cfg.max_new_tokens = 1
    cfg.gamma = float(gamma)
    cfg.adaptive_gamma = bool(adaptive)
    cfg.max_memory = {"cuda:0": "10GiB", "cpu": "64GiB"}
    if apply_to == "attn":
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj"]
    else:
        cfg.apply_to = ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]
    cfg.qk_coupling = True
    return cfg

def set_vps_enabled(model, enabled:bool):
    for m in model.modules():
        if m.__class__.__name__ == "VPSLinear":
            if not hasattr(m, "_saved_gamma"):
                m._saved_gamma = float(getattr(m.cfg, "gamma", 0.0))
            m.cfg.gamma = float(m._saved_gamma) if enabled else 0.0

def robust_letter_id(tok, letter:str)->int:
    for s in (letter, " "+letter, "\n"+letter):
        ids = tok(s, add_special_tokens=False)["input_ids"]
        if len(ids)==1: return ids[0]
    ids = tok(" "+letter, add_special_tokens=False)["input_ids"]
    return ids[-1]

def robust_letter_ids(tok, letter:str)->List[int]:
    for s in (letter, " "+letter, "\n"+letter):
        ids = tok(s, add_special_tokens=False)["input_ids"]
        if len(ids) >= 1: return ids
    return [robust_letter_id(tok, letter)]

def load_pool(task:str, n_pool:int, seed:int)->List[Tuple[str,str]]:
    if task=="arc_c":
        ds = load_dataset("ai2_arc", "ARC-Challenge", split="validation")
        get = lambda i: (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {ds[i]['question']}\n\nOptions:\n" +
            "\n".join([f"{l}) {t}" for l,t in zip(ds[i]["choices"]["label"], ds[i]["choices"]["text"])]) +
            "\n\nAnswer:"
            , ds[i]["answerKey"].strip().upper()
        )
        total = len(ds)
    else:
        ds = load_dataset("tau/commonsense_qa", split="validation")
        get = lambda i: (
            "You are a careful reasoning assistant.\n"
            "Pick the BEST option. Output **one letter only** (A/B/C/D/E).\n\n"
            f"Question: {ds[i]['question']}\n\nOptions:\n" +
            "\n".join([f"{c['label']}) {c['text']}" for c in ds[i]["choices"]]) +
            "\n\nAnswer:"
            , ds[i]["answerKey"].strip().upper()
        )
        total = len(ds)

    idx = list(range(total)); random.Random(seed).shuffle(idx)
    idx = idx[:n_pool]
    return [get(i) for i in idx]

@torch.inference_mode()
def phrase_scores(model, tok, prompt:str, phrase_for:Dict[str,str]) -> Dict[str,float]:
    model.config.use_cache = False
    dev = next(model.parameters()).device
    p = tok(prompt, return_tensors="pt", add_special_tokens=True)
    p = {k: v.to(dev) for k,v in p.items()}
    prefix = p["input_ids"][0]

    out_scores = {}
    for L, phrase in phrase_for.items():
        t = tok(phrase, add_special_tokens=False, return_tensors="pt")
        t_ids = t["input_ids"][0].to(dev)
        full = torch.cat([prefix, t_ids], dim=0).unsqueeze(0)
        attn = torch.ones_like(full, dtype=torch.long, device=dev)
        out = model(input_ids=full, attention_mask=attn, use_cache=False)
        logits = out.logits[0]
        T = t_ids.shape[0]
        start = prefix.shape[0] - 1
        logp = torch.log_softmax(logits[start:start+T, :], dim=-1)
        tok_logp = logp[torch.arange(T, device=dev), t_ids]
        out_scores[L] = float(tok_logp.sum().item())
    return out_scores

def vps_iter_once(model, tok, prompt:str, gold_letter:str, tail_tokens:int, ce_T:int=6):
    dev = next(model.parameters()).device
    base = tok(prompt, return_tensors="pt").to(dev)
    # tail
    L = base["input_ids"].shape[1]
    if tail_tokens > 0 and L > tail_tokens:
        sl = slice(L - tail_tokens, L)
        tail = {k:(v[:,sl] if v.dim()==2 and v.size(1)==L else v) for k,v in base.items()}
    else:
        tail = base
    # entropy probe
    with torch.inference_mode():
        out = model(**tail, use_cache=False)
        ent = compute_token_entropy(out.logits[0, -1, :])
    for m in model.modules():
        if hasattr(m, "policy") and m.policy is not None:
            try: m.policy.set_token_entropy(float(ent))
            except: pass
    # tiny CE on gold letter tokens
    gold_ids = torch.tensor(robust_letter_ids(tok, gold_letter), dtype=torch.long, device=dev)
    T = max(1, min(ce_T, gold_ids.shape[0]))
    targets = gold_ids[-T:].unsqueeze(0)

    old_cache = getattr(model.config, "use_cache", True)
    model.config.use_cache = False
    model.zero_grad(set_to_none=True)
    with torch.enable_grad():
        out2 = model(**tail, use_cache=False)
        logits2 = out2.logits[:, -T:, :]
        ce = torch.nn.functional.cross_entropy(
            logits2.reshape(-1, logits2.size(-1)),
            targets.reshape(-1)
        )
        ce.backward()
    model.config.use_cache = old_cache

def margin_from_scores(scores:Dict[str,float])->float:
    # top1 - top2
    vals = sorted(scores.values(), reverse=True)
    return float(vals[0] - vals[1]) if len(vals)>=2 else 0.0

def mcnemar_chi2(b:int, c:int)->float:
    if b+c == 0: return 0.0
    return (abs(b-c)-1)**2 / (b+c)

def run(task:str, n_pool:int, n_select:int, seed:int,
        model_name:str, dtype:str, gamma:float, adaptive:bool,
        tail_tokens:int, iters:int, apply_to:str, phrase_template:str):

    set_seed(seed)
    phrase_for = {L: phrase_template.replace("{L}", L) for L in LETTERS}

    # -------- Baseline model (no VPS) --------
    print("\n[Phase] BASELINE (VPS OFF) — building model...", flush=True)
    cfg0 = build_cfg(model_name, dtype, apply_to, gamma, adaptive)
    tok0, model0, hooks0 = build(cfg0)
    set_vps_enabled(model0, False)
    wrapped0 = sum(1 for _ in model0.modules() if _.__class__.__name__=="VPSLinear")
    any_gamma0 = None
    for _,m in model0.named_modules():
        if hasattr(m,"cfg") and hasattr(m.cfg,"gamma"): any_gamma0 = m.cfg.gamma; break
    print(f"[VPS] Wrapped layers: {wrapped0}")
    print(f"[diag] sample gamma = {any_gamma0}", flush=True)
    model0.eval(); model0.config.use_cache=False

    pool = load_pool(task, n_pool, seed)

    # First pass: baseline phrase scores for uncertainty selection
    pool_recs = []
    for i,(prompt,gold) in enumerate(pool,1):
        sc = phrase_scores(model0, tok0, prompt, phrase_for)
        pred = max(sc.items(), key=lambda kv: kv[1])[0]
        marg = margin_from_scores(sc)
        pool_recs.append((prompt, gold, sc, pred, marg))
        if i % 20 == 0 or i==len(pool):
            print(f"[select] scored {i}/{len(pool)}", flush=True)

    # select most uncertain n_select (smallest margin)
    pool_recs.sort(key=lambda r: r[4])
    sel = pool_recs[:n_select]

    # Baseline accuracy on selected
    base_correct = sum(int(r[3]==r[1]) for r in sel)
    base_acc = base_correct / max(1,len(sel))
    print(json.dumps({"phase":"baseline","task":task,"n":len(sel),"acc":base_acc}), flush=True)

    del model0, tok0
    free_cuda()

    # -------- VPS model (full power) --------
    print("\n[Phase] VPS ON — building model...", flush=True)
    cfg1 = build_cfg(model_name, dtype, apply_to, gamma, adaptive)
    tok1, model1, hooks1 = build(cfg1)
    set_vps_enabled(model1, True)
    wrapped1 = sum(1 for _ in model1.modules() if _.__class__.__name__=="VPSLinear")
    any_gamma1 = None
    for _,m in model1.named_modules():
        if hasattr(m,"cfg") and hasattr(m.cfg,"gamma"): any_gamma1 = m.cfg.gamma; break
    print(f"[VPS] Wrapped layers: {wrapped1}")
    print(f"[diag] sample gamma = {any_gamma1}", flush=True)
    model1.eval(); model1.config.use_cache=False

    b_to_v, v_to_b = 0, 0
    vps_correct = 0
    t0 = time.time()
    for i,(prompt,gold,sc_base,pred_base,_) in enumerate(sel,1):
        try:
            for _ in range(max(1,iters)):
                vps_iter_once(model1, tok1, prompt, gold, tail_tokens=tail_tokens, ce_T=6)
        except torch.cuda.OutOfMemoryError:
            free_cuda()
            try:
                vps_iter_once(model1, tok1, prompt, gold, tail_tokens=max(8, tail_tokens//2), ce_T=4)
            except Exception:
                pass

        sc_vps = phrase_scores(model1, tok1, prompt, phrase_for)
        pred_vps = max(sc_vps.items(), key=lambda kv: kv[1])[0]
        vps_correct += int(pred_vps==gold)
        if pred_base!=gold and pred_vps==gold: b_to_v += 1
        if pred_base==gold and pred_vps!=gold: v_to_b += 1

        if i%10==0 or i==len(sel):
            eta = (time.time()-t0)/i*(len(sel)-i)
            print(f"[eval] {i}/{len(sel)} done ~ETA {int(eta//60)}m{int(eta%60)}s", flush=True)

    vps_acc = vps_correct / max(1,len(sel))
    chi2 = mcnemar_chi2(b_to_v, v_to_b)

    print(json.dumps({
        "phase":"vps",
        "task":task,
        "n":len(sel),
        "acc":vps_acc,
        "delta_acc": vps_acc - base_acc,
        "flips_to_correct": b_to_v,
        "flips_to_wrong": v_to_b,
        "mcnemar_chi2": chi2
    }), flush=True)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--task", choices=["arc_c","csqa"], default="arc_c")
    ap.add_argument("--n_pool", type=int, default=240)
    ap.add_argument("--n_select", type=int, default=80)
    ap.add_argument("--seed", type=int, default=1234)
    ap.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-3B-Instruct")
    ap.add_argument("--dtype", choices=["fp16","bf16","fp32"], default="fp16")
    ap.add_argument("--gamma", type=float, default=0.65)
    ap.add_argument("--adaptive_gamma", action="store_true")
    ap.add_argument("--tail_tokens", type=int, default=24)
    ap.add_argument("--iters", type=int, default=3)
    ap.add_argument("--apply_to", choices=["full","attn"], default="full")
    ap.add_argument("--phrase_template", type=str, default=" Answer: {L}")
    args, _ = ap.parse_known_args()

    run(args.task, args.n_pool, args.n_select, args.seed,
        args.model_name, args.dtype, args.gamma, args.adaptive_gamma,
        args.tail_tokens, args.iters, args.apply_to, args.phrase_template)

if __name__ == "__main__":
    main()
PY


In [ ]:


#Experiments results:
%%bash
set -e
# free stray JAX VRAM on Colab/T4
export JAX_PLATFORMS=cpu
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_MEM_FRACTION=.0

export PYTHONPATH=/content:/content/vps:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128,expandable_segments:True
export PYTHONUNBUFFERED=1

# ARC-Challenge, pool=240 → select 80 hardest-by-margin
python -u vps/scripts/confidence_filtered_eval.py \
  --task arc_c --n_pool 240 --n_select 80 --seed 1234 \
  --model_name "Qwen/Qwen2.5-3B-Instruct" --dtype fp16 \
  --gamma 0.65 --adaptive_gamma --tail_tokens 24 --iters 3 \
  --apply_to full --phrase_template " Answer: {L}"



[Phase] BASELINE (VPS OFF) — building model...
[VPS] Wrapped layers: 252
[VPS] Wrapped layers: 252
[diag] sample gamma = 0.0
[select] scored 20/240
[select] scored 40/240
[select] scored 60/240
[select] scored 80/240
[select] scored 100/240
[select] scored 120/240
[select] scored 140/240
[select] scored 160/240
[select] scored 180/240
[select] scored 200/240
[select] scored 220/240
[select] scored 240/240
{"phase": "baseline", "task": "arc_c", "n": 80, "acc": 0.5125}

[Phase] VPS ON — building model...
[VPS] Wrapped layers: 252
[VPS] Wrapped layers: 252
[diag] sample gamma = 0.65
[eval] 10/80 done ~ETA 5m37s
[eval] 20/80 done ~ETA 4m48s
[eval] 30/80 done ~ETA 4m1s
[eval] 40/80 done ~ETA 3m12s
[eval] 50/80 done ~ETA 2m24s
[eval] 60/80 done ~ETA 1m36s
[eval] 70/80 done ~ETA 0m48s
[eval] 80/80 done ~ETA 0m0s
{"phase": "vps", "task": "arc_c", "n": 80, "acc": 0.5375, "delta_acc": 0.025000000000000022, "flips_to_correct": 6, "flips_to_wrong": 4, "mcnemar_chi2": 0.1}


E0000 00:00:1757367528.894703   17965 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757367528.919549   17965 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757367528.984725   17965 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757367528.984820   17965 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757367528.984834   17965 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757367528.984846   17965 computation_placer.cc:177] computation placer already registered. Please check linka